In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import nltk
import re
#import datasets
from tqdm import tqdm
import torch
import time
import spacy
from nltk.corpus import stopwords
nltk.download("stopwords")
from heapq import nlargest

from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# NOT RUNNING CODE

In [2]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [3]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [33]:
filename = 'input/AFD_Wahlprogramm_2021.txt'

In [34]:
with open(filename, encoding="utf8") as f:
    text = f.read()
    #text = text.replace(r'\\d+\\n', '$')
    #text=   re.sub("\n\n", ".", text)
    #text=   re.sub("\d+.", ".", text)
    text=   re.sub(" \d+\n", ".", text)
    text=   re.sub("\n\d+", " ", text)
    text=   re.sub("\n", " ", text)
sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)

In [63]:
def summarize_text(input_text: list, percentage):
    spacy_model = spacy.load("de_core_news_sm")
    german_stop_words = stopwords.words('german')
    text_list = [word.lower() for word in input_text if word not in german_stop_words and word not in [",", ".", " ", ":", "'", '"']]

    #print(text_list)

    text = spacy_model(list_to_string(text_list))
    tokens  = [token.text for token in text]
    freq_words = {}

    for word in text:
        if word not in freq_words.keys():
            freq_words[word.text] = 1
        else:
            freq_words[word.text] += 1

    max_freq = max(freq_words.values())

    for word in freq_words.keys():
        freq_words[word] = freq_words[word]/max_freq

    print(freq_words)
    
    sentence_tokens = [sent for sent in text.sents]
    sentence_scores = {}

    for sent in sentence_tokens:
        for word in sent:
            if word in freq_words.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = freq_words[word.text]
                else:
                    sentence_scores[sent] += freq_words[word.text]
    
    #print(sentence_scores)
    
    select_length = int(len(sentence_tokens)*percentage)
    result = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    final_summary = [word.text for word in result]
    sum =  ''.join(final_summary)
    return sum

In [64]:
sum = summarize_text(sentences, 0.05)
sum

{'deutschland': 1.0, '.': 1.0, 'aber': 1.0, 'normal': 1.0, 'programm': 1.0, 'der': 1.0, 'alternative': 1.0, 'für': 1.0, 'die': 1.0, 'wahl': 1.0, 'zum': 1.0, '20.': 1.0, 'deutschen': 1.0, 'bundestag': 1.0, ' ': 1.0, 'ad': 1.0, 'inhaltsverzeichnis': 1.0, 'demokratie': 1.0, 'und': 1.0, 'rechtsstaat': 1.0, 'das': 1.0, 'volk': 1.0, 'ist': 1.0, 'souverän': 1.0, 'volksabstimmungen': 1.0, 'nach': 1.0, 'schweizer': 1.0, 'modell': 1.0, 'gewaltenteilung': 1.0, 'gewährleisten': 1.0, 'entpolitisierung': 1.0, 'justiz': 1.0, 'verfassungsschutz': 1.0, 'reformieren': 1.0, 'trennung': 1.0, 'von': 1.0, 'amt': 1.0, 'mandat': 1.0, 'kampf': 1.0, 'ämterpatronage': 1.0, 'macht': 1.0, 'parteien': 1.0, 'beschränken': 1.0, 'freie': 1.0, 'listenwahl': 1.0, 'freies': 1.0, 'verkleinerung': 1.0, 'parlamente': 1.0, 'berufspolitikertum': 1.0, ':': 1.0, 'mandats-': 1.0, 'amtszeiten': 1.0, 'begrenzen': 1.0, 'lobbyismus': 1.0, 'eindämmen': 1.0, 'freiheit': 1.0, 'verantwortung': 1.0, 'bürgerlich-freiheitliche': 1.0, 'rech

''

In [11]:
sentences = sentences[:len(sentences)//8]
len(sentences)

32

In [12]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed")

In [13]:
print("Inputs")
inputs = tokenizer(sentences, return_tensors='pt', padding=True)

Inputs


In [15]:
print("Predictions Model")
start_time = time.time()
prediction = model.generate(**inputs)
end_time = time.time()
print("Zeit: " + end_time-start_time)

Attention type 'block_sparse' is not possible if sequence_length: 30 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Predictions Model


c:\Users\nikla\anaconda3\envs\NLP-New\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print("Predictions Batch Decode")
prediction = tokenizer.batch_decode(prediction)

# RUNNING CODE

In [3]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [4]:
text1 = """Wir halten die unmittel-
bare Demokratie für ein
unverzichtbares Mittel,
um dem autoritären
und teilweise totalitären
Gebaren der
Regierungspolitiker
Einhalt zu gebieten.

Demokratie und Rechtsstaat

Demokratie und Rechtsstaat

Das Volk ist der Souverän

Die Regierungspolitiker in Bund und Ländern haben mit
ihrer Flüchtlings-, Europa- und Corona-Politik die
Prinzipien der deutschen Staatlichkeit, des Rechts und
der Verfassung vielfach verletzt.

Zugleich haben sich die Volksvertreter der etablierten
Parteien den grundgesetzlich garantierten Parlaments-
vorbehalt für alle wichtigen Entscheidungen im Staat
ohne Widerstand nehmen lassen.

Die Bundesregierung kommt ihrer Pflicht, Vertrags-
brüchen und Selbstermächtigungen durch
EU-Institutionen entgegenzutreten, nicht nach. Einzelne
rechts- und verfassungswidrige Mafsnahmen wurden
zwar durch mutige Richter in Hunderten von Urteilen zu
Fall gebracht. In unserem Land hat sich aber eine
politische Klasse herausgebildet, deren vordringliches
Interesse ihrer Macht, ihrem Status und ihrem
materiellen Wohlergehen gilt. Diese setzt die soziale und
kulturelle Zukunft unseres Volkes, die Stärke unserer
Wirtschaft und damit unseres Wohlstandes aufs Spiel
und stellt Multikulturalität, Diversität, Globalisierung
und vermeintliche Gendergerechtigkeit über alles.

Sie hält die Schalthebel der staatlichen Macht, der
politischen Bildung und des informationellen und
medialen Einflusses auf die Bevölkerung in Händen.

Deshalb halten wir die unmittelbare Demokratie für ein
unverzichtbares Mittel, um dem autoritären und
teilweise totalitären Gebaren der Regierungspolitiker
Einhalt zu gebieten.

Demokratie und Rechtsstaat 13

Volksabstimmungen nach Schweizer Modell

Die AfD fordert Volksentscheide nach Schweizer Modell
auch für Deutschland. Die uneingeschränkte
Volkssouveränität in ihrer fast 200 Jahre bewährten und
optimierten Gestaltung hat dem eidgenössischen Bun-
desstaat eine fortwährende Spitzenstellung in
Wohlstand, Frieden und Freiheit gewährleistet.

Durch die Volksabstimmungen wollen wir die Flut der
oftmals unsinnigen Gesetzesvorlagen eindammen und
die Parlamente zu sorgfältiger Arbeit zwingen. Unange-
messenen Beschlüssen der Abgeordneten in eigener
Sache wollen wir einen Riegel vorschieben. Hierzu
zahlen etwa die Diäten, Fraktions- und Parteien-
finanzierung sowie die Finanzierung von „parteinahen'
Stiftungen. Der Wortbruch der etablierten Parteien be-
züglich des Euros und die immer weitere Zentralisierung
der EU bedrohen die Zukunft unseres Landes (sowie der
nachfolgenden Generationen). Diese fatale Politik wollen

wir beenden. Die AfD fordert
Volksentscheide nach

. oe . Schweizer Modell auch
Die Einführung von Volksabstimmungen nach für Deutschland.

Schweizer Modell ist für die AfD nicht verhandelbarer a
. . ..: . . Die Einführung von
Inhalt jeglicher Koalitionsvereinbarungen und beinhaltet yonsansiimmungen

insbesondere folgende Elemente: nach Schweizer Modell
ist für die AfD nicht
verhandelbarer Inhalt

Ohne Zustimmung des Volkes darf das Grundgesetz jeglicher Koalitions-

nicht geändert und kein bedeutsamer völkerrechtlicher nbarungen
Vertrag geschlossen werden. Wir wollen dem Volk das

Recht geben, den Abgeordneten auf die Finger zu

schauen und vom Parlament beschlossene Gesetze zu

andern oder abzulehnen.

14

In Schicksalsfragen
der Nation entscheiden
Bürger weitsichtiger,
friedfertiger und
gemeinwohl-
orientierter als
Berufspolitiker.

Demokratie und Rechtsstaat

Das Volk soll die Möglichkeit erhalten, Gesetzesinitia-
tiven einzubringen und per Volksabstimmung zu be-
schliefsen. Hierbei soll es jenseits des Art. 79 Abs. 3 GG
keine thematischen Beschränkungen geben.

Die Qualität der Entscheidung der Bürger muss
gesichert werden durch einen ausgeprägten und auto-
nomen Prozess der Willensbildung, durch Informations-
pflichten und -rechte und eine breite gesellschaftliche
Debatte. Auch die Parlamente sollen die Möglichkeit
erhalten, eine Volksabstimmung zu initiieren.

Als einzige der im Bundestag vertretenen Parteien
halten wir das deutsche Volk für ebenso mündig wie das
der Schweizer. Entgegen anderslautenden
Behauptungen entscheiden Bürger in Schicksalsfragen
der Nation weitsichtiger, friedfertiger und gemeinwohl-
orientierter als Berufspolitiker. Das ist die Lehre aus der
Wiedervereinigung, aus der Ablehnung des
„EU-Verfassungsvertrags durch das französische und
das niederländische Volk und der Entscheidung des
Vereinigten Königreichs, aus der Europäischen Union
auszutreten.

Wir vertrauen nicht mehr darauf, dass Regierungen und
Parlamente zu Währungskrisen, Migration,
Islamisierung oder zur Energiewende tragfähige Lösun-
gen finden. Das Volk als Souverän muss in direkter
Mitbestimmung Träger solcher schicksalhaften
Entscheidungen sein.

Demokratie und Rechtsstaat 15

Die Gewaltenteilung gewährleisten

Zahlreiche Gesetze und die politische Praxis haben die

Gewaltenteilung in Deutschland als Kernelement des
Rechtsstaats über die Jahre in Gefahr gebracht und zu
einer überbordenden Staatsgewalt geführt: Ehemalige
Politiker auf Richterstühlen, Abgeordnete, die zugleich
Kanzler, Ministerpräsident oder Minister sind und sich
insoweit selbst kontrollieren, sowie parteipolitische
Netzwerke, die durch verbotene, verfassungswidrige
Ämterpatronage entstehen, können nicht länger
toleriert werden. Öffentliche Ämter sind ausschließlich
nach „Eignung, Befähigung und fachlicher Leistung’ zu
vergeben (Art. 33 Abs.2 GG).

Entpolitisierung der Justiz

Die AfD will die Einflussnahme der politischen Parteien

auf die Ernennung von Richtern und Staatsanwälten
beenden. Auch wollen wir die Praxis ändern, dass die

Staatsanwaltschaft weisungsgebunden und im Einzelfall

dem Justizminister berichtspflichtig ist. Die Unabhän-
gigkeit der dritten Gewalt muss durch eine Selbstver-
waltung der Justiz ausgebaut werden, wie esin vielen
anderen europäischen Ländern bereits üblich ist. Wir

unterstützen daher den Modellvorschlag des Deutschen

Richterbundes, einen Justizwahlausschuss und einen
Justizverwaltungsrat einzurichten. Insbesondere die
Verfassungsgerichte und Rechnungshöfe sind vor
parteipolitischer Einwirkung zu schützen.

Die AfD will die Einfluss-
nahme der politischen
Parteien auf die Ernen-
nung von Richtern und
Staatsanwälten
beenden.
Insbesondere die
Verfassungsgerichte
und Rechnungshöfe
sind vor partei-
politischer Einwirkung
zu schützen.

16

Die AfD fordert:

Die Mitarbeiter der
öffentlichen Verwaltung
haben unabhängig und
loyal ihren Dienst zu
verrichten.

Beamten- und Richter-
stellen sind ausschliejs-
lich nach Qualifikation
zu besetzen.

Demokratie und Rechtsstaat

Verfassungsschutz reformieren

Der behördlich organisierte „Verfassungsschutz” muss
schnellstmöglich auf den Prüfstand gestellt und so
reformiert werden, dass er in Zukunft nicht mehr als
parteipolitisches Instrument gegen politische Gegner
missbraucht werden kann.

Trennung von Amt und Mandat und
Kampf der Ämterpatronage

Kanzler, Ministerpräsidenten und Minister dürfen nicht
mehr zugleich Abgeordnete sein. Die Institution der
parlamentarischen Staatssekretäre ist abzuschaffen.
Dies gilt auch für das Institut der „politischen Beamten’
in der Staatsverwaltung - vom Ministerialdirektor bis
zum Polizeipräsidenten. Die Mitarbeiter der Öffentlichen
Verwaltung haben unabhängig und loyal ihren Dienst
zu verrichten. Beamten- und Richterstellen sind aus-
schliefslich nach Qualifikation zu besetzen. Die „Partei-
buchwirtschaft muss beendet werden.

Grundsätze der fachlichen Eignung und beruflichen
Qualifikation haben auch für das Spitzenpersonal des
Staates in den Kabinetten zu gelten. Parteipolitische
Karrieren können fachliche Defizite nicht ausgleichen.

Macht der Parteien beschränken

Die Parteien sollen am politischen System mitwirken
(Art. 21, Abs.1GG), es aber nicht beherrschen. Die All-
macht der Parteien und deren „Ausbeutung des Staates‘
gefährden unsere Demokratie. Diese politische
Instrumentalisierung des Staates ist auch Ursache der
verbreiteten Politikverdrossenheit.

Demokratie und Rechtsstaat 17

Die Loyalität der Abgeordneten muss den Bürgern und
dem Staat gelten, nicht den Parteivorständen.

Neben der offiziellen Parteienfinanzierung in Höhe von

mittlerweile jährlich ca. 194 Millionen Euro haben sich

die Parteien zusätzliche Finanzausstattung verschafft.

Hervorzuheben ist dabei die Finanzierung so-

genannter politischer Stiftungen, denen alleine 600

Millionen Euro aus Mitteln des Bundeshaushalts jahrlich ea les:
zuflieisen. Ferner halten wir die jedem Bundestagsabge-  siaatliche Parteienfi-
ordneten zustehende „Mitarbeiterpauschale” von 22436 nanzierung neu regeln
Euro monatlich für überhöht. Ob dabei in jedem Fall a
wissenschaftlich qualifiziertes Personal oder Partei- Finanzierung von
soldaten angestellt werden, darf hinterfragt werden. - nn en ganz
Die AfD will die gesamte staatliche Parteienfinanzierung

neu regeln und deutlich begrenzen, sowie die staatliche

Finanzierung von Parteistiftungen ganz abschaffen.

Alle Mittel der Parteienfinanzierung sollen entgegen

derzeitiger Praxis der unbeschränkten Kontrolle der

Rechnungshöfe unterliegen. Unabdingbar ist auch eine

restriktive und Korruption vermeidende Neuordnung

der Spendenregelungen. Ferner soll den Parteien

endlich die Beteiligung an Unternehmen, insbesondere
Medienunternehmen, verboten werden.

Freie Listenwahl und freies Mandat

Die politische Klasse Deutschlands hat das Wahlrecht
und die Wahlverfahren im Lauf der Zeitimmer besser
ausgenutzt und angepasst, um den Einfluss des Volkes
auf die Bewerberauswahl zu minimieren. Dem Wähler
werden derzeit starre Wahllisten der Parteien zum
Ankreuzen vorgelegt.

13

Um unser Ideal des
Bürgerabgeordneten zu
verwirklichen,

fordern wir eine Begren-
zung der Mandatszeit
für Abgeordnete auf vier
und der Amtszeit für
den Bundeskanzler

auf zwei Legislatur-
perioden.

Demokratie und Rechtsstaat

Über die „sicheren Listenplätze“ bestimmen de facto die
Parteivorstände die Zusammensetzung des Parlaments.
Mit einer demokratischeren Gestaltung des Wahlsys-
tems will die AfD dem Wähler die Entscheidung über die
personelle Zusammensetzung der Parlamente zurück-
geben.

Deshalb treten wir für die „freie Listenwahl” ein, mit der
Möglichkeit des Kumulierens, Panaschierens und Strei-
chens von Kandidaten.

Verkleinerung der Parlamente

Mit seinen mittlerweile 709 Abgeordneten leistet sich
Deutschland eines der gröfsten Parlamente der Welt.
Die im Oktober 2020 von CDU/CSU und SPD durchge-
setzte Wahlrechtsänderung entspricht nicht der vom
Bundesverfassungsgericht angemahnten Reform der
Wahl zum Bundestag. Mit etwa 2.500 Abgeordneten in
Bund und Ländern und dem dazugehörigen
Verwaltungsapparat hat Deutschland übergrofse
Parlamente. Wir streben an, den Bundestag um etwa
die Hälfte zu verkleinern.

Berufspolitikertum:
Mandats- und Amtszeiten begrenzen

Die ungebrochene Tendenz zum Berufspolitikertum hat
der Monopolisierung der Macht Vorschub geleistet und
die unübersehbare Kluft zwischen Wählern und
Gewählten stetig vergröfsert. Vetternwirtschaft, Filz,
korruptionsfördernde Strukturen und Lobbyismus sind
die Folge.

Demokratie und Rechtsstaat 19

Um unser Ideal des Bürgerabgeordneten zu verwirkli-

chen, fordern wir eine Begrenzung der Mandatszeit für
Abgeordnete auf vier und der Amtszeit für den Bundes-
kanzler auf zwei Legislaturperioden. Nicht die Abgeord- __

netenvergütung, sondern politischer Idealismus soll Wir wollen die Rege-
lungen hinsichtlich der

Motivation zur Kandidatur für ein Nebentätigkeiten von

Mandat sein. Parlamentariern
deutlich verschärfen,
um Bestechung zu

Wir wollen, dass Bundestagsabgeordnete ihre volle
Arbeitskraft der parlamentarischen Arbeit widmen.
Das Mandat darf nicht unter bezahlten Nebentätigkeiten
leiden. Der überbordende Lobbyismus in Brüssel und
Berlin muss eingedämmt werden. Wir wollen die
Regelungen hinsichtlich der Nebentätigkeiten von
Parlamentariern deutlich verschärfen, um Bestechung
zu verhindern. Unseren eigenen Abgeordneten haben
wir bereits jetzt über unsere Satzung entsprechende
Regeln auferlegt. Ferner setzen wir uns für ein
Lobbyisten-Gesetz mit konkreten Rechten, Pflichten
und Sanktionen für Mandatsträger und Lobbyisten ein,
das vorbildlichen Regelungen anderer Demokratien
entspricht.
"""

In [5]:
text2 = """Die Hilfe für Menschen
in Not muss von der
Aufnahme in
Deutschland entkoppelt
und in den Krisen-
regionen selbst geleistet
werden.

Migration, Asyl und Integration

Migration, Asyl und Integration:
Human und rational handeln -
Deutschland schützen

Die Fluchtbewegungen des 21. Jahrhunderts können
nicht mittels Aufnahme der Migranten in Europa und
Deutschland bewältigt werden

Noch nie gab es global mehr Flüchtlinge alsin den
letzten Jahren. Neben kriegerischen Auseinanderset-
zungen und staatlichen Verfolgungshandlungen spielt
insbesondere das enorme Bevölkerungswachstum auf
dem afrikanischen Kontinent eine überragende Rolle.
Da dies mit Armut und dem Kampf ums tägliche Über-
leben verbunden ist, wächst der Migrationsdruck auf
Europa in besonderer Weise. Entgegen der Intention des
UN-Migrationspakts kann dieses globale Problem nicht
durch Migration gelöst werden. Selbst kleinere Teile
dieser Entwicklung, die sich in Millionen von Menschen
ausdrücken lassen, Können weder ökonomisch, noch
sozial- und integrationspolitisch in Europa aufgefangen
werden. Deutschland und die anderen europäischen
Länder würden destabilisiert, ohne dass ein wirkungs-
voller Beitrag zur Lösung des globalen Problems geleistet
wird.

Asylmissbrauch beenden

Das derzeitige Asylsystem ist dysfunktional und gleicht
einer Lotterie. Finanzielle Fehlanreize und unterlassene
Abschiebungen führen zu illegaler Migration, Schleuser-
kriminalität und zahlreichen Todesopfern im Mittelmeer.
Die meisten dieser Migranten begeben sich in der

Migration, AsylundIntegration 91

Hoffnung aufein besseres Leben auf den Weg nach
Europa. Die Hilfe für Menschen in Not muss jedoch von
der Aufnahme in Deutschland entkoppelt und in den
Krisenregionen selbst geleistet werden, wo dies weit
effektiver ist und viel mehr wirklich Hilfsbedürftige
erreicht.

Die unlösbaren Folgeprobleme

Die desaströsen Folgen der unregulierten Massenzu-

wanderung seit 2015 sind unübersehbar und verschär-

fen sich weiter: überproportionale Zuwandererkrimi-

nalität, Terroranschläge und islamischer Separatismus

belegen dies ebenso wie dreistellige Milliardenkosten,
Wohnraummangelund die hohe Arbeitslosigkeitunterr

den Zuwanderern. Erfolgreiche Integration kann und Eine existentielle Frage

wird so nicht gelingen. wie die Zuwanderung
muss in demokratischer

Selbstbestimmung
Unsere Lösung: Ein Schutzsystem des 21. Jahrhunderts in «u nationaler Ebene

. entschieden werden.
Anlehnung an das australische Modell.

Eine existentielle Frage wie die Zuwanderung mussin
demokratischer Selbstbestimmung auf nationaler Ebene
entschieden werden. Unser Modell für das 21. Jahrhun-
dert hatneben der Ablösung der überholten internatio-
nalen Regelwerke, wie der Genfer Flüchtlingskonvention
von 1951, folgende Kernpunkte:

» Fokus aufhumanitäre Hilfe in Krisenregionen selbst
(intrakontinentale Fluchtalternativen)

» Zurückweisung von Asylsuchenden bei Einreise an
der Grenze gemäfs Art. 16a Abs. 2S.1GG

» Ausschliefslich qualifizierte Einwanderung nach
japanischem Vorbild

92 Migration, Asyl und Integration

» Humanitäre Aufnahme nur für vom Bundestag
ausgewählte, besonders schutzbedürftige Personen,
für deren Auswahl ein mit der deutschen Werte- und
Gesellschaftsordnung vereinbarer kultureller und
religiöser Hintergrund ein wichtiges Kriterium ist.

Solange ein dauerhafter und wirksamer Schutz der
EU-Außengrenzen fehlt und die untaugliche über-
staatliche Rechtslage fortbesteht, muss Deutschland
die Kontrolle der Grenzen und die Unterbindung
illegaler Grenzüberschreitungen selbst in die Hand
nehmen.

Wir fordern daher:

» Einreisen darf künftig nurnoch, wem dies erlaubt
ist. Zurückweisungen an der Grenze müssen wieder
als selbstverständliches Recht souveräner Staaten
aufgefasst werden.

» Die Grenzen müssen zur Abwehr unerlaubter Zuwan-
derung und zur Kriminalitätsbekämpfung mit

Deutschland muss die einermodernen und wirksamen Sicherung ausge-
Kontrolle der Grenzen stattet werden. Dabei muss grenzüberschreitender
und die Unterbindung dl laub dw kehr oh

illegaler Grenzüber- Pendler-, Urlaubs- und Warenverkehr ohne unzu-

schreitungen selbst in mutbare Einschränkungen gewährleistet sein.

ne Hand nehmen » Um die „Grüne Grenze‘ mit derselben Effektivität

überwachen zu können, wird ein umfassendes und
modernes Grenzsicherungskonzept erarbeitet.
Hierzu Können auch physische Barrieren wie z.B.
Grenzzäune zur Sicherung der Staatsgrenzen
gehören.

Migration, AsylundIntegration 93

Asylparadies Deutschland schließen

Seit 2015 trägt Deutschland die Hauptlast der

illegalen Zuwanderung nach Europa, verursacht durch
eine Vielzahl selbstverschuldeter Ursachen. Die AfD hat
einen umfassenden Mafßsnahmenkatalog zur Wieder-
herstellung geordneter Verhältnisse im Asylverfahren
erarbeitet und wird dessen Umsetzung konsequent
einfordern:

1. Keine pauschalen Zuwanderungskontingente und
Umverteilungsquoten, egal ob im nationalstaatlichen,
europäischen oder außereuropäischen Rahmen (z. B.

Resettlement).

2. Keine Aufnahme von Migranten im Rahmen
einer „europäischen Lösung), die durch Miss- Die AfD hat einen
brauch der Seenotrettung nach Europa gelangen. umfassenden Maß-

. .. . . nahmenkatalog zur
Vielmehr muss gemäfs dem internationalen Seerecht yedernersielnung

grundsätzlich eine Ausschiffung der Betroffenen am _ geordneter Verhält-
nächstgelegenen geeigneten Seehafen erfolgen. Die nn verfahren
heute praktizierte Form der Seenotrettung wirkt häu-
fig als tödlich endender Pull-Faktor.
3. Beantragung von Asyl und Eröffnung des Antragsver-
fahrens nur bei nachgewiesener Identität und Staats-
angehörigkeit.
A. Strenge Bestrafung von Falschangaben im
Asylverfahren.
5. Ausweitung der Zahl sicherer Herkunftsstaaten.
6. Bekämpfung des Missbrauchs legal erteilter Visa zum
zweck der Asylantragstellung;
7. Verlust des Schutzstatus nach Reisen von Asylbe-
rechtigten in ihr Herkunftsland. Wer sein „Verfolger-
land besucht hat, ist beim Versuch der Wiederein-
reise an der deutschen Grenze zurückzuweisen.

94

Die AfD fordert die
Wiederherstellung
rechtsstaatlicher
Zustände bei
Abschiebungen.

Migration, Asyl und Integration

10.

11.

12.

13.

14.

15.

Ablehnung jeglichen Familiennachzuges für
Flüchtlinge.

Abschaffung des „Spurwechsels’ abgelehnter und
ausreisepflichtiger Asylbewerber. Keine Belohnung
für illegale Einreise und Verweigerung der Ausreise.
Niederlassungserlaubnis für anerkannte Asyl-
bewerber erst nach 10 Jahren.

Beendigung des Wildwuchses bei der Asylverfahrens
beratung und -betreuung. Die „Beratung und Betreu-
ung erfolgt meist durch nichtstaatliche, freie Träger,
die ein finanzielles Interesse an steigenden
Flüchtlingszahlen und am Verbleib möglichst vieler
Abgelehnter haben. Diese Organisationen zählen
zur „Asyl-Industrie‘, die sich am Asylsystem berei-
chert. Aus Verfahrens- und Rückkehrberatung wird
nahezu immer eine Verweigerungs- und Bleibe-
beratung. Verfahrensberatung darfnurnoch nach

8 12 a Asylgesetz und nur durch staatliche Stellen
erfolgen.

Keine Arbeitserlaubnis mehr für Asylantragsteller
und abgelehnte Asylbewerber.

Jährliche Prüfung des Fortbestands von Asyl-
gründen über zehn Jahre, da Asyl ein Recht auf Zeit ist
und keine Anwartschaft auf dauerhafte
Einwanderung begründet.

Ausnahmslose Abschiebung von Gefährdern und
Straftätern, auch nach Afghanistan, Irak und Syrien.
Rückkehrpflichtige sollen eine handwerklich-prak-
tische Grundausbildung erhalten Können, die sie zum
Wiederaufbau ihrer Heimat und zur dortigen
Existenzgründung befähigt („fit for return‘).

Migration, Asyl und Integration

Abschiebung und Rückkehr

Die unterlassene Aufenthaltsbeendigung abgelehnter
und ausreisepflichtiger Asylbewerber ist das Gegenteil
von Rechtsstaatlichkeit: Sie ist Ausdruck fortgesetzten
Staatsversagens. Die Zahl der Ausreisepflichtigen ist da-
bei kontinuierlich gestiegen und hat im zweiten Quartal
2021 die Zahl von ca. 300.000 Personen erreicht -
während die Zahl freiwilliger Ausreisen und erfolgrei-
cher Abschiebungen kontinuierlich sinkt.

Die Politik hat vor der Aufgabe, diesen Widerspruch
aufzulösen, Kapituliert - getrieben von einer mächtigen
Anti-Abschiebe-Industrie. Es ist zynisch und inhuman,
Menschen mit der faktischen Garantie auf dauerhaften
Verbleib im „reichen’ Deutschland anzulocken und dabei
deren möglichen Tod im Mittelmeer in Kaufzu nehmen.

Freiwillige Rückkehr ist einer Abschiebung aus finanzi-
ellen, organisatorischen und humanitären Gründen im-
mer vorzuziehen, weshalb Rückkehrbereitschaft deutlich
mehr als bisher gefördert werden muss.

Die AfD fordert daher zur Wiederherstellung rechts-
staatlicher Zustände:

1. Eine Abschiebeoffensive zum Abbau des Überhangs
an Ausreisepflichtigen.

2. Eine nationale und eine supranationale „Remigra-
tionsagenda’ als Schutzgewährung in Herkunfts- und
Transitregionen nach dem Grundsatz „Hilfe vor Ort‘

3. Keine Duldungen mehr, sondern Bescheinigungen
über die Ausreisepflicht.

95

Die AfD fordert eine
nationale und eine
supranationale
„Remigrationsagenda'
als Schutzgewährung in
Herkunfts- und Transit-
regionen.

96 Migration, Asyl und Integration

A. Zuständigkeitsübertragung für den Vollzug von
Abschiebungen an die Bundespolizei und deren
entsprechende Ausstattung mit Personal und
Kompetenzen.

5. Verhinderung von Abschiebungen zu Straftatbestand
erklären.

6. Nutzung freier Platzkapazitäten bei Transportflügen
der Bundeswehr, um die Anzahl der Abschiebungen
zu erhöhen.

7. Aufhebung der Kirchenasyl-Vereinbarung zwischen
BAMF und Kirchen. Es ist nicht hinnehmbar, dass die
Kirchen unter Missbrauch des Kirchenasyls die
Fristen für Abschiebungen in EU-Länder aushebeln.

Leben retten und soziale Gerechtigkeit herstellen

Im internationalen Vergleich sind die Leistungen
Deutschlands für Asylbewerber sehr hoch. Anerkannte
erwartet eine lebenslängliche materielle Absicherung.
Die Mittel dafür werden aufgebracht von Steuerzahlern,
die schwer dafür arbeiten müssen, aber keine
Wirtschaftsflüchtlinge eingeladen haben.

Die AfD fordert die

Wahrung sozialer Die Grofßszügigkeit des deutschen Staates verursacht eine
Gerechtigkeit, indem . . . .

nur Sachleistungen als erhebliche Sogwirkung, die zum Tod vieler Menschen
Sozialleistungen für führt. Die Bundesregierung lädt damit schwere

Asylbewerber und Aus-
reisepflichtige erbracht
werden.

moralische Schuld auf sich.

Die Asylanerkennung führt zu einer sofortigen Gleich-
stellung in allen sozialen Sicherungssystemen, ohne jede
vorangegangene Teilnahme am Solidarsystem.

Das ist sozial ungerecht.

Migration, Asylund Integration 97

Die AfD fordert daher zur Senkung von Anreizen und zur
Wahrung der sozialen Gerechtigkeit:

» Sozialleistungen für Asylbewerber und Ausreise
pflichtige sind ausnahmslos als Sachleistungen zu
erbringen.

» Asylanerkennung darf nicht länger den automati-
schen, sofortigen, zeitlich und inhaltlich unbegrenz-
ten Anspruch auf alle Leistungen des Wohlfahrts-
staates bedeuten. Nach österreichischem Vorbild
könnte eine Leistungsgewährung in voller Höhe auch
vom Erreichen guter Sprachkenntnisse abhängig
gemacht werden.

» Eines der Hauptmotive für die gegenwärtige
Massenmigration nach Deutschland und Europa ist
die Möglichkeit von Migranten, in Deutschland
bezogene Transferleistungen und auch legal oder
illegal verdientes Geld zurück in ihre Heimat-
länder zu überweisen. Um diesen Migrationsanreiz
zu beseitigen, befürworten wir die Einführung einer
Steuer auf Rücküberweisungen, die den
Europäischen Wirtschaftsraum (EWR) verlassen.

Hilfe vor Ort statt Tod auf dem Weg nach Europa

Hilfe vor Ort hat für die AfD höchste Priorität. Ihr ist

absoluter Vorrang zu geben vor einer Aufnahme von

Asylbewerbern in Deutschland. Zugleich werden damit

wichtige Fluchtgründe abgebaut. Die AfD will die Zuwan-

derung wieder national

Ein Teil der im Rahmen der Asylpolitik eingesetzten regein

zweistelligen Milliardensummen kann heimatnah eine
vielfache Wirkung entfalten und so helfen, den
Migrationsdruck auch nachhaltig zu senken.

98 Migration, Asyl und Integration

Zuwanderung wieder nationalregeln

Die zuwanderungspolitische Souveränität Deutschlands
ist schon heute nicht mehr gegeben. Hauptgrund hierfür
ist eine weitreichende Abgabe von Hoheitsrechten an die
EU und der Beitritt zuinternationalen Vereinbarungen,
die allesamt eine linksextreme „One-world-Utopie‘
verfolgen.

Die AfD stellt sich dem entgegen. Sie tritt für die Rück-
gabe der Hoheitsrechte zur Rechtssetzung von Asyl
und Zuwanderung ein und schlägt dafür die folgenden
Mafßsnahmen vor:

» Die Vergemeinschaftung des Aufenthalts- und
Asylrechts wird rückgängig gemacht.

Die europäische Grenzsicherung verbleibt dabeiin
europäischer Verantwortung (Frontex) parallel zur
nationalstaatlichen Verantwortung für die eigene
Grenzsicherung.

» Deutschland kündigt unverzüglich den UN-Migrati-
onspakt und den UN-Flüchtlingspakt auf, da diese
trotz ihrer angeblichen Unverbindlichkeit in
nationales Recht eingehen werden.

_ » Derneue, von der Europäischen Kommission ange-

Die AfD bekennt sich BR
ie AfD bekennt sic strebte EU-Migrationspakt soll verhindert werden
dazu, die kulturelle

Identität Deutschlands bzw. der Beitritt Deutschlands unterbleiben.
en Fe » Kein ‚Gemeinsames europäisches Asylsystem‘
Ir [fordern die AUSAT- j
beitung eines (GEAS) mit von der EU festgelegten Aufnahmequoten
umfassenden für Flüchtlinge.

migrationspolitischen
Ansatzes nach
Japanischem Vorbild.

Migration, Asyl und Integration

Qualifizierte Zuwanderung

Das aktuelle Einwanderungssystem schadet
Deutschland kulturell sowie wirtschaftlich und ist
dringend reformbedürftig. Die AfD bekennt sich dazu,
die kulturelle Identität Deutschlands zu wahren.

Vorbild einer identitätswahrenden Migrationspolitik
können für Deutschland nicht klassische Einwande-
rungsländer von der Gröfse Kanadas oder Australiens
sein, sondern eher Länder wie Japan, die eine ihrer
Landesstruktur entsprechende Begrenzung und Steue-
rung der Migration verfolgen.

Durch Lockdown-Krise, Automatisierung und Digita-
lisierung sind die Zeiten rekordhoher Beschäftigung
absehbar vorbei. Massenmigration erzeugt Lohndruck
bis in die Mittelschicht hinein und führt zu Konkurrenz
um Sozialleistungen.

Wir fordern die Ausarbeitung eines umfassenden migra-
tionspolitischen Ansatzes nach japanischem Vorbild mit
dem Primärziel der Interessenwahrung der Deutschen
und bereits zugewanderter Bürger, die sich zur
kulturellen Identität Deutschlands bekennen.

Die Behauptung einiger Wirtschaftsverbände und
Lobbyisten, in Deutschland herrsche Fachkräftemangel
und damit ein Bedarf an qualifizierter Einwanderung,
dient nur vordergründig dem Gemeinwohl und darf
deshalb nicht maßgeblich sein. Der schädliche Einfluss
wirtschaftlicher Partikularinteressen oder gar damit
kaschierter antideutscher Ressentiments muss beendet
werden.

99

Integration gelingt nur,
wenn die Regeln,
Traditionen und Werte
unserer Gesellschaft
akzeptiert werden.
Unverzichtbar ist der
Erwerb der deutschen
Sprache.

100 Migration, Asyl und Integration

Integration

Bei der Integration sehen wir Zuwanderer und Personen,
denen wir einen vorübergehenden Schutz gewähren,
in der Pflicht. Integration gelingt nur, wenn die Regeln,
Traditionen und Werte unserer Gesellschaft akzeptiert
werden. Unverzichtbar ist der Erwerb der deutschen
Sprache.

Die AfD fordert:

» Beiqualifizierten Zuwanderen setzen wir eine in-
tensive Integrationsbereitschaft voraus. Der Fokus
muss auf dem Spracherwerb liegen, den wir bis zum
Niveau B2 unterstützen wollen.

» Die Bevorzugung von Migranten in Form einer Quo-
tierung, unter Missachtung des Leistungsprinzips,
muss ausgeschlossen werden. Abstriche hin-
sichtlich der Anforderungen bei der Ausbildung oder
bei der Einstellung in den Öffentlichen Dienst lehnen
wir ab.

» Integrationsprojekte sind einer staatlichen Evalua-
tions- und Erfolgsprüfung zu unterziehen.

Die AfD fordert Das Profitgeschäft von NGOs und Wohlfahrts-
Integrationsprojekte verbänden ist zu beenden.

einer staatlichen a
Evaluations- und » Die AfD verlangt, dass das Kopftuch als religiös-
Erfolgsprüfung zu politisches Zeichen im Öffentlichen Dienst generell

hen .
nn nicht gestattet und in öffentlichen Schulen weder

von Lehrerinnen noch Schülerinnen getragen wird.
Dies betrachten wir als eine der wichtigsten
Bedingungen für eine gelingende Integration. Das
Kopftuchverbot trägt zur Wahrung des Schulfriedens
bei und ermöglicht jungen Mädchen eine freie
Persönlichkeitsentwicklung und Selbstbestimmung.

Migration, Asylund Integration 101

Die deutsche Staatsangehörigkeit ist
ein kostbares Gut

Das Geburtsortprinzip (Erwerb der deutschen Staats-
angehörigkeit durch Geburt in Deutschland, auch wenn

kein Elternteil Deutscher ist) wollen wir wieder aus dem
Gesetz streichen und zum Abstammungsprinzip zurück-
kehren, wie es bis zum Jahr 2000 galt. Die Anforderun-

gen - besonders auch in zeitlicher Hinsicht - für eine
Ermessenseinbürgerung müssen erheblich steigen und
einen Anspruch auf Einbürgerung darf es nicht mehr

geben. Die Verleihung der deutschen Staats-

angehörigkeit soll als Erfolg eigener Anstrengungerlebt _

werden und die Betroffenen mit Stolz erfüllenkönnen. Pwertdie
Rückkehr zum Abstam-
mungsprinzip für die
Staatsbürgerschaft, wie
es bis zum Jahr 2000
galt."""

In [6]:
text_afd = """Deutschland. Aber normal. normal.

Programm der Alternative für Deutschland
für die Wahl zum 20. Deutschen Bundestag

AD.

2 Inhaltsverzeichnis

Demokratie und Rechtsstaat 12
Das Volk ist der Souverän 12
Volksabstimmungen nach Schweizer Modell 13
Die Gewaltenteilung gewährleisten 15
Entpolitisierung der Justiz 15
Verfassungsschutz reformieren 16
Trennung von Amt und Mandat und Kampf der Ämterpatronage 16
Macht der Parteien beschränken 16
Freie Listenwahl und freies Mandat 17
Verkleinerung der Parlamente 18
Berufspolitikertum: Mandats- und Amtszeiten begrenzen 18
Lobbyismus eindämmen 19

Freiheit und Verantwortung 22
Bürgerlich-freiheitliche Rechtsordnung 22
Leistung statt Quote - Freiheit statt Diskriminierung 24
Schutz der persönlichen Freiheit vor digitalen Übergriffen 24
Hinweisgeber (Whistleblower) schützen 25

EU und Europa 28
Ein Europa der Vaterländer 28
Zurückführung der Europäischen Union in einen Staatenbund
souveräner Staaten 29

Steuern und Finanzen 34
Keine EU-Steuern 34
Für ein Steuersystem mit wenigen Steuerarten und verständlicher Systematik 34
Abschaffung der Substanzsteuern 35
Wir wollen ein Land von Wohnungseigentümern werden 37
Elementare Steuergerechtigkeit statt Nominalwertprinzip
und Ungleichbehandlung von Steuerbürgern 37

Besteuerung an Digitalzeitalter anpassen: Einführung einer Digitalsteuer 39

Inhaltsverzeichnis 3

Wirtschaft: Für einen „Blue Deal” 42
Wirtschafts- und Innovationspolitik für Deutschland 42
Den Mittelstand entlasten 43
Politisch induzierte Belastungen aussetzen 43
Technologieführerschaft mit „Blue Deal” 43
Soziale Marktwirtschaft statt sozialistischer „Industriepolitik“ 45

Der Euro ist gescheitert 50
Deutschland muss aus dem untergehenden Euro-System austreten 50

Keine Schuldenaufnahme durch die EU, Hilfskredite nur aufnationaler Ebene 51
Zusätzliche Inflationierung durch überzogene Corona-Mafsnahmen beenden 52

EZB-Nullzinspolitik beenden 53
Wohlstand und sozialer Frieden in Europa durch Wiedereinführung
nationaler Währungen 55
Erhalt des Bargelds: Bargeld ist Freiheit und Schutz vor Enteignung
und Totalüberwachung 56
Target-2-Risiko beenden 57
Nationale Investitionen 58
Absicherung gegen die Eurokrise, Geldsystem reformieren,
deutsches Staatsgold vollständig zurück nach Deutschland 59
Außen- und Verteidigungspolitik 62
Die Außenpolitik Deutschlands muss sich an deutschen Interessen ausrichten 62
Das Verhältnis zu wichtigen Staaten im Wandel der Geopolitik 64
Wiederherstellung der Wehrfähigkeit Deutschlands 66
Entwicklungspolitik: Stabilität als Sicherheits- und Wirtschaftsinteresse 68
Deutschland muss seine wirtschaftlichen Stärken ausbauen 70

Auswärtige Kultur- und Bildungspolitik Deutschlands 71

4 Inhaltsverzeichnis

Innere Sicherheit 76
Linke Gewalt entschieden bekämpfen 76
Wirksame Bekämpfung der Ausländerkriminalität 77
Vollstreckung im Ausland 78
Jugendstrafrecht 78
Bundespolizei stärken - Grenzschutz ist nationale Aufgabe 78
Reform der Polizei 79
Fahndungsmöglichkeiten verbessern 80
Organisierte Kriminalität bekämpfen 80
Sicherheit der Bürger verbessern 81
Strafjustiz verbessern 81

Islam 84

Migration, Asyl und Integration:

Human und rational handeln - Deutschland schützen 90
Die Fluchtbewegungen des 21. Jahrhunderts können nicht mittels
Aufnahme der Migranten in Europa und Deutschland bewältigt werden. 90
Asylmissbrauch beenden 90
Die unlösbaren Folgeprobleme 91
Asylparadies Deutschland schliefsen 93
Abschiebung und Rückkehr 95
Leben retten und soziale Gerechtigkeit herstellen 96
Hilfe vor Ort statt Tod auf dem Weg nach Europa 97
Zuwanderung wieder national regeln 98
Qualifizierte Zuwanderung 99
Integration 100

Die deutsche Staatsangehörigkeit ist ein kostbares Gut 101

Inhaltsverzeichnis 5

Familienpolitik 104
Familien stärken und fördern 104
Ursachen der demografischen Katastrophe 105
Leistungsfähigkeit der Familien erhalten 106
Familien unterstützen 107
Familienfreundliches Deutschland 108
Kinderin den Familien belassen, Kindesentzug nur in Ausnahmefällen 109
Willkommenskultur für Kinder 110
Trennungsfolgen mindern 112
Verantwortungsvolle Kinderbetreuung 113
Das Geschlecht ist eine biologische Tatsache 114

Arbeits- und Sozialpolitik 118
Zuwanderung von EU-Ausländern in unsere Sozialsysteme begrenzen 118
Nationale Gesetzgebungshoheit im Sozialrecht verteidigen 118
Keine Auszahlungen von Sozialleistungen auf ausländische Konten 119
Lohndumping in der Leiharbeit verhindern 119
Mindestlohn beibehalten 119
Arbeitsleben würdigen - Arbeitslosengeld I gerecht gestalten 120
Reform der Arbeitslosenversicherung 121
Aktivierende Grundsicherung - Arbeit, die sich lohnt 121
Finanzkontrolle Schwarzarbeit stärken 121
Arbeitsrecht reformieren und vereinfachen 122
Gezielte Mafßfnahmen gegen Wohnungs- und Obdachlosigkeit 122

Mehr Arbeitsplätze mit fairer Entlohnung für schwerbehinderte Menschen 122
Verbesserungen für Menschen mit Behinderungen in Krankenhäusern 123

6 Inhaltsverzeichnis

Das Rentenkonzept der AfD 126
Renten zukunftsfest 126
Freiheit beim Renteneintritt 127
Altersarmut verhindern 127
Abschaffung der Politikerpensionen 128
Reduzierung des Beamtenstatus auf originär hoheitliche Aufgaben 128
Lastengerechtigkeit zwischen Familien und Kinderlosen herstellen 129
Ungerechtigkeiten bei der Überleitung der Ostrenten beseitigen 130
Keine Doppelbesteuerung, Steuerfreibetrag anheben 130
Umlagefinanzierte Rente stärken 131
Wehrdienst rentenpolitisch würdigen 131

Gesundheitspolitik 134
Schluss mit den unverhältnismäfßigen Corona-Mafsnahmen 134
Impfen muss freiwillig bleiben 135

Einsetzung eines parlamentarischen Corona-Untersuchungsausschusses 135
Krankenhäuser und Kliniken brauchen Planungssicherheit und

eine individuell auskömmliche Finanzierung 136
Serviceorientierte ambulante Versorgung durch Beendigung

der Budgetierung 137
Medizinische Versorgung auf dem Land wiederher und sichern 138
Zugang zu Heil- und Hilfsmitteln entbürokratisieren 139
Kurzzeitpflegeplätze in Krankenhäusern finanzieren 139
Beendigung des Pflegenotstands 139
Vorfahrt für die häusliche Pflege 140
Vereinfachung und bessere Übersichtlichkeit in der

sozialen Pflege- und gesetzlichen Krankenversicherung 141
Arzneimittelversorgung verlässlich machen 141
Organspende, Sterbehilfe 142
Cannabis nur iin der Medizin 142
Antibiotikaresistenzen vermeiden und konsequent bekämpfen 143
Den Beruf des Heilpraktikers erhalten 143
Anforderungen an ausländisches Personal im Gesundheitswesen 144
Souveränität des Patienten über seine Daten herstellen 144

WHO grundlegend reformieren 145

Inhaltsverzeichnis 7

Bildung, Wissenschaft und Forschung 148
Mut zur Leistung statt Akademisierungswahn 148
Mehrgliedriges Schulsystem erhalten 148
Berufliche Bildung stärken: Meister statt Master 149
Digitalisierung ist kein Selbstzweck 149
Keine ideologisch motivierte Inklusion: Förder- und Sonderschulen erhalten 150
Bildung und Migration 150
Neutralität der Schule wiederherstellen 151
Kompetenzorientierung entwertet Bildung 151
Deutsches Kulturgut im Unterricht vermitteln 152
Kein bekenntnisgebundener Islamunterricht an deutschen Schulen 152
Keine Sonderrechte für Muslime an unseren Schulen. 152
Kein Koranunterricht durch verfassungsfeindliche Moscheevereine 153
Keine Konfuzius-Institute an Öffentlichen Bildungseinrichtungen 153
Bologna ist gescheitert: Diplom und Magister wieder einführen 153
Autonomie der Hochschulen stärken, Freiheit von Forschung
und Lehre bewahren 154
Kein Geld für „Gender Studies‘, keine Gender-Quoten 154

Kultur 158
Deutsche Leitkultur statt „Multikulturalismus‘ 158
Die deutsche Sprache als Zentrum unserer Identität 159
Kulturförderung 159
Brauchtum und Gedenken 160

Medien 164
Den öffentlich-rechtlichen Rundfunk neu gestalten 164
„Cancel Culture” bekämpfen 164
Meinungsfreiheit statt Tugendterror 165

Bauen, Wohnen, Mieten 168
Wohneigentumsförderung 169

Mehr Wohngeld statt Sozialer Wohnungsbau 170

8 Inhaltsverzeichnis

Klima, Energie, Technik und Digitalisierung 174
Dem Klimawandel positiv begegnen 174
Kernforderungen zur Klimapolitik 175
Technischen Fortschritt begrüfsen 175
Energie - Nein zum „Green Deal’ 176
Digitalisierung muss uns Menschen dienen 180
Datenschutz und Datensicherheit 182
Infrastruktur ist die Grundlage für den weiteren Ausbau derDigitalisierung 183
Digitalisierung der Öffentlichen Verwaltung 184
Digitale Souveränität 185

Mobilität und Infrastruktur 188
Verkehrspolitik 188
Motorisierten Individualverkehr schützen 188
Bundesautobahnen entlasten - Transitverkehr auf Schiene und
Wasserwege verlagern 189
Sichererer, sauberer und verlässlicher Schienenverkehr 189
Flugverkehr als Wirtschaftsfaktor stärken 190
Die deutsche Automobilindustrie als Leitindustrie erhalten 190
Stadt und Land gleichrangig entwickeln 191
Entscheidungen näher zum Bürger bringen 192
Landschaft als Teil regionaler Identität 192
Infrastruktur 193
Kommunen stärken - Erfüllung der Pflichtaufgaben sicherstellen 193
Ausbau digitaler Infrastrukturen beschleunigen 194

Nord Stream 2 endlich fertigstellen 194

Landwirtschaft, Umwelt- und Verbraucherschutz 198
EU-Agrarpolitik renationalisieren 199
Grüne Berufe wertschätzen 199
Familienbetriebe in den Mittelpunkt stellen 200
Düngung am Bedarf der Pflanze orientieren 200
Praxiskonformen Pflanzenschutz etablieren 201
Tierwohl, Haltung und Transport weiterentwickeln 201
Schutz der heimischen Ökosysteme verbessern 202
Zeitgemäfses Wildtiermanagement umsetzen 203
Unsere Wälder schützen und bewahren 203
Gesunde Lebensmittel aus heimischer Produktion fördern 204
Transparenz im Verbraucherschutz ausbauen 204
Wasserversorgung in Öffentlicher Hand belassen 205

Ausbau von erneuerbaren Energien einschränken 205

KAPITEL]1

Demokratie und
Rechtsstaat

12

Wir halten die unmittel-
bare Demokratie für ein
unverzichtbares Mittel,
um dem autoritären
und teilweise totalitären
Gebaren der
Regierungspolitiker
Einhalt zu gebieten.

Demokratie und Rechtsstaat

Demokratie und Rechtsstaat

Das Volk ist der Souverän

Die Regierungspolitiker in Bund und Ländern haben mit
ihrer Flüchtlings-, Europa- und Corona-Politik die
Prinzipien der deutschen Staatlichkeit, des Rechts und
der Verfassung vielfach verletzt.

Zugleich haben sich die Volksvertreter der etablierten
Parteien den grundgesetzlich garantierten Parlaments-
vorbehalt für alle wichtigen Entscheidungen im Staat
ohne Widerstand nehmen lassen.

Die Bundesregierung kommt ihrer Pflicht, Vertrags-
brüchen und Selbstermächtigungen durch
EU-Institutionen entgegenzutreten, nicht nach. Einzelne
rechts- und verfassungswidrige Mafsnahmen wurden
zwar durch mutige Richter in Hunderten von Urteilen zu
Fall gebracht. In unserem Land hat sich aber eine
politische Klasse herausgebildet, deren vordringliches
Interesse ihrer Macht, ihrem Status und ihrem
materiellen Wohlergehen gilt. Diese setzt die soziale und
kulturelle Zukunft unseres Volkes, die Stärke unserer
Wirtschaft und damit unseres Wohlstandes aufs Spiel
und stellt Multikulturalität, Diversität, Globalisierung
und vermeintliche Gendergerechtigkeit über alles.

Sie hält die Schalthebel der staatlichen Macht, der
politischen Bildung und des informationellen und
medialen Einflusses auf die Bevölkerung in Händen.

Deshalb halten wir die unmittelbare Demokratie für ein
unverzichtbares Mittel, um dem autoritären und
teilweise totalitären Gebaren der Regierungspolitiker
Einhalt zu gebieten.

Demokratie und Rechtsstaat 13

Volksabstimmungen nach Schweizer Modell

Die AfD fordert Volksentscheide nach Schweizer Modell
auch für Deutschland. Die uneingeschränkte
Volkssouveränität in ihrer fast 200 Jahre bewährten und
optimierten Gestaltung hat dem eidgenössischen Bun-
desstaat eine fortwährende Spitzenstellung in
Wohlstand, Frieden und Freiheit gewährleistet.

Durch die Volksabstimmungen wollen wir die Flut der
oftmals unsinnigen Gesetzesvorlagen eindammen und
die Parlamente zu sorgfältiger Arbeit zwingen. Unange-
messenen Beschlüssen der Abgeordneten in eigener
Sache wollen wir einen Riegel vorschieben. Hierzu
zahlen etwa die Diäten, Fraktions- und Parteien-
finanzierung sowie die Finanzierung von „parteinahen'
Stiftungen. Der Wortbruch der etablierten Parteien be-
züglich des Euros und die immer weitere Zentralisierung
der EU bedrohen die Zukunft unseres Landes (sowie der
nachfolgenden Generationen). Diese fatale Politik wollen

wir beenden. Die AfD fordert
Volksentscheide nach

. oe . Schweizer Modell auch
Die Einführung von Volksabstimmungen nach für Deutschland.

Schweizer Modell ist für die AfD nicht verhandelbarer a
. . ..: . . Die Einführung von
Inhalt jeglicher Koalitionsvereinbarungen und beinhaltet yonsansiimmungen

insbesondere folgende Elemente: nach Schweizer Modell
ist für die AfD nicht
verhandelbarer Inhalt

Ohne Zustimmung des Volkes darf das Grundgesetz jeglicher Koalitions-

nicht geändert und kein bedeutsamer völkerrechtlicher nbarungen
Vertrag geschlossen werden. Wir wollen dem Volk das

Recht geben, den Abgeordneten auf die Finger zu

schauen und vom Parlament beschlossene Gesetze zu

andern oder abzulehnen.

14

In Schicksalsfragen
der Nation entscheiden
Bürger weitsichtiger,
friedfertiger und
gemeinwohl-
orientierter als
Berufspolitiker.

Demokratie und Rechtsstaat

Das Volk soll die Möglichkeit erhalten, Gesetzesinitia-
tiven einzubringen und per Volksabstimmung zu be-
schliefsen. Hierbei soll es jenseits des Art. 79 Abs. 3 GG
keine thematischen Beschränkungen geben.

Die Qualität der Entscheidung der Bürger muss
gesichert werden durch einen ausgeprägten und auto-
nomen Prozess der Willensbildung, durch Informations-
pflichten und -rechte und eine breite gesellschaftliche
Debatte. Auch die Parlamente sollen die Möglichkeit
erhalten, eine Volksabstimmung zu initiieren.

Als einzige der im Bundestag vertretenen Parteien
halten wir das deutsche Volk für ebenso mündig wie das
der Schweizer. Entgegen anderslautenden
Behauptungen entscheiden Bürger in Schicksalsfragen
der Nation weitsichtiger, friedfertiger und gemeinwohl-
orientierter als Berufspolitiker. Das ist die Lehre aus der
Wiedervereinigung, aus der Ablehnung des
„EU-Verfassungsvertrags durch das französische und
das niederländische Volk und der Entscheidung des
Vereinigten Königreichs, aus der Europäischen Union
auszutreten.

Wir vertrauen nicht mehr darauf, dass Regierungen und
Parlamente zu Währungskrisen, Migration,
Islamisierung oder zur Energiewende tragfähige Lösun-
gen finden. Das Volk als Souverän muss in direkter
Mitbestimmung Träger solcher schicksalhaften
Entscheidungen sein.

Demokratie und Rechtsstaat 15

Die Gewaltenteilung gewährleisten

Zahlreiche Gesetze und die politische Praxis haben die

Gewaltenteilung in Deutschland als Kernelement des
Rechtsstaats über die Jahre in Gefahr gebracht und zu
einer überbordenden Staatsgewalt geführt: Ehemalige
Politiker auf Richterstühlen, Abgeordnete, die zugleich
Kanzler, Ministerpräsident oder Minister sind und sich
insoweit selbst kontrollieren, sowie parteipolitische
Netzwerke, die durch verbotene, verfassungswidrige
Ämterpatronage entstehen, können nicht länger
toleriert werden. Öffentliche Ämter sind ausschließlich
nach „Eignung, Befähigung und fachlicher Leistung’ zu
vergeben (Art. 33 Abs.2 GG).

Entpolitisierung der Justiz

Die AfD will die Einflussnahme der politischen Parteien

auf die Ernennung von Richtern und Staatsanwälten
beenden. Auch wollen wir die Praxis ändern, dass die

Staatsanwaltschaft weisungsgebunden und im Einzelfall

dem Justizminister berichtspflichtig ist. Die Unabhän-
gigkeit der dritten Gewalt muss durch eine Selbstver-
waltung der Justiz ausgebaut werden, wie esin vielen
anderen europäischen Ländern bereits üblich ist. Wir

unterstützen daher den Modellvorschlag des Deutschen

Richterbundes, einen Justizwahlausschuss und einen
Justizverwaltungsrat einzurichten. Insbesondere die
Verfassungsgerichte und Rechnungshöfe sind vor
parteipolitischer Einwirkung zu schützen.

Die AfD will die Einfluss-
nahme der politischen
Parteien auf die Ernen-
nung von Richtern und
Staatsanwälten
beenden.
Insbesondere die
Verfassungsgerichte
und Rechnungshöfe
sind vor partei-
politischer Einwirkung
zu schützen.

16

Die AfD fordert:

Die Mitarbeiter der
öffentlichen Verwaltung
haben unabhängig und
loyal ihren Dienst zu
verrichten.

Beamten- und Richter-
stellen sind ausschliejs-
lich nach Qualifikation
zu besetzen.

Demokratie und Rechtsstaat

Verfassungsschutz reformieren

Der behördlich organisierte „Verfassungsschutz” muss
schnellstmöglich auf den Prüfstand gestellt und so
reformiert werden, dass er in Zukunft nicht mehr als
parteipolitisches Instrument gegen politische Gegner
missbraucht werden kann.

Trennung von Amt und Mandat und
Kampf der Ämterpatronage

Kanzler, Ministerpräsidenten und Minister dürfen nicht
mehr zugleich Abgeordnete sein. Die Institution der
parlamentarischen Staatssekretäre ist abzuschaffen.
Dies gilt auch für das Institut der „politischen Beamten’
in der Staatsverwaltung - vom Ministerialdirektor bis
zum Polizeipräsidenten. Die Mitarbeiter der Öffentlichen
Verwaltung haben unabhängig und loyal ihren Dienst
zu verrichten. Beamten- und Richterstellen sind aus-
schliefslich nach Qualifikation zu besetzen. Die „Partei-
buchwirtschaft muss beendet werden.

Grundsätze der fachlichen Eignung und beruflichen
Qualifikation haben auch für das Spitzenpersonal des
Staates in den Kabinetten zu gelten. Parteipolitische
Karrieren können fachliche Defizite nicht ausgleichen.

Macht der Parteien beschränken

Die Parteien sollen am politischen System mitwirken
(Art. 21, Abs.1GG), es aber nicht beherrschen. Die All-
macht der Parteien und deren „Ausbeutung des Staates‘
gefährden unsere Demokratie. Diese politische
Instrumentalisierung des Staates ist auch Ursache der
verbreiteten Politikverdrossenheit.

Demokratie und Rechtsstaat 17

Die Loyalität der Abgeordneten muss den Bürgern und
dem Staat gelten, nicht den Parteivorständen.

Neben der offiziellen Parteienfinanzierung in Höhe von

mittlerweile jährlich ca. 194 Millionen Euro haben sich

die Parteien zusätzliche Finanzausstattung verschafft.

Hervorzuheben ist dabei die Finanzierung so-

genannter politischer Stiftungen, denen alleine 600

Millionen Euro aus Mitteln des Bundeshaushalts jahrlich ea les:
zuflieisen. Ferner halten wir die jedem Bundestagsabge-  siaatliche Parteienfi-
ordneten zustehende „Mitarbeiterpauschale” von 22436 nanzierung neu regeln
Euro monatlich für überhöht. Ob dabei in jedem Fall a
wissenschaftlich qualifiziertes Personal oder Partei- Finanzierung von
soldaten angestellt werden, darf hinterfragt werden. - nn en ganz
Die AfD will die gesamte staatliche Parteienfinanzierung

neu regeln und deutlich begrenzen, sowie die staatliche

Finanzierung von Parteistiftungen ganz abschaffen.

Alle Mittel der Parteienfinanzierung sollen entgegen

derzeitiger Praxis der unbeschränkten Kontrolle der

Rechnungshöfe unterliegen. Unabdingbar ist auch eine

restriktive und Korruption vermeidende Neuordnung

der Spendenregelungen. Ferner soll den Parteien

endlich die Beteiligung an Unternehmen, insbesondere
Medienunternehmen, verboten werden.

Freie Listenwahl und freies Mandat

Die politische Klasse Deutschlands hat das Wahlrecht
und die Wahlverfahren im Lauf der Zeitimmer besser
ausgenutzt und angepasst, um den Einfluss des Volkes
auf die Bewerberauswahl zu minimieren. Dem Wähler
werden derzeit starre Wahllisten der Parteien zum
Ankreuzen vorgelegt.

13

Um unser Ideal des
Bürgerabgeordneten zu
verwirklichen,

fordern wir eine Begren-
zung der Mandatszeit
für Abgeordnete auf vier
und der Amtszeit für
den Bundeskanzler

auf zwei Legislatur-
perioden.

Demokratie und Rechtsstaat

Über die „sicheren Listenplätze“ bestimmen de facto die
Parteivorstände die Zusammensetzung des Parlaments.
Mit einer demokratischeren Gestaltung des Wahlsys-
tems will die AfD dem Wähler die Entscheidung über die
personelle Zusammensetzung der Parlamente zurück-
geben.

Deshalb treten wir für die „freie Listenwahl” ein, mit der
Möglichkeit des Kumulierens, Panaschierens und Strei-
chens von Kandidaten.

Verkleinerung der Parlamente

Mit seinen mittlerweile 709 Abgeordneten leistet sich
Deutschland eines der gröfsten Parlamente der Welt.
Die im Oktober 2020 von CDU/CSU und SPD durchge-
setzte Wahlrechtsänderung entspricht nicht der vom
Bundesverfassungsgericht angemahnten Reform der
Wahl zum Bundestag. Mit etwa 2.500 Abgeordneten in
Bund und Ländern und dem dazugehörigen
Verwaltungsapparat hat Deutschland übergrofse
Parlamente. Wir streben an, den Bundestag um etwa
die Hälfte zu verkleinern.

Berufspolitikertum:
Mandats- und Amtszeiten begrenzen

Die ungebrochene Tendenz zum Berufspolitikertum hat
der Monopolisierung der Macht Vorschub geleistet und
die unübersehbare Kluft zwischen Wählern und
Gewählten stetig vergröfsert. Vetternwirtschaft, Filz,
korruptionsfördernde Strukturen und Lobbyismus sind
die Folge.

Demokratie und Rechtsstaat 19

Um unser Ideal des Bürgerabgeordneten zu verwirkli-

chen, fordern wir eine Begrenzung der Mandatszeit für
Abgeordnete auf vier und der Amtszeit für den Bundes-
kanzler auf zwei Legislaturperioden. Nicht die Abgeord- __

netenvergütung, sondern politischer Idealismus soll Wir wollen die Rege-
lungen hinsichtlich der

Motivation zur Kandidatur für ein Nebentätigkeiten von

Mandat sein. Parlamentariern
deutlich verschärfen,
um Bestechung zu

Wir wollen, dass Bundestagsabgeordnete ihre volle
Arbeitskraft der parlamentarischen Arbeit widmen.
Das Mandat darf nicht unter bezahlten Nebentätigkeiten
leiden. Der überbordende Lobbyismus in Brüssel und
Berlin muss eingedämmt werden. Wir wollen die
Regelungen hinsichtlich der Nebentätigkeiten von
Parlamentariern deutlich verschärfen, um Bestechung
zu verhindern. Unseren eigenen Abgeordneten haben
wir bereits jetzt über unsere Satzung entsprechende
Regeln auferlegt. Ferner setzen wir uns für ein
Lobbyisten-Gesetz mit konkreten Rechten, Pflichten
und Sanktionen für Mandatsträger und Lobbyisten ein,
das vorbildlichen Regelungen anderer Demokratien
entspricht.

Freiheit und
Verantwortung

22

Die Alternative für
Deutschland tritt für die
Wiederherstellung der
bürgerlichen
Selbstbestimmung im
Zivilrechtsverkehr ein.
Deshalb lehnen wir sog.
‚Antidiskriminierungs-
gesetze' ab.

Freiheit und Verantwortung

Freiheit und Verantwortung

Im Zentrum unseres Menschenbildes stehen Freiheit
und Verantwortung. Wir gehen von der Befähigung und
Bereitschaft jedes einzelnen aus, in Freiheit und Verant-
wortung sein Leben zu gestalten und seine Angelegen-
heiten und die seiner Familie zu besorgen. Gängelung
und Indoktrination durch Staat oder gesellschaftliche
Gruppen lehnen wir ab.

Bürgerlich-freiheitliche Rechtsordnung

Die Prinzipien der Gleichheit vor dem Gesetz und der
bürgerlichen Vertragsfreiheit gehören zu den großen
zivilisatorischen Errungenschaften Europas. Während
des weitaus grölsten Teils der Menschheitsgeschichte
gehörte es zur unhinterfragten Normalität, dass
Menschen durch Geburt oder Stand ungleich und in
ihrer rechtlichen Selbstbestimmung beschränkt
waren, bis hin zum Extrem der Rechtlosigkeit in Form
von Leibeigenschaft oder Sklaverei. Erst die christliche
und humanistische Kultur der europäischen Völker
brachte die vorgenannten Prinzipien hervor und
verwirklichte sie in einem jahrhundertelangen Prozess.
In den letzten Jahrzehnten ist erschreckenderweise
eine Abkehr von diesen europäischen Grundwerten zu
verzeichnen. Diesem zivilisatorischen Rückschritt giltes
entgegenzutreten.

In Artikel 3 des Grundgesetzes heifst es: „Männer und
Frauen sind gleichberechtigt. Der Staat fördert die
tatsächliche Durchsetzung der Gleichberechtigung von
Frauen und Männern und wirkt auf die Beseitigung

Freiheitund Verantwortung 23

bestehender Nachteile hin. Niemand darf wegen seines
Geschlechtes benachteiligt oder bevorzugt werden.
Diese Sätze haben einen klaren und guten Sinn.

Das Recht, nicht vom Staat aufgrund des eigenen
Geschlechts benachteiligt zu werden, ist ein individuelles
Menschenrecht. Es kommt jedem Menschen in gleicher
Weise zu.

Manche Juristen vertreten die falsche These, Artikel 3 GG
erlaube es dem Staat, einzelne Menschen
„stellvertretend für ihr jeweiliges Geschlecht zu
bevorzugen oder zu benachteiligen, um die als

Kollektive vorgestellten Geschlechter „gleichzustellen‘.
Jedoch kann esin einem freiheitlichen Rechtsstaat nie-
mals legitimes Ziel staatlichen Handelns sein, in willkür-
lich abgegrenzten gesellschaftlichen Teilbereichen eine
ebenso willkürlich festgesetzte „Geschlechter-

quote zu erzwingen. Genauso wenig kann eine gesetz-
lich vorgeschriebene Ungleichbehandlung ein Mittel zur
Verwirklichung von Gleichberechtigung sein. Gesetze,
die solches vorschreiben, sind verfassungswidrig und
abzulehnen. Die AfD wird sich im Deutschen Bundestag
für ihre Aufhebung einsetzen.

Die Alternative für Deutschland tritt für die Bewahrung
bzw. Wiederherstellung der bürgerlichen Selbst-
bestimmung im Zivilrechtsverkehr ein. Deshalb lehnen
wir sog. ‚Antidiskriminierungsgesetze ab. Zentraler
Grundwert einer freiheitlichen Zivilrechtsordnung ist die
Vertragsabschlussfreiheit, also die Freiheit jedes
Einzelnen, selbst darüber zu entscheiden, ob er mit
einem anderen Bürgerin rechtliche Beziehungen treten
will oder nicht.

24

Die Alternative für
Deutschland tritt für
die Gleichheit vor dem
Gesetz ein. Deshalb
lehnen wir sogenannte

„Quotenregelungen” ab.

Freiheit und Verantwortung

Unter dem Einfluss der Europäischen Union wird dieses
fundamentale Prinzip der Privatautonomie in der
deutschen Gesetzgebung Schritt für Schritt zerstört.

In einer freiheitlichen Grundordnung ist der Staatin
seinem Handeln gegenüber dem Bürger der Pflicht zur
Gleichbehandlung unterworfen, der Bürger selbst jedoch
istin der Gestaltung seiner privaten Rechtsverhältnisse
frei. Eine Aufhebung dieser Trennung zwischen Staat
und Gesellschaft beschädigt die freiheitliche Natur der
Rechtsordnung und Öffnet Tür und Tor für eine in letzter
Konsequenz totalitäre Herrschaftsordnung.

Die AfD wird sich im Deutschen Bundestag gegen diese
Entwicklung und für die Bewahrung einer freiheitlichen
Privatrechtsordnung einsetzen.

Leistung statt Quote - Freiheit statt Diskriminierung

Die grundgesetzlich garantierte Gleichberechtigung
von Mann und Frau hat mit der vom Parteienkartell
propagierten Gleichstellung und Gleichmacherei
nichts zu tun.

Die AfD lehntjede Art von Quoten ab. Sie sind leistungs-
feindlich, ungerecht und stellen eine verfassungswidrige
Diskriminierung derer dar, die nichtin den Genuss der
Quote kommen.

Schutz der persönlichen Freiheit vor
digitalen Übergriffen

Eingriffe in die Privatsphäre durch monopolistische,
datengetriebene Grofskonzerne, Behörden und Geheim-
dienste überschreiten zum Teil deutlich die grundge-

Freiheitund Verantwortung 25

setzlich garantierten Freiheitsrechte. Dieser Gefährdung
unserer Freiheit wollen wir entgegenwirken, den
Bürgern ihre informationelle Selbstbestimmung
zurückgeben und diese verfassungsrechtlich stärken.

Hinweisgeber (Whistleblower) schützen

Die totalitären Tendenzen haben auch in Deutschland
ein demokratiegefährdendes Mafs angenommen.

Die verfassungswidrige Grenzöffnung und die massiven
Grundrechtseinschränkungen im Namen der
Corona-Krise waren die bisherigen Höhepunkte im
täglichen Bruch von Recht und Verfassung durch
Regierungspolitiker. Die Mainstream-Medien haben ihre
elementare Aufgabe als vierte Gewalt aufgegeben und
kritisch-objektive Berichterstattung gegen Regierungs-
lobhudelei getauscht. So bleiben oftmals nur noch alter-
native Medien und Whistleblower zur Aufdeckung und
Offenlegung regierungsamtlichen Unrechts. Aus diesem
Grund sind anonyme Hinweisgeber zur Aufdeckung von
Korruption, Machtmissbrauch und anderen Straftaten in
Politik, Verwaltung, Wirtschaft und
zivilgesellschaftlichen Organisationen unverzichtbar.

Die EU-Richtlinie zum Hinweisgeberschutz ist
unzureichend. Wir fordern ein einheitliches deutsches wir wollen den Bürgern
Gesetz mit verständlichen klaren Regeln sowie eine ihre informationelle

. . Selbstbestimmung
entsprechende Anpassung des Beamtenrechts. Wichtige „uriekgeben und diese
Bestandteile eines solchen Gesetzes sind für uns vorge- _ verfassungsrechtlich
schriebene interne Whistleblower-Kanäle und altemati-
ve externe Meldemöglichkeiten bei einer unabhängigen,
selbstständigen Behörde sowie der vollumfängliche
Schutz des Hinweisgebers vor Repressalien, inklusive

einer gesetzlich garantierten Vertraulichkeit.

KAPITEL

EU und Europa

28

Die AfD steht für die
Freiheit und Selbstbe-

stimmung der europäi-

schen Nationen.
Wir wollen den souve-
ränen, demokratischen

Nationalstaat erhalten.

EU und Europa

EU und Europa

Ein Europa der Vaterländer

Die AfD steht für die Freiheit und Selbstbestimmung der
europäischen Nationen. Wir bekennen uns zu einem
Europa der Vaterländer als einer Gemeinschaft
souveräner Staaten, die aufall jenen Gebieten
zusammenarbeiten, die gemeinsam besser gestaltet
werden können. Dazu gehört insbesondere ein freier
Handel mit fairem Wettbewerb.

Eine staatsähnliche Europäische Union, wie sie von den
etablierten Parteien angestrebt wird, halten wir im Sinne
eines prosperierenden und friedlichen Europas für
kontraproduktiv. Selbstverantwortliche und von
lebendigen Demokratien gestaltete Nationalstaaten sind
durch übernationale Einrichtungen nicht ersetzbar.

Eine Gruppe benachbarter Staaten kann sehr gut auf
völkerrechtlicher Basis konstruktiv und friedlich
kooperieren. Der Versuch jedoch, aus derzeit 27 oder
noch mehr Staaten mitjeweils eigenen Sprachen,
Kulturen und historischen Erfahrungen einen wie auch
immer ausgestalteten Gesamtstaat zu bilden, muss
scheitern. Ein solches Gebilde verfügt weder über ein
Staatsvolk, noch über das erforderliche Mindestmafs an
kultureller Identität, welche notwendige Voraussetzun-
gen für gelingende Staaten sind. Wir wollen den
souveränen, demokratischen Nationalstaat erhalten.
Nur dort kann Volkssouveränität gelebt werden, die
Mutter und das Herzstück der Demokratie.

EU und Europa

Zurückführung der Europäischen Union in einen
Staatenbund souveräner Staaten

Eine Union europäischer Staaten wird nur dann eine
Zukunft haben, wenn es gelingt, dem sich immer
schneller drehenden Rad der Entdemokratisierung und
zentralisierung in die Speichen zu greifen, bevor die
heutige EU durch die Pervertierung ihrer Gründungs-
idee an sich selbst zugrunde geht. Die wirtschafts-
geschichtlich ungewöhnliche Idee einer Einheits-
währung für wirtschaftlich völlig unterschiedlich
entwickelte Staaten ist gescheitert. Es handelte sich um
eine politische Wunschvorstellung, die mit

ökonomischen Gesetzen nichtin Einklang zu bringenist.

Mit der Einführung des sogenannten „Corona-
Wiederaufbaupaktes wurde die Transferunion in eine

29

Wir halten einen Austritt
Deutschlands aus der
Europäischen Union

neue Dimension gehoben. Diese Transferunion steht und die Gründung einer
nichtnur im Widerspruch zu den europäischen neuen europäischen

Verträgen und den Versprechen der deutschen Politiker,

Wirtschafts- und Inter-
essengemeinschaft für

sondern wird den Abstieg aller europäischen notwendig.

Volkswirtschaften und Konflikte zwischen den Staaten
zur Folge haben.

Ein vergleichbares Versagen zeigt die EU in der jahrelang
anhaltenden Migrationskrise. Auch das Krisen-
management in der Corona-Krise war verheerend.

Die Fehlentwicklungen in der Klima- und Energiepolitik
treibt die EU durch unvorstellbar kostspielige Gesetzes-
und Subventionspakete voran. Das langfristige Unheil
dieser Verschuldungs- und Umverteilungs-Eskapaden
wird vor allem den deutschen Steuerzahler treffen.

30

EU und Europa

Die Vehemenz, mit welcher die Europäische Union die
Transformation zum planwirtschaftlichen Superstaat

in den letzten Jahren vorangetrieben hat, hat uns zu der
Erkenntnis gebracht, dass sich unsere grundlegenden
Reformansätze in dieser EU nicht verwirklichen lassen.
Wir halten einen Austritt Deutschlands aus der
Europäischen Union und die Gründung einer neuen
europäischen Wirtschafts- und Interessengemeinschaft
für notwendig.

31

Steuern und
Finanzen

34 Steuern und Finanzen

Steuern und Finanzen

Keine EU-Steuern

Das Recht, Steuern zu erheben, muss vollständig in
nationaler Kompetenz der Mitgliedstaaten der Europäi-
schen Union verbleiben. Steuereinnahmen bestimmen
die nationalen Haushalte. Die Budgethoheit stellt die
zentrale Kontrollmöglichkeit des Parlaments über das
Regierungshandeln dar. Die im Zusammenhang mit dem
"Recovery Fonds’ von der EU-Kommission geforderte
Einführung mehrerer Steuern in der Rechts-

setzungs- und Ertragshoheit der EU muss daher
verhindert werden.

Für ein Steuersystem mit wenigen Steuerarten und
verständlicher Systematik

Das Steuerreformkonzept des ehemaligen
Verfassungsrichters Kirchhoff ist ein gutes Beispiel für
eine grofse Reform. In Orientierung daran könnten, bei
Konzentration auf die beiden grofsen Steuerarten
(Umsatzsteuer und Einkommenssteuer), die

Das Recht, Steuern zu

erheben, muss Grundsteuer, die Gewerbesteuer und etliche, nach
vollständig in ihrem Aufkommen betrachtet, weitere kleinere

nationaler Kompetenz f

der Mitaliedstaaten der  Verbrauchsteuern auf Bundesebene ersatzlos entfallen.
Europäischen So zum Beispiel die Energiesteuer, die

umion verbleiben, Schaumweinsteuer und die Kaffeesteuer. Auf

Landesebene sollten die Biersteuer und auf der
Kommunalebene die Vergnügungssteuer, die
Schankerlaubnissteuer, die Jagd- und Fischereisteuer
und die Zweitwohnungssteuer entfallen. Ihnen allen ist
gemeinsam, dass sie verwaltungsaufwendig und
aufkommensschwach sind. Sie leisten keinen nennens-
werten Beitrag zur Staatsfinanzierung.

SteuernundFinanzen 35

Abschaffung der Substanzsteuern

Ähnlich verhält es sich mit den sogenannten
Substanzsteuern. Dazu zählen die schon erwähnte
Grundsteuer, die an Haus- und Grundbesitz anknüpft,
die Vermögenssteuer sowie die Erbschafts- und

Schenkungssteuer. Bei derzeit rund 800 Mrd. jährlichem
Steueraufkommen des Gesamtstaates liegt deren Anteil

insgesamt bei einem niedrigen einstelligen

Prozentbetrag. Zudem ist die Vermögenssteuer bereits

1995 für verfassungswidrig erklärt worden.

Die Grundsteuer wird in der Gesetzgebungshoheit des
Bundes geregelt. Ihr Aufkommen fliefst ausschliefslich
den Kommunen zu. Auch sie wurde wegen der
Bewertungsprobleme des Grund- und
Gebäudevermögens für verfassungswidrig erklärt. Ihr
steuersystematischer Erhebungsgrund ist unklar,
weshalb ihre Existenzberechtigung und ihre
Verfassungsmäßsigkeit in der Rechtswissenschaft
hochumstritten ist. Wenn sie in Wahrheit eine zweite
Vermögenssteuer ist, wofür vieles spricht, ist sie
ebenfalls konfiskatorisch und daher abzuschaffen. Ihr
Aufkommen entspricht etwa 10 % des gesamten
Steueraufkommens der Kommunen.

Als Ersatzeinnahme für die Kommunen strebt die AfD
eine Änderung des Verteilungsschlüssels der großen
Steuerarten zu Gunsten der Kommunen an.
Grundsätzlich ist es das Ziel der AfD, die Steuer- und
Abgabenbelastung in Deutschland deutlich zu senken

(laut OECD weltweit nur noch von Belgien übertroffen!).

Grundsätzlich ist es das
Ziel der AfD, die Steuer-
und Abgabenlast in
Deutschland deutlich zu
senken.

Es gilt, die in den letzten
Jahren erfolgte massive
Abwanderung von deut-
schen Unternehmen
und Fachkräften ins
Ausland zu stoppen.

36

Wir wollen den Erwerb
von Wohneigentum
erleichtern.

Steuern und Finanzen

Es gilt, die in den letzten Jahren erfolgte massive Abwan-
derung von deutschen Unternehmen und Fachkräften
ins Ausland zu stoppen und den Wirtschaftsstandort
Deutschland für Investitionen wieder attraktiv zu ma-
chen.

Die Wirtschaftsstruktur Deutschlands ist, am welt-
weiten Malsstab gemessen, extrem stark von mittel-
ständischen Unternehmen geprägt. Sie beschäftigen

die relativ meisten Arbeitnehmer, bilden mit Abstand

die meisten Lehrlinge aus und existieren vielfach über
mehrere Generationen. Im Falle des Unternehmens-
übergangs im Erbgang, insbesondere auch innerhalb der
Familie, wurde bei größeren Vermögen umso drastischer
auf die Unternehmenssubstanz im Wege der Erbschafts-
steuer steuerlich zugegriffen mit der Folge der Zerstö-
rung funktionierender Unternehmen.
Generationsübergreifender Werterhalt zum Nutzen

von uns allen wird seit Jahrzehnten durch sozialistische
Gleichheitsvorstellungen und klassenkämpferische
Neidgefühle verhindert. Dem stellt sich die AfD
entgegen. Erbgänge sind zunächst Angelegenheiten
zwischen Erblassern und Erben. Es gibt keinen
akzeptablen Grund, aus Trauerfällen Steuerfälle zuma-
chen.

Die AfD fordert daher die Abschaffung der
Erbschaftssteuer.

SteuernundFinanzen 37

Wir wollen ein Land von Wohnungseigentümern
werden

Der Erwerb von Grund und Boden und von Wohneigen-
tum unterliegt der Grunderwerbsteuer. Sie bemisst sich
in Prozenten der Erwerbskosten und erschwert damit
die Eigentumsbildung breiter Bevölkerungsschichten,
die weit überwiegend als Ziel eines selbstbestimmten
Lebens den Erwerb von Wohnungseigentum anstreben.
Deutschland ist europaweit mit unter 50 % das Land mit
der geringsten Eigentumsquote bei der Wohnungsver-
sorgung. Das wollen wir ändern und den Erwerb von
Wohneigentum unter anderem durch die Abschaffung
der Grunderwerbsteuer beim Erwerb von Immobilien

zur Eigennutzung erleichtern. Bereits vor Jahrzehnten —
Die AfD setzt sich ein

gab es für eine gewisse Zeit unter bestimmten Voraus- _fürdie Reduzierung der
setzungen eine einmalige Grunderwerbssteuer- Steuer- und Abgaben-
befreiung für den Erwerb von selbstgenutztem miele
Wohnungseigentum.

Elementare Steuergerechtigkeit statt Nominalwert-
prinzip und Ungleichbehandlung von Steuerbürgern

Da es nahezu immer eine gewisse Inflationierung des
Geldes gibt, war und istes ein Akt grober
Ungerechtigkeit, die progressive Tabellenbelastung des
Einkommens und die Bemessungsgrundlagen, die
durch Freibeträge, Freigrenzen, Pauschbeträge und
Pauschalen mafsgeblich bestimmt werden, über lange
Zeiträume nominal konstant zu lassen („kalte
Progression”). Bei einer Inflation von jährlich 2 %
schmelzen Freibeträge innerhalb von 10 Jahren um über
20 % ihres ursprünglich gesetzgeberisch gewollten Geld-
wertes ab.

38

Die AfD fordert die
Einführung einer
Digitalsteuer, die nicht
auf den Gewinn,
sondern auf den Umsatz
der Digital-Konzerne
erhoben wird.

Steuern und Finanzen

Das Gleiche gilt für die überproportional steigende Steu-
erlast bei lediglich nominalem Einkommenswachstum
durch die Festschreibung der Steuertabelle.

Seit der 18. Legislaturperiode des Bundestages werden
zwar durch Einzelgesetze die Kinder- und die Grund-
freibeträge an die Inflation angepasst und inzwischen
auch die Tabellenbeträge angehoben. Im Jahrzehnt davor
ist dies jedoch nicht geschehen, mit der Folge,

dass manin Deutschland, etwa im krassen Unterschied
zur Schweiz, mit leicht überdurchschnittlichem
Einkommen bereits zu Teilen mit dem Spitzensteuersatz
belastet wird. Es ist ein Gebot elementarer Gerechtigkeit,
für beide Phänomene eine automatische gesetz-
geberische Dynamisierung vorzunehmen. Dies wurde
daher auch von der AfD-Bundestagsfraktion wiederholt
in Antragsform gefordert.

Genauso elementar wird die Steuergerechtigkeit
verletzt, wenn der Solidaritätszuschlag, der aus Gründen
der Finanzierung des Aufbau Ost eingeführt wurde, nach
dem Wegfall dieser Transferleistungen nicht ebenfalls
entfällt. Mafsgebliche Verfassungsrechtler halten die
nunmehrige vollständige Abschaffung des „Soli für
verfassungsrechtlich zwingend geboten.

Die AfD-Bundestagsfraktion hat bereits mehrfach
entsprechende Anträge im Bundestag zur Abstimmung
gestellt, die alle mit breiter Mehrheit abgelehnt wurden.
Mit Wirkung für 2021 wurde nunmehr für untere und
mittlere Einkommensgruppen der Steuerzuschlag
abgeschafft - mit der Konsequenz, dass etwa die Hälfte
der 20 Mrd. des jährlichen Aufkommens entfällt.

Steuern und Finanzen

Die zweite Hälfte des Aufkommens bleibt jedoch -
beispielsweise auch für alle kleinen GmbH - für die
übrigen Einkommensteuerzahler als Zusatzbelastung
erhalten. Der Grundsatz der Gleichbehandlung aller
Steuerzahler wird damit eklatant verletzt.

Das Bundesverfassungsgericht wird über die
Rechtmäfsigkeit dieses Vorgehens zu entscheiden haben.

Besteuerung an ’Digitalzeitalter anpassen:
Einführung einer Digitalsteuer

Die AfD fordert die Einführung einer Digitalsteuer für
Tech-Riesen (,Gafam“), die nicht auf den Gewinn,
sondern auf den Umsatz dieser Unternehmen erhoben
wird. Denkbar ist dabei die Besteuerung von Umsätzen
aus dem Platzieren personalisierter Werbung auf der
Basis von Nutzerdaten, Umsätzen aus Vermittlungs-
diensten zwischen Nutzern und Umsätzen aus dem
Verkauf von Nutzerdaten. Da die Verhandlungen über
die Einführung einer Digitalsteuer auf europäischer
Ebene vorerst gescheitert sind, planen bereits mehrere
europäische Länder eine entsprechende Regelung auf
nationaler Ebene oder haben eine solche in Teilen
bereits umgesetzt. Deutschland sollte sich zügig
anschliefsen und eine nationale Digitalsteuer auf den
Weg bringen.

39

KAPITEL

Wirtschaft: Für
einen „Blue Deal"

42

Die AfD steht für eine
marktwirtschaftliche
Wirtschaftspolitik. Wir
wollen die Wirtschaft
von politisch herbei-
geführtenBelastungen
komplett befreien.

Wirtschaft: Für einen „Blue Deal“

Wirtschaft: Für einen „Blue Deal“

Wirtschafts- und Innovationspolitik für Deutschland

Die AfD steht für eine marktwirtschaftliche
Wirtschaftspolitik: Schutz des Eigentums,
Vertragsfreiheit, die Einheit von Handeln und Haftung,
Berufsfreiheit und eine wettbewerbliche
Wirtschaftsordnung sind elementare Bestandteile einer
freiheitlichen und prosperierenden bürgerlichen
Gesellschaft. Diese wollen wir erhalten und
Fehlentwicklungen korrigieren.

In den letzten Jahren haben sich die Bedingungen zur
Mehrung des Wohlstandes verschlechtert. Die weltweite
Konjunktur hat an Fahrt verloren. Politisch initiierte
Umbrüche wie die „Klimarettung' fordern enorme
Investitionen und Umstrukturierungen und treffen
manche Branchen brachial. So wird der Kern der
deutschen Wirtschaft geschwächt. Gleichzeitig liegt
Deutschland gemäfs einer Studie der Europäischen
zentralbank bei diversen Wohlstandsindikatoren im
europäischen Vergleich auf den hinteren Plätzen.

In dieser Situation legt die Corona-Pandemie die
internationale Wirtschaft zeitweise weitgehend lahm
und führt zu einem dramatischen Rückgang der
Wertschöpfung. Die deutsche Volkswirtschaft ist
exportorientiert und von internationaler Arbeitsteilung
abhängig. Deshalb ist unser Land ganz besonders von
den weltweiten Auswirkungen der Corona-Krise
betroffen.

Die AfD will deshalb die Volkswirtschaft aufeinen
normalen Entwicklungspfad zurückführen.

Wirtschaft: Für einen „BlueDeal“ 43

Den Mittelstand entlasten

Der deutsche Mittelstand stellt nicht nur einen Grofsteil
der Arbeitsplätze bereit, er bildet auch mit seiner
Flexibilität und seinem Erfindungsreichtum einen
Stabilitätsanker für unsere Wirtschaftsstruktur. Dies
schafft er oft ohne staatliche Subventionen und Steuer-
erleichterungen, von denen vor allem Grofskonzerne
profitieren.

Er verkraftet aber nicht die Doppelbelastung durch
politisch initiierte Strukturbrüche und pandemie-
bedingte Einschränkungen. Billionenschwere Rettungs-
pakete können nicht eine Erholung der Realwirtschaft
ersetzen oder über Nacht herbeiführen; stattdessen
wird damit nur eine Umverteilung im Sinne der „Grofsen
Transformation und zum Nachteil unserer Konkurrenz-
fähigkeit auf dem Weltmarkt vorangetrieben.

Politisch induzierte Belastungen aussetzen

Wir setzen uns für die

Wir wollen die Wirtschaft von politisch herbeigeführten .,......;
ntschädigung der

Belastungen komplett befreien. Dafür wollen wir, wo Wirtschaftssektoren
nötig, auch die Rechtslage anpassen. Davon unbenom- . nn
. . B . von den Lorona-Ma]jJs-
men setzen wir uns auf nationaler Ebene für eine Ent- nahmen besonders
schädigung der Wirtschaftssektoren ein, die von betroffen sind.

den Corona-Mafsnahmen besonders betroffen sind.

Technologieführerschaft mit „Blue Deal“

Der deutsche Erfindergeist ist legendär. Heute sind
Erfinder aber mobil und wandern oft aus, da sie hier mit
Auflagen gegängelt werden und nicht die Entwicklungs-
unterstützung und Geldgeber finden, die den Aufbau
eines Unternehmens fördern.

44 Wirtschaft: Für einen „Blue Deal“

Diese Entwicklung müssen wir umkehren. Wir wollen
mit einem Blue-Deal-Entwicklungsplan Investitionen in
Technologieführerschaft entfesseln, indem wir:

» Schulbildung, Hochschulbildung und Forschung
verstärkt auf MINT-Fächer ausrichten,

» die Umsetzung wissenschaftlicher Erkenntnisse in
Produkte fördern,

» Selbständigkeit und Erfindergeist fördern und damit
die Dynamik kleiner innovativer Einheiten nutzen.

Technologische und wirtschaftliche Zukunftschancen
sehen wir beispielsweise

» im Wiederaufbau eines nationalen
pharmazeutisch-medizinischen Kompetenz-Clusters,

» in Aufstellung und Umsetzung eines Entwicklungs-
plans Quantencomputing,

» inder Entwicklung von Technologien und Geschäfts-
modellen zur Weltraumnutzung.

Dazu wollen wir

Die AfD will die » die Rahmenbedingungen für Investitionen im Inland
Rahmenbedingungen verbessern,

Jurinvestilionendurd‘ > die Infrastruktur auf einen zeitgemäßen Stand
eine zeitgemäße .
Infrastruktur und die bringen,

Beendigung von Staats » Staatseingriffe in den Energiemarkt beenden, damit

eingriffen in den .
Energiemarkt der Wirtschaftsstandort Deutschland international

verbessern. wettbewerbsfähig bleibt.

Wirtschaft: Für einen „BlueDeal“ 45

Wir wollen Bürokratie abbauen und den Staat effizienter
machen durch

» Entschlackung und Flexibilisierung des Arbeitsrechts,

» Beendigung der Benachteiligung des Mittelstands
gegenüber multinationalen Grofskonzernen,

» Sicherstellung verlässlicher rechtlicher und regulato-
rischer Rahmenbedingungen ,

» Überprüfung, ggf. Abbau bzw. Befristung von Regulie-
rungen und Subventionen,

» Überprüfung und Reduzierung der wirtschaftsrele-
vanten Rechtsgebiete auf Streitanfälligkeit.

Soziale Marktwirtschaft statt
sozialistischer „Industriepolitik“

Zu unserem Verständnis von Sozialer Marktwirtschaft

gehört ein gedeihliches Zusammenwirken von
Unternehmern und Beschäftigten. Sie sind keine
Gegner, sondern kooperieren im Geiste der
Sozialpartnerschaft, um betriebliche Ziele zu erreichen
und Interessengegensätze ins Gleichgewicht zu bringen.
Als AfD bekennen wir uns zur Mitwirkung und
Mitbestimmung der Beschäftigten in den Betrieben und
zu allgemeinverbindlichen Tarifverträgen. Forderungen
nach Enteignungen, Abschaffung von Privateigentum
und Vertragsfreiheit, hetzerische Klassenkampfrhetorik
oder vorsätzlich herbeigeführte Konflikte durch
Vertreter linker Parteien lehnen wir dagegen
entschieden ab. Sie gefährden den sozialen Frieden und
eine vertrauensvolle Zusammenarbeit.

Forderungen nach Ent-
eignungen, Abschaffung
von Privateigentum und
Vertragsfreiheit lehnen
wir entschieden ab.

46

Wirtschaft: Für einen „Blue Deal“

Die Zukunft der organisierten Unternehmer- und
Beschäftigtenvertreter ist blau:

Unser sozialpolitisches Ziel bleibt „Wohlstand für alle“!

Wirlehnen es ab, dass sich die Bundesregierung
zunehmend als Unternehmer versteht und betätigt. Sie
nimmt direkten Einfluss auf Unternehmen und Innova-
tionen und versucht die Wirtschaft durch Vorgaben und
Subventionen zu steuern. Diese Art von Staatswirtschaft
führt regelmäfsig in den wirtschaftlichen Niedergang.
Wir wollen die Soziale Marktwirtschaft von

Ludwig Erhard wiederbeleben und Wohlstand für alle
schaffen. Die Aufgabe des Staates soll die Erhaltung des
Wettbewerbs und die Verhinderung von Monopolen,
Kartellen und sonstigen, den Marktmechanismus
schädigenden Einflüssen sein.

Der Schutz vor Wettbewerb, staatlich verordneten
Preiserhöhungen für vermeintlich gute Zwecke oder die
von der Bundesregierung propagierte Schaffung
„europäischer Monopole’ gehören nicht dazu.

47

Der Euro ist
gescheitert

50

Wir wollen die
„Iransferunion“
aufkündigen und
wieder eine nationale
Währung einführen.

Der Euro ist gescheitert

Der Euro ist gescheitert

Die seit 30 Jahren laufende, schleichende Umwandlung
des Staatenbunds Europa in einen EU-Zentralstaat ist
von den europäischen Völkern nie gewollt oder
demokratisch legitimiert worden. Der Euro besitzt nicht
die institutionelle Grundlage, die unabdingbare
verfassungsrechtliche Voraussetzung für seine
Legitimität wäre. Die EU geriert sich heute als
Bundesstaat - und nicht mehr als Staatenbund
souveräner Nationalstaaten.

Die Geschäftsgrundlage des Euro ist ebenfalls schon seit
über zehn Jahren zerstört. Mit der Corona-Krise sind nun
auch noch die letzten Hemmungen zum
Verfassungsbruch gefallen. Wir erleben heute eine
vertragswidrige, sogar sozialistische Gemeinschafts-
haftung, Transferzahlungen und eine Verschuldung auf
Ebene der EU sowie eine verbotene Staatsfinanzierung
und mandatswidrige Wirtschaftspolitik durch die EZB -
alles weitgehend auf Deutschlands Kosten.

Deutschland muss aus dem untergehenden
Euro-System austreten

Die AfD steht auch in der Währungspolitik für ein auf
sozialer Marktwirtschaft und nationaler Eigenverant-
wortung basierendes, freundschaftliches Miteinander in
Europa.

Die Grundlage der Eurowährung war: Vermeidung über-
bordender Schulden in allen EU-Staaten und keine
Haftung der Staaten untereinander. Beide Regeln
werden permanent gebrochen. Der Euro wird heute

DerEuroistgescheitertt 51

jeden Tag mit Milliardenbeträgen „gerettet. Dies über
billionenschwere Garantien, Bürgschaften und sogar
direkte Geldgeschenke an „notleidende’ Staaten;

über „Fazilitäten‘, „Wirtschaftsstabilisierungsfonds',
„Next-Generation-EU -Fond, „Europäische
Stabilitäts-Mechanismen‘, EU-Kurzarbeiterprogramme,
zinsfreie Kredite, Verrechnungskredite (Target?2),
Aufkäufe von Staatsanleihen durch das EZB-Noten-
banksystem.

All diese Mafßsnahmen sind ebenso wie geplante
EU-Steuern illegal, da sie gegen die Verbote der
monetären Staatsfinanzierung oder der gegenseitigen
Haftungsübernahme verstofsen.

Deshalb muss Deutschland diese „Transferunion — |
.. . Die AfD fordert eine

aufkündigen und den Irrweg der Dauerrettung durch sofortige Einstellung
Wiedereinführung einer nationalen Währung beenden, Jegliche

f llel ihehal desE d . Kreditaufnahme derEU,
ggf. unter paralleler Beibehaltung des Euro oder einer die zu Lasten künftiger
ECU-ähnlichen flexibleren Verrechnungseinheit. Jede deutscher Generationen
weitere Teilnahme an der „Rettungspolitik” kommteiner "©
Insolvenzverschleppung auf Kosten künftiger deutscher

Steuerzahler gleich.

Keine Schuldenaufnahme durch dieEU,
Hilfskredite nur auf nationaler Ebene

Die EU mafßst sich heute an, entgegen dem Wortlaut
der EU-Verträge eigene Schulden aufzunehmen. Dabei
werden die Mittel durch Neuverschuldung beschafft
und anschliefsend von EU-Bürokraten über unzählige,
zumeist ideologisch motivierte Projekte nach Guts-
herrenart ausgegeben. Haftung und Tilgung verbleiben
jedoch bei den Nationalstaaten. Die Mithaftung
Deutschlands für diese von der EU aufgenommenen

52

Die AfD fordert: Die
Bundesregierung

darf keinen EU-
Corona-Programmen
zustimmen, für die
Deutschland nur mit-
haftet und aus denen es
wenig Leistung erhält.

Der Euro ist gescheitert

Kredite, die weitgehend den Euro-Krisenländern
zugutekommen werden, führt zwangsläufig zu einer
Verschlechterung der Bonität Deutschlands.

Die AfD fordert eine sofortige Einstellung jeglicher
Kreditaufnahme der EU, die zu Lasten künftiger
deutscher Generationen gehen würde. Wenn Kredite zur
Abmilderung der Folgen von überzogenen Corona-
Mafsnahmen der Regierung aufgenommen werden,
dann darf dies ausschließlich auf nationaler Ebene
geschehen.

Zusätzliche Inflationierung durch überzogene
Corona-Maßnahmen beenden

Durch die überzogene Politik der Bundesregierung, der
EU und der EZB in der Corona-Krise drohen gleichzeitig
eine wirtschaftliche Depression und eine massive
Teuerung, da grofse Teile der „Corona-Förder-
programme unproduktiv sind und als reine Geld-
mengenausweitung in den Konsum fließen, der auf der
Angebotsseite nicht befriedigt werden kann.

Die AfD fordert, die Corona-bedingten Ausgabenpro-
gramme und die dazugehörige Verschuldung auf das
notwendige Mais zu beschränken. Die Bundesregierung
darf keinen EU-Corona-Programmen zustimmen, für die
Deutschland nur mithaftet, aus denen es wenig Leistung
erhält, aber durch die drohende Inflation zusätzliche
Schäden erleiden wird.

Der Euro ist gescheitert

EZB-Nullzinspolitik beenden

Die EZB sollte eine zweite Deutsche Bundesbank sein.
Stattdessen betreibt sie eine Währungspolitik der
wirtschaftsfeindlichen Null- und Negativzinspolitik.
Diese zerstört langfristig alle kapitalgedeckten Alters-
sicherungssysteme von Betriebsrenten, über staatlich
geförderte kapitalgedeckte Rentensysteme, private
Lebensversicherungen bis zu privaten Sparvermögen.
Inzwischen fehlen über die vergangenen Jahre mehr
als 500 Mrd. Euro für die Altersvorsorge der Deutschen.
Die Negativzinspolitik der EZB führt dazu, dass nicht
mehr wettbewerbsfähige Unternehmen als „ Zombies‘
im Markt verbleiben und den gesunden Unternehmen
einen unfairen Wettbewerb liefern.

Nullzinsen führen zu gravierender Kapitalfehlallokation
in der Realwirtschaft. In Deutschland könnten
inzwischen 20% bis 50% der Firmen ihre Kapitalkosten
in einem normalen Zinsumfeld nicht mehr verdienen.
Das kommt einer permanenten, staatlich geförderten
Insolvenzverschleppung durch die Bundesregierung
gleich. Bereits nach wenigen Jahren wird das zu kas-
kadenartigen Zusammenbrüchen von sogar bis dahin
noch gesunden Unternehmen führen und den Auftakt
zu einer umfassenden Depression bilden. Negativzin-
sen führen außerdem zu weiteren volkswirtschaftlichen
Schäden, weil der deutsche Kapitalstock von vielen
Billionen Euro nicht rentabel eingesetzt werden kann.

Die durch die EZB-Politik verursachte Wechselkursab-
wertung und die Preisexplosion am Aktien- und Immo-
bilienmarkt zerstören die Kaufkraft, wovon insbesondere

53

Die AfD will die
kapitalgedeckten Alters-
sicherungssysteme wie
Betriebsrenten, private
Lebens-
versicherungen bis zu
privaten Sparvermögen
erhalten und lehnt des-
wegen die Nullzins-
politik der EZB
entschieden ab.

54

Die AfD fordert: Alle

Mafsnahmen der Euro-

päischen Zentralbank
zur Manipulation des
freien Kapitalmarkts
müssen eingestellt
werden.

Der Euro ist gescheitert

Arbeitnehmer und Rentner betroffen sind. Die Zins-
manipulation treibt die Immobilienpreise nach oben
und damit auch die Mieten. Negativzinsen enteignen
Sparer, Mieter und Rentner und führen langfristig zu
Altersarmut.

Weiterhin führt die EZB-Politik zur Zerstörung der
Märkte für Anleihen. Während die Deutsche Bundes-
bank nur einmal in 50 Jahren Staatsanleihen angekauft
hat, ist das für die EZB seit 2015 das tägliche „Geschäfts -
modell‘. Per Ende 2020 hatte sie für fünf Billionen Euro
staatliche und private Anleihen erworben; zu großen
Teilen betreibt die EZB dabei auch noch ideologische
und verbotene Wirtschaftspolitik. Diese suprastaatliche
„Rettungspolitik’ verletzt geltendes EU-Recht, darunter
das Verbot der Staatsfinanzierung durch die Zentralbank
und das Verbot der Haftung für Schulden anderer Mit-
gliedstaaten. Die Rettungspolitik verstöfst zudem gegen
alle seit den 1990er-Jahren gegebenen Zusicherungen,
„niemals eine Haftung Deutschlands für andere Staaten
zuzulassen.

Die AfD fordert eine Rückkehr zum Recht: (1) Alle
Mafßsnahmen der EZB zur Manipulation des freien
Kapitalmarkts müssen eingestellt werden, (2) keine
Staatsfinanzierung durch die EZB, (3) keine faktische
Vergemeinschaftung der Schulden, (4) Beschränkung
der EZB aufreine Geldpolitik, (5) keine heimliche und
hoch ideologische Wirtschaftspolitik über „Green Deal‘
und Anleihekäufe und (6) Ende der Nullzins-Politik und
der EZB-Anleihekaufprogramme. (7) Die Bundesbank
wird aufgefordert, sich mit allen Mitteln bei der EZB
dafür einzusetzen, um eine Änderung der Geschäfts-
ordnung bei den Abstimmungsmodalitäten bzgl. der
Stimmrechte der Mitglieder des EZB-Rates bei geld-

DerEuroistgescheitert 55

politischen Entscheidungen herbeizuführen. Die Stimm-
rechte der Mitglieder des EZB-Rates haben sich am
Anteil des haftenden Eigenkapitals der Mitgliedsstaaten
zu orientieren. Das bisher praktizierte Rotations-
verfahren ist zu verwerfen.

Wohlstand und sozialer Frieden in Europa durch
Wiedereinführung nationaler Währungen

Der Euro ist für ein Wirtschaftsgebiet mit derzeit 19 sehr

unterschiedlich leistungsfähigen Volkswirtschaften eine

Fehlkonstruktion und kann in dieser Form weder Öko-

nomisch noch sozial funktionieren. Die Arbeitslosigkeit

ist seit Jahren im Süden Europas erschreckend hoch als

Folge der fehlenden bzw. verloren gegangenen Wett- nu
bewerbsfähigkeit dieser Länder innerhalb des Euro- Die wiedereingeführte
Korsetts. Auch die soziale Euro-Krise ist ein systemi- Deulsehe Mark wurde

. . . . in ihre hohe Kaufkraft
sches Problem. Die Lösung besteht in der Wiedereinfüh- gegenüber den anderen
rung der nationalen Währungen, in denen sich die Ländern analog zur

. . 2: . hohen Leistungsfä-
Leistungen der jeweiligen Volkswirtschaften ef

higkeit der deutschen

widerspiegeln. Wirtschaft
wiedergewinnen.

Durch nationale Währungen wird jeder Staat wieder für

seine Wirtschafts-, Währungs- und Finanzpolitik verant-

wortlich und erhält damit seine politische Souveränität

zurück. Damit werden auch die internen Konflikte in der

Euro-Zone gelöst: Keinem früheren Euroland müssen

danach noch strenge Vorgaben gemacht werden, um

seine im Euro-Korsett verloren gegangene Wettbe-

werbsfähigkeit wiederherzustellen.

Die Staaten können ihre Wettbewerbsfähigkeit dann
wieder, wie vor der Euro-Einführung, durch eigene

56

Die AfD fordert den
dauerhaften Erhalt der
uneingeschränkten
Bargeldnutzung als
wichtigem bürgerlichen
Freiheitsrecht und seine
Verankerung im Grund-
gesetz.

Der Euro ist gescheitert

Entscheidung über die Veränderung des Wechselkurses
wiederherstellen. Die wiedereingeführte Deutsche Mark
würde ihre hohe Kaufkraft gegenüber den anderen
Ländern analog zur hohen Leistungsfähigkeit der
deutschen Wirtschaft wiedergewinnen. Die Senkung der
Importpreise würde zu steigenden Realeinkommen
führen, die der gesamten deutschen Volkswirtschaft
zugute kämen und nicht nur, wie derzeit, einigen Export-
firmen.

Der ehemalige Bundeswirtschafts- und Finanzminister
Schiller hatte dies einst so formuliert: „Jede Währungs-
aufwertung ist eine Sozial-Dividende für das deutsche
Volk, denn ihr Könnt danach im Supermarkt alles billiger
kaufen‘.

Der AfD ist bewusst, dass die Wiedereinführung der
nationalen Währung nicht ohne Umstellungs-
belastungen erfolgen kann. Diese werden jedoch niedri-
ger sein als die Kosten des weiteren Verbleibs im
Eurosystem.

Erhalt des Bargelds: Bargeld ist Freiheit und
Schutz vor Enteignung und Totalüberwachung

Unser Bargeld istin Gefahr. Mit Unterstützung von
Bundesregierung, IWF und EZB wird die Abschaffung
des Bargelds vorbereitet. Mit dessen Abschaffung würde
die Rolle des Geldes als gebührenfreiem Wert-

speicher eingeschränkt. Eine Abschaffung des Bargeldes
macht im Krisenfall die Enteignung von Kontoinhabern
möglich. Bargeld ist ein natürliches Bollwerk gegen
enteignende Negativzinsen. Ein Bargeldverbot würde
bedeuten, dass Zahlungsvorgänge nur noch elektronisch
stattfinden können. Dies eröffnet dem Staat und Banken

DerEuroistgescheitert 57

die totale Kontrolle über alle Geldströme und
Wirtschaftsaktivitäten, über alle Finanztransaktionen,
unter Umständen sogar über Aufenthaltsorte aller
Bürger. Aus dem gläsernen Bankkunden würde der
gläserne Mensch - Vollüberwachung bis in private, ja
intime Lebensbereiche. Mit einem freiheitlichen Rechts-
staat sind solche Kontrollmöglichkeiten nicht vereinbar.

Die AfD fordert den dauerhaften Erhalt der
uneingeschränkten Bargeldnutzung als wichtigem
bürgerlichen Freiheitsrecht und seine Verankerung
im Grundgesetz. Die AfD spricht sich auch gegen von
zentralbanken herausgegebene digitale Währungen aus,
die ein Einfallstor für die schleichende Abschaffung des
Bargeldes wären. Die Verfügbarkeit und Wert- —
Als Sofortmafsnahme

aufbewahrungsfunktion des Bargeldes muss erhalten fordert die AfD die Bun-

bleiben. desregierung und die
Deutsche Bundesbank

auf, die Überbeanspru-
Target-2-Risiko beenden chung des Verrech-
IRRE nungskontos „Target-2“
Auf der Aktivseite der Bundesbankbilanz belaufen sich _ zu unterbinden.

die Überziehungskredite einzelner Notenbanken der
Euro-Staaten (Target-2-Forderungen der
Gläubigerstaaten) aufüber eine Billion Euro. Diese
Forderungen der Deutschen Bundesbank sind
unbesicherte, derzeit unverzinste, nicht fällig stellbare
Forderungen. Ihre Beitreibung ist mangels Sicherheiten
ungewiss. Es gibt eine Reihe wahrscheinlicher Szenarien
für eine Ausbuchung dieser Beträge. Target-2-
Wertverluste stellen ein hohes Milliarden-Risiko für den
Bundeshaushalt und damit für den deutschen
Steuerzahler dar. Sicher ist, dass die EZB als Anspruchs-
gegner der Bundesbank im Falle einer (Teil-)Auflösung
der Eurozone die deutschen Target-Forderungen nur

58

Die AfD fordert die
Deutsche Bundesbank
auf, das im Ausland
gelagerte Staatsgold
vollständig und
umgehend nach
Deutschland zu über-
führen.

Der Euro ist gescheitert

noch in abgewerteten Euros oder gar nicht mehr zurück-
zahlen wird. Wegen der substanzlosen Target-
Forderungen kommen die noch immer sehr hohen
deutschen Aufsenhandelsüberschüsse innerhalb der
Eurozone der deutschen Volkswirtschaft kaum noch
zugute.

Als Sofortmafgnahme fordert die AfD darum die Bundes-
regierung und die Deutsche Bundesbank auf, die bislang
geduldete Überbeanspruchung des Verrechnungskontos
„Target-2 zu unterbinden. Die Target-2-Forderungen der
Deutschen Bundesbank sind abzuschmelzen,

täglich auszugleichen, angemessen zu verzinsen und mit
Sicherheiten zu unterlegen. Die AfD ist auch offen für
weitere sinnvolle Vorschläge, die Target-Forderungen zu
„monetarisieren bzw. nutzbar zu machen. So Könnte die
Bundesbank z. B. ihre Target-Forderungen am Kapital-
markt an Investoren weiterverkaufen und die erhaltenen
Mittel in ausländischen Aktien, ausländischem Gold
oder anderen Sachwerte anlegen.

Nationale Investitionen

Solange der Euro zu Lasten Deutschlands weiterbesteht,
kann und sollte auch Deutschland einen grofsen Teil
seines Refinanzierungsbedarfs über die EZB und die EU
abwickeln. Notprogramm-Mittel sind strikt für Hilfen
und Investitionen in Deutschland einzusetzen und wo
immer möglich über EU oder EZB zu finanzieren. Auch
der deutsche Target-Forderungssaldo sollte dafür
mobilisiert werden.

Der Euro ist gescheitert

Absicherung gegen die Eurokrise,
Geldsystem reformieren, deutsches Staatsgold
vollständig zurück nach Deutschland

Deutschland muss auch in einem plötzlichen währungs-
politischen Krisenfall handlungsfähig sein. Bei einer
Wiedereinführung der Deutschen Mark Könnte das
teilweise im Ausland gelagerte Staatsgold als

temporäre Deckungsoption dienen. Gold istnach aller
historischer Erfahrung eine potenzielle Deckung für eine
neue Währung, was besonders in Krisen-

situationen bzw. nach Einführung einer neuen Währung
zum Vertrauensaufbau relevant wird. Staatsgold muss
ohne Gegenparteien- oder Verwahrrisiko im eigenen
Land aufbewahrt werden.

Die AfD fordert die Deutsche Bundesbank auf, dasim
Ausland gelagerte Staatsgold vollständig und umgehend
nach Deutschland zu überführen. Zudem fordert die AfD
die Herbeiführung eines verfassungsrechtlich
abzusichernden bilanziellen Sonderstatus der
Goldreserve zur Abwehr der Ausbuchungsgefahr
aufgrund der sich seit 2007 dramatisch
verschlechternden Qualität der Bundesbank-Bilanz.

Langfristig halten wir eine grundsätzliche Reform des
Geldsystems für notwendig, hin zu einem freien
Währungswettbewerb oder einem goldgedeckten
Währungssystem.

59

KAPITEL

Außen- und
Verteidigungs-
politik

62

Die Vergemeinschaf-
tung der europäischen

Aufsen- und Sicherheits-

politik (GASP) und den
europäischen
Auswärtigen Dienst
lehnt die AfD ab.

Außen- und Verteidigungspolitik

Außen- und Verteidigungspolitik

Die Außenpolitik Deutschlands muss sich an
deutschen Interessen ausrichten

Deutschland ist als eine der bedeutenden Wirtschafts-
und Kulturnationen der Welt daran interessiert, zu allen
Staaten gute Beziehungen zu pflegen und das friedliche
Zusammenleben der Völker zu fördern.

Die AfD bekennt sich zu den Werten der Charta der
Vereinten Nationen und des Völkerrechts. Insbesondere
das Selbstbestimmungsrecht der Völker darf nicht durch
die Agenden zwischenstaatlicher Organisationen, von
NGOs und durch den Machtzuwachs großer, global
agierender Konzerne ausgehöhlt werden.

Die internationalen Beziehungen befinden sich im
Umbruch hin zu einer multipolaren Weltordnung.
Sicherheitspolitisch ist dabei langfristig die Erlangung
strategischer Autonomie für Deutschland und seine
europäischen Partner anzustreben. In diesem Rahmen
muss Deutschland seine Rolle in Europa
verantwortungsbewusst wahrnehmen.

Die Vergemeinschaftung der europäischen Aufsen- und
Sicherheitspolitik (GASP) und den europäischen
Auswärtigen Dienst lehnt die AfD ab.

Eine stabile europäische Friedensordnung bedarf einer
ausgewogenen Zusammenarbeit sowohl mit den USA als
auch mit Russland. Dabei sind die legitimen Sicherheits-
interessen aller europäischen Staaten zu berücksichtigen.

Außen- und Verteidigungspolitik 63

Auch die Vereinten Nationen sind so zu reformieren,
dass den veränderten Gewichtungen in der Welt Rech-
nung getragen wird. Wir streben einen ständigen Sitz
Deutschlands im Sicherheitsrat sowie die Abschaffung
der gegen Deutschland gerichteten Feindstaaten-
klausel in der Charta der Vereinten Nationen an und
wollen den UN-Migrationspakt (Global Compact for
Migration) aufkündigen.

Die AfD fordert eine strikte Einhaltung des Nicht-
einmischungsgebots in innere Angelegenheiten von
Staaten, auch durch nichtstaatliche Akteure. Die Rolle
der OSZE bei der Stabilisierung von Krisenregionenin
Europa und seiner Peripherie hat sich dank des
Konsensprinzips bewährt und muss ausgebaut werden.

Wir setzen uns nachdrücklich für die Stärkung und
Erweiterung der Kontrollmechanismen in der nuklearen
und konventionellen Rüstung ein sowie für die Neu-
verhandlung kürzlich ausgesetzter Verträge (INF, Open
Skies).

Der islamistische Terrorismus stellt weiterhin eine
ernste Bedrohung der internationalen Staaten-
gemeinschaft dar. Er muss mit allen zur Verfügung
stehenden legalen Mitteln bekämpft werden.

Die AfD fordert eine
strikte Einhaltung des
Nichteinmischungs-
gebots in innere Angele-
genheiten von Staaten.

64

Eine stabile europäl-
sche Friedensordnung
bedarf einer ausgewo-
genen Zusammenarbeit
sowohl mit den USA als
auch mit Russland.

Außen- und Verteidigungspolitik

Das Verhältnis zu wichtigen Staaten im
Wandel der Geopolitik

Die Mitgliedschaft in der NATO und eine aktive Rolle
Deutschlands in der OSZE sind bis auf Weiteres zentrale
Elemente unserer Sicherheitsstrategie.

Die NATO muss aber wieder ein reines Verteidigungs-
bündnis werden. Die AfD tritt dafür ein, das Einsatz-
gebiet der NATO auf das Gebiet der Bündnisstaaten zu
begrenzen.

Die USA sind derzeit der stärkste Bündnispartner
Deutschlands. Leitbild der Beziehungen zwischen
unseren Ländern muss die Gleichberechtigung beider
Partner sein. Sanktionsmafsnahmen der USA gegen
Deutschland sind nicht hinnehmbar.

Im Einklang mit den US-Forderungen nach einer ge-
rechten Verteilung der Lasten und den europäischen
Bestrebungen nach mehr Mitsprache in der NATO istees
nur folgerichtig und im deutschen Interesse, die
europäische Säule der NATO zu stärken.

Eine Entspannung im Verhältnis zu Russland ist Vor-
aussetzung für einen dauerhaften Frieden in Europa.
Es liegt im deutschen und europäischen Interesse,
Russland in eine sicherheitspolitische Gesamtstruktur
einzubinden.

Wir setzen uns für die Wiederaufnahme regelmäfßsi-
ger Gespräche im Rahmen des NATO-Russland-Rats
ein. Ferner sollen die Erweiterung und Vertiefung der

Außen- und Verteidigungspolitik 65

vertrauens- und sicherheitsbildenden Mafsnahmen, die
Kooperation im Bereich der Rüstungskontrolle und der
Verzicht auf den Ausbau militärischer Infrastruktur in
unmittelbarer Nähe der jeweiligen Interessenbereiche
der Entspannung dienen.

Die AfD tritt für die Aufhebung der EU-Sanktionen und
den Ausbau der Wirtschaftsbeziehungen zu Russland
ein. Die Fertigstellung und Inbetriebnahme der
Gasleitung Nord Stream 2 halten wir für unverzichtbar.

Die Volksrepublik China gewinnt für Deutschland vor
allem als Handelspartner an Bedeutung. Mit dem Projekt 41. Kooperation mit
der „Neuen Seidenstrafse” hat China ein Jahrhundertvor- China darfnur unter Be-
haben gestartet. Um hier mitgestalten zu können, setzt en
sich die AfD für eine offensive Beteiligung Deutschlands _raimess erfolgen. Ein
an dem Projekt ein. Die chinesische Seidenstrafsen- nn u

. t. . -
Strategie von Ost nach West sollte Deutschland durch en eu
eine Initiative von West nach Ost ergänzen. Das gilt verhindert werden.

insbesondere für überregionale Infrastrukturprojekte.

Die zunehmende Einflussnahme Chinas in der Welt ist
eine Herausforderung. Eine Kooperation mit China darf
nur unter Bedingungen der Gleichberechtigung und
Faimess erfolgen. Das setztinsbesondere eine
Angleichung derrechtlichen Rahmenbedingungen für
Handel und Investitionen voraus. Ein weiterer
Ausverkauf deutscher bzw. europäischer Technologie
muss verhindert werden.

Grofsbritannien wird auch nach dem Brexit wirtschaftlich
eng mit Europa verflochten bleiben. Die AfD befürwortet

66

Die AfD tritt für die
Wiedereinsetzung der
Wehrpflicht ein.

Außen- und Verteidigungspolitik

daher die Pflege enger Beziehungen zum Vereinigten
Königreich. Mögliche Blockaden und Strafmafsnahmen
seitens der EU lehnen wir ab.

Die Beziehungen zur Türkei sind schwierig und müssen
neu gestaltet werden. Die Türkei gehört kulturell nicht
zu Europa. Ihre zunehmende Islamisierung gibt Anlass
zur Sorge und zeigt, dass die Türkei sich noch weiter von
Europa und der westlichen Wertegemeinschaft entfernt
hat. Die AfD lehnt den Beitritt der Türkei zur EU daher ab
und fordert das sofortige Ende aller Beitritts-
verhandlungen.

Wir unterstützen hingegen die Bemühungen der
Visegrad-Staaten zur Bewahrung der europäischen
Identität.

Der Westbalkan als wichtiger Teil Europas wird zu-
nehmend zum Interessengebiet fremder Mächte.

Um ihn zu stabilisieren, tritt die AfD für eine
privilegierte Partnerschaft mit den Ländern der
Region ein. Die sicherheitspolitische Zusammenarbeit
mitihnen muss auch in Bezug auf das Migrations-
geschehen vertieft werden. Eine Erweiterung der EU
wird strikt abgelehnt.

Wiederherstellung der Wehrfähigkeit Deutschlands

Die deutschen Streitkräfte befinden sich in einem deso-
laten Zustand. Es fehlt an Personal und einsatzfähiger
Ausrüstung. Damit dem Hauptauftrag der Landes- und
Bündnisverteidigung wieder Rechnung getragen
werden kann, muss unsere Bundeswehr nicht nur finan-
ziell gut ausgestattet sein, ihr muss die Eigenständigkeit
insbesondere bei Material und Personal zurückgegeben

Außen- und Verteidigungspolitik 67

werden. Privatisierungen und Zentralisierungen
gehören zurückgedreht und dürfen nur noch dort
Anwendung finden, wo sie nachweisbar notwendig sind
und die Einsatzfähigkeit der Truppe erhöhen. Die
Verantwortung muss wieder in die Hände der Inspek-
teure und Kommandeure, die für die Auftragserfüllung
verantwortlich sind, gegeben werden.

Die AfD lehnt nukleare Erstschlagvorbehalte ab und setzt
sich für die globale Abschaffung von ABC-Waften ein.
Ziel muss der Abzug aller Atomwaffen aus Deutschland,
aber auch der auf Deutschland gerichteten atomaren
Kurzstreckenwaffen sein. Damit würde die nukleare
Teilhabe der Bundesrepublik obsolet.

Die AfD fordert außerdem den Erhalteiner autonomen nr AfD fordert den

und leistungsfähigen wehrtechnischen Industrie in Erhalt einer autonomen
. und leistungsfähigen
Deutschland. Insbesondere den neuen hybriden Bedro- ensechnischen

hungen und den Cyberattacken staatlicher und nicht- Industrie in
staatlicher Akteure muss durch eine massive Stärkung
der Bundeswehr auf diesem Gebiet Rechnung getragen

werden.

Die Bundeswehr soll wieder einen starken Korpsgeist,
ihre Traditionen und deutsche Werte pflegen. Die Tugen-
den des Soldaten sind Ehre, Treue, Kameradschaft und
Tapferkeit. Die Bundeswehr muss die besten Traditio-
nen der deutschen Militärgeschichte leben. Sie helfen,
soldatische Haltung und Tugenden - auch in der Öffent-
lichkeit - zu manifestieren. Militärisches Liedgut und
Brauchtum sind Teil davon.

Über mehr als fünf Jahrzehnte hat die Wehrpflichtarmee
der Bundesrepublik Deutschland gute Dienste geleistet
und für eine Verankerung der Streitkräfte in der
Gesellschaft gesorgt.

68 Außen- und Verteidigungspolitik

Die AfD tritt daher für die Wiedereinsetzung der Wehr-
pflicht ein. Die Wehrpflicht soll um ein Gemeinschafts-
dienstjahr ergänzt werden. Dieses soll Anwendung
finden für Frauen sowie für Männer, die sich nicht für
den Wehrdienst entscheiden. Der Gemeinschaftsdienst
sollin den Bereichen Pflege, Feuerwehr oder
Technisches Hilfswerk geleistet werden.

Um Loyalitätskontlikte zu vermeiden, soll der Dienstin
den Streitkräften ausschließlich Deutschen vorbehalten
sein, die über keine andere Staatsangehörigkeit
verfügen.

Entwicklungspolitik: Stabilität als
Sicherheits- und Wirtschaftsinteresse

Die AfD fordert einen grundsätzlichen Strategie-
wechsel in der Entwicklungspolitik, welcher auch
deutschen Sicherheits- und Wirtschaftsinteressen
Rechnung trägt.

Die AfD fordert einen Entwicklungspolitik wird vom Prinzip der Eigen-

grundsätzlichen verantwortung, der Achtung des Rechts auf staatliche
Strategiewechsel in der Selbstb . dd R kt d
Entwicklungspolitik elbstbestimmung und dem Respekt vor der

welcher auch deut- kulturellen Identität aller Partner getragen. Hilfe soll
sehen sleherneiis und Hilfe zur Selbsthilfe sein - und keine Unterstützung
Wirtschaftsinteressen

Rechnung trägt. korrupter Systeme.

Deutschland ist mit über 21Milliarden Euro das zweit-
wichtigste Geberland für Entwicklungshilfe weltweit.
Dabei sollte die deutsche und europäische Entwick-

Außen- und Verteidigungspolitik

lungspolitik einen realistischen Blick auf die
Bevölkerungsexplosion in Afrika wahren

(von 1,2 Milliarden Einwohnern in 2015 auf 2,4 Milliarden
in 2050), die auch durch eine optimierte Entwicklungs-
politik von aufsen kaum beeinflusst werden kann.

Der Erfolg der deutschen Entwicklungszusammenarbeit
wird durch das strikte Prinzip der Nachfrageorientierung
seitens der Partnerländer gesichert. Priorität besitzen
langfristige Kooperationsvorhaben. Eine signifikan-

te finanzielle Selbstbeteiligung der Partnerländer von
mindestens 51% des gesamten Finanzierungsvolumens
eines Projektes muss obligatorisch sein. Dieses muss
messbare Ergebnisse liefern und sollte einer regel-
mäfsigen unabhängigen Evaluierung unterzogen
werden.

Aufgrund der begrenzten Mittel soll vielmehr eine auch
an deutschen Interessen orientierte Entwicklungspolitik
auf ausgewählte Länder konzentriert werden. Wichtige
Kriterien sind dabei (a) ein niedriger Korruptionsindex,
(b) der pflegliche Umgang mit früheren Entwicklungs-
projekten, (c) Eigenanstrengungen im Bildungsbereich,
(d) die Offenlegung der Staatseinkünfte, insbesondere
aus Bodenschätzen und (e) der bisherige Beitrag des
Empfängerlandes zur regionalen Stabilität. Aufserdem
ist die Gewährung von Entwicklungshilfe an die Bereit-
schaft zur Rücknahme ausreisepflichtiger Migranten zu
knüpfen.

Um dieses Konzept umsetzen zu können, will die AfD im
Rahmen der EU-Reform die Entwicklungspolitik wieder
ganz auf die nationale Ebene der Mitgliedstaaten

69

Um dieses Konzept
umsetzen zu können,
will die AfD im Rahmen
der EU-Reform die
Entwicklungspolitik
wieder ganz auf die
nationale Ebene

der Mitglied-

staaten verlegen

70

Entscheidungen aus-
ländischer nationaler
Gerichte mit
extraterritorialer
Wirkung betrachten

wir als nichtig, einseitig
verhängte Sanktionen
gegen Wirtschafts-
projekte unseres Landes
als völkerrechtswidrig.

Außen- und Verteidigungspolitik

verlegen.
Deutschland muss seine wirtschaftlichen
Stärken ausbauen

Die Weltwirtschaft befindet sich heute im Spannungsfeld
zwischen einem weiteren Ausbau der globalen Verflech-
tungen und ihrer maßvollen Rückführung in dezentrale,
regional fokussierte Wertschöpfungsstrukturen.

Deutsche Aufsenhandelspolitik muss dabei einen
diskriminierungsfreien Zugang zu ausländischen
Import- und Exportmärkten für deutsche Unternehmen
gewährleisten. Das schliefst den Zugang zu Rohstoffen
und die Freiheit der Handelswege mit ein.

Ihre Kernaufgabe ist es daher, verlässliche Handels-
regeln zu erhalten und gemäfs den Interessen unseres
Landes weiterzuentwickeln. Ideologisch motivierte
Alleingänge, wie das Lieferkettengesetz, lehnen wir ab.

Wir fordern aufserdem eine stärkere Kontrolle und
Berücksichtigung gesamtwirtschaftlicher und
verteidigungspolitischer deutscher Interessen bei der
Übernahme deutscher Unternehmen durch
ausländische Konzerne und Finanzinvestoren.
Sicherheitsrelevante und grundversorgungsrelevante
Bereiche müssen vor Übernahmen geschützt werden.

Entscheidungen ausländischer nationaler Gerichte mit
extraterritorialer Wirkung betrachten wir als nichtig, ein-
seitig verhängte Sanktionen gegen Wirtschaftsprojekte
unseres Landes als völkerrechtswidrig.

Außen- und Verteidigungspolitik

Auswärtige Kultur- und Bildungspolitik Deutschlands

Die AfD will deutsche Kultur und Sprache im Ausland

in deutlich stärkerem Mafse als bisher fördern. Die
deutschen Auslandssender, die Goethe-Institute und
verschiedenen Kulturprojekte sollen ein positives Bild
Deutschlands in der Welt vermitteln. Dabei können
sowohl aktuelle Entwicklungen in den Wissenschaften
und Künsten als auch unser reiches kulturelles Erbe für
unser Land werben. Angesichts zunehmender Mobilität
im Arbeitsleben ist hierbei auch an die Deutschen im
Ausland und deren Kinder zu denken, diein ihrer
Muttersprache informiert werden wollen. Der Auftrag
der Deutschen Welle und der Goethe-Institute, als Mitt-

71

ler deutscher Sprache und Kultur zu fungieren, Die AfD bekennt sich

darf nicht verwässert werden.

ausdrücklich

zur Fürsorgepflicht
Deutschlands gegen-

Die Auswärtige Kulturpolitik Deutschlands muss über deutschen Minder-

außerdem das Ziel verfolgen, zu anderen Völkern und

heiten und wird deren
Interessen im Rahmen

Staaten Brücken zu bauen. Wirtschaft, Politik, Kultur ihrer Politik nach-

und nicht zuletzt der Tourismus profitieren davon.

Ein besseres Verständnis für unsere kulturellen Werte,
Traditionen und Geschichte kann Vertrauen schaffen
und Deutschland als zuverlässigen Partner empfehlen.

Der Schutz und Ausbau der Sprachkenntnisse in
Gebieten mit deutschen Minderheiten erfordern
besondere Beachtung und Förderung. Die AfD
bekennt sich ausdrücklich zur Fürsorgepflicht
Deutschlands gegenüber deutschen Minderheiten
und wird deren Interessen im Rahmen ihrer Politik
nachdrücklich unterstützen.

drücklich unterstützen.

72

Deutsch muss neben
Englisch und Franzö-
sisch in den europä-
ischen Institutionen
als Verfahrenssprache
gleichgestellt werden.

Außen- und Verteidigungspolitik

In den europäischen Institutionen muss die deutsche
Sprache gemäfßs ihrer rechtlichen Stellung und ihrer
Verbreitung auch tatsächlich den Verfahrenssprachen
Englisch und Französisch gleichgestellt werden.
Durch den Austritt Großbritanniens aus der EU hat
diese Forderung noch mehr Nachdruck erhalten.

73

Innere
Sicherheit

76

Wir fordern die
Wiedereinführung einer
verbindlichen Extremis-
musklausel.

Innere Sicherheit

Innere Sicherheit

Der Terroranschlag auf den Weihnachtsmarkt an der
Berliner Gedächtniskirche war der vorläufige Tiefpunkt
einer ganzen Serie von Ereignissen, die zeigen, dass es
um unsere Sicherheit schlecht bestellt ist. Die
Gewaltexzesse der linksextremen „Antifa und migran-
tischer Jugendbanden in den Strafsen unserer Städte
machen deutlich, wie sehr die innere Sicherheit Schaden
genommen hat. Zunehmende Straßengewalt, Terror und
Kriminalität sind jedoch keine Naturereignisse, dieman
kritiklos hinnehmen muss. Sie sind menschengemacht
und haben Ursachen, die man bekämpfen kann. Genau
das wollen wir tun.

Linke Gewalt entschieden bekämpfen

Der Linksextremismus überschreitet zunehmend die
Schwelle zum Linksterrorismus. Wir fordern daher
verstärkte Anstrengungen im Kampf gegen den
Linksextremismus.

Die Gewährung staatlicher Fördergelder muss wieder an
eine Verpflichtungserklärung gekoppelt werden, keine
extremistischen Strukturen zu unterstützen. Deshalb
fordern wir die Wiedereinführung einer verbindlichen
Extremismusklausel. Darüber hinaus fordern wir das
Verbot der linksextremistischen Plattform „Indymedia.

Gewalttätige und -unterstützende linksextremistische
und autonome Gruppen, die der sogenannten Antifa
zuzuordnen sind, sind als terroristische Vereinigung
einzustufen. Die von der Bundesregierung beabsichtigte

Innere Sicherheit 77

Privilegierung von „Antifa-Recherchen” muss verhindert
oder ggf. umgehend abgeschafft werden.

Wirksame Bekämpfung der Ausländerkriminalität

Der erhebliche Anteil von Ausländern insbesondere

bei der Gewalt- und Drogenkriminalität, aber auch bei

öffentlichen Unruhen, führt derzeit viel zu selten zu

ausländerrechtlichen Mafsnahmen. Ausländische

Kriminelle können sich sehr häufig auf Abschiebungs-

hindernisse berufen und sind auf diese Weise von

Abschiebung verschont. Wir fordern daher (a) die

Erleichterung der Ausweisung, insbesondere die

Wiedereinführung der zwingenden Ausweisung auch

schon bei geringfügiger Kriminalität, (b) die Ausweisung __

bereits durch die Strafgerichte, (c) die Ermöglichung Die AfD fordert die
. . oo. . Erleichterung der

der Unterbringung nicht abgeschobener Krimineller im

Ausweisung,

Ausland, (d) die Schaffung der Möglichkeit, Gefährder, insbesondere die

insbesondere Terroristen, so lange in Haft zunehmen, a

wie sie im Inland sind und von ihnen eine erhebliche der zwingenden

Gefahr für die Allgemeinheit ausgeht. Ausweisung auch
schon bei

geringfügiger
Die Einbürgerung Krimineller ist zuverlässig zu verhin- Kriminalität.

dern durch die Ablehnung des Erwerbs der deutschen
Staatsangehörigkeit durch Geburt in Deutschland, weil
hierdurch u. a. Angehörige krimineller Clans automa-
tisch zu deutschen Staatsbürgern werden können, die
Abschaffung des einklagbaren Anspruchs auf Einbürge-
rung und die Verschärfung der Ausschlussgründe

für Einbürgerung bei Kriminalität.

Bei schwerer Kriminalität innerhalb von zehn Jahren
nach erfolgter Einbürgerung (Mitwirkung in Terror-

78

Wir wollen die Bundes-
polizei personell und
strukturell auf einen
wirksamen Grenzschutz
unseres Landes
ausrichten.

Innere Sicherheit

organisationen, Zugehörigkeit zu kriminellen Clans) ist
die Einbürgerung zurückzunehmen. Hierzu ist Art. 16
Abs.1GG entsprechend anzupassen.

Vollstreckung im Ausland

Zur Entlastung der Justizvollzugsanstalten sind für aus-
ländische Straftäter Vereinbarungen mit möglichst
heimatnahen Staaten zur Einrichtung von Vollzugs-
anstalten zu treffen, die den Anforderungen der
europäischen Menschenrechtskonvention entsprechen.
Diese sollen unter deutscher Leitung und der Anwen-
dung deutschen Rechts stehen.

Jugendstrafrecht

Die Sicherheitslage verschärft sich vor allem in
Ballungsgebieten dramatisch. Eine besondere Rolle
hierbei spielen gerade junge Täter, denen derzeitein
geradezu zahnloses Recht gegenübersteht. Erzieherische
Erfolge in diesem Segment lassen sich erfahrungs-
gemäfsnur durch sofortige Inhaftierung der Täter
schwerer Delikte erreichen. Wir fordern daher eine
entsprechende Änderung der einschlägigen Gesetze,
insbesondere des Haftrechts. Wegen der immer früher
einsetzenden kriminellen Entwicklung muss das
Strafmündigkeitsalter auf zwölf Jahre abgesenkt und mit
dem Erreichen der Volljährigkeit auch das
Erwachsenenstrafrecht Anwendung finden.

Bundespolizei stärken -
Grenzschutz ist nationale Aufgabe

Die Sicherheit einer Nation basiert auf sicheren Gren-
zen. In Zeiten ungeregelter Migration und grenzüber-

Innere Sicherheit

schreitender Kriminalität und Terrorismus kommt dem
Schutz der europäischen Binnengrenzen eine
wachsende Bedeutung zu. Die Bundespolizei muss
rechtlich, personell und technisch in die Lage versetzt
werden, diese Aufgabe zu erfüllen. Wir wollen daher den
Zuständigkeitsbereich der Bundespolizei zur
Bekämpfung illegaler Migration und grenzüberschrei-
tender Kriminalität auf 50 km an den Landesgrenzen
und im Seebereich auf 80 km ausweiten. Zudem
fordern wir für die Bundespolizei den Einsatz
modernster Fahndungstechnik wie den Einsatz von
Gesichtserkennungssoftware. Fahndungserfolge bei
Grenzkontrollen anlässlich internationaler Gipfeltreffen
belegen das Erfordernis eines effizienten Grenzschutzes.
Wir wollen die Bundespolizei personell und strukturell
aufeinen wirksamen Grenzschutz unseres Landes
ausrichten.

Reform der Polizei

Die Polizeibeamten in Bund und Ländern gewährleisten
die öffentliche Sicherheit unter großem persönlichen
Einsatz. Dabei werden sie oft vom Staat im Stich
gelassen: Personalmangel, strukturelle Unzulänglich-
keiten, unzureichende Ausrüstung und Bewaffnung bei
schlechter Bezahlung und miserabler sozialer
Absicherung. Diese Zustände sind empörend.

Damit muss endlich Schluss sein.

Die AfD fordert für Polizei, Soldaten und Rettungsdienste
eine bundeseinheitliche Besoldung und angemessene
Vergütungen mit Gefährdungszulagen sowie weitere
Zulagen für Überstunden und Sondereinsätze. Ferner
fordern wir angemessene Bewaffnung und Ausrüstung,
u. a. Körperkameras.

79

Die AfD fordert für
Polizei, Soldaten und
Rettungsdienste eine
bundeseinheitliche
Besoldung und
angemessene
Vergütungen mit
Gefährdungszulagen.

80

Die Organisierte
Kriminalität (OK) wollen
wir entschieden und
nachhaltig bekämpfen.
Fahndungs-
möglichkeiten sind zu
verbessern und durch
kriminelle Aktivitäten
erworbene Vermögen
müssen eingezogen
werden.

Innere Sicherheit

Der Dienstherr muss die Dienstunfähigkeits- und
Diensthaftpflichtversicherung übernehmen sowie für
eine Anpassung der Hinterbliebenenversorgung in Höhe
der Pensionsansprüche des letzten Amtes bei Tod durch
qualifizierten Dienstunfall sorgen.

Fahndungsmöglichkeiten verbessern

Zur Verbesserung der Fahndungsmöglichkeiten

sollen die Polizeibehörden an kriminalitäts-
neuralgischen Plätzen und Gebäuden eine Video-
überwachung mit Gesichtserkennungssoftware
einsetzen Können. Bei der Fahndung nach unbekannten
Tätern soll es erlaubt werden, vorhandenes
DNS-Spurenmaterial auch auf biogeografische
Merkmale der gesuchten Person untersuchen zu lassen,
um zielgerichtete Fahndungsmafsnahmen zu ermögli-
chen. Durch zwischenstaatliche Vereinbarungen soll
erreicht werden, dass auch der Polizeibeamte im Einsatz
in die Lage versetzt wird, festzustellen, ob und ggf.
welche Ermittlungs- und Strafverfahren gegen eine
kontrollierte Person im Ausland anhängig waren und
sind.

Organisierte Kriminalität bekämpfen

Die Organisierte Kriminalität (OK) wollen wir entschie-
den und nachhaltig bekämpfen. Durch kriminelle Aktivi-
täten erworbene Vermögen müssen eingezogen werden.
Das geschieht bisher noch nicht in zufriedenstellender
Weise. Die schon bestehenden rechtlichen Instrumen-
te dafür müssen besser genutzt und weiterentwickelt
werden.

Die Mehrzahl der Täter im Bereich der Organisierten
Kriminalität sind Ausländer. Ihre Ausweisung muss

Innere Sicherheit

vereinfacht werden. Deshalb ist für diesen Personenkreis
die OK-Zugehörigkeit als Ausweisungsgrund
einzuführen.

Sicherheit der Bürger verbessern

Auf Vorschlag der EU-Kommission hat das EU-Parla-
mentin Reaktion auf die Terroranschläge von Paris im
November 2015 die EU-Feuerwaffenrichtlinie

geändert. Damit Kommt es zu einer weiteren Verschär-
fung des Waffenrechts. Betroffen von der Verschärfung
des ohnehin schon restriktiven Waffenrechts sind vor
allem Sportschützen, Jäger und Waffensammler. Illegale,
für terroristische Anschläge benutzte Waffen werden
davon nicht erfasst. Die Umsetzung der EU-
Feuerwaffenrichtlinie lehnen wir weiterhin ab.

Statt Legalwaffenbesitzer zu kriminalisieren, ist dem
illegalen Waffenbesitz und -handel ein Riegel vorzu-
schieben.

Strafjustiz verbessern

Die AfD setzt sich für eine schnelle Verbesserung der
Justizorganisation, für eine Stärkung unseres Rechts-
systems und für einen effektiven Rechtsschutz der
Bürger ein.

Im Einzelnen wollen wir die Verfahren beschleunigen
und dafür (a) dienoch aus dem 19. Jahrhundert
stammende Prozessordnung modernisieren, (b) mehr
Personal für das Justizwesen einstellen, (c) Schwer-
kriminalität durch Erleichterung der Inhaftnahme
wirkungsvoller bekämpfen, (d) weniger Strafen zur
Bewährung aussetzen und (e) das Mindeststrafmaß bei
„Messerdelikten' erhöhen.

8

Die AfD setzt sich für
eine schnelle Verbesse-
rung der Justizorganisa-
tion, für eine Stärkung
unseres Rechtssystems
und für einen effektiven
Rechtsschutz der Bürger
ein.

84 Islam

Islam

Die AfD steht uneingeschränkt zur Glaubens-, Gewis-
sens- und Bekenntnisfreiheit nach Art. 4 Grundgesetz.

Muslime, die sich integrieren und unsere Grundordnung
und die Grundrechte anerkennen, sind geschätzte
Mitglieder unserer Gesellschaft.

— In unserem säkularen Staat steht staatliches Recht über
Die AfD steht religiösen Geboten. Soweit der Islam einen Herrschafts-
uneingeschränkt zur
Glaubens-, Gewissens-  anspruch als alleingültige Religion erhebt, die Rechts-
und Bekenninispreihel!  grundsätze der Scharia auch in Deutschland durch-
nach Art. 4 Grundgesetz. . .
setzen will und unsere Rechtsordnung nicht voll
anerkennt oder sogar bekämpft, ister mit der
freiheitlich-demokratischen Grundordnung Deutsch-
lands nicht vereinbar und gefährdet den inneren

Frieden.

Eine Kritik des Islams ist wie jede andere Religionskritik
legitimiert durch das Grundrecht auf freie Meinungs-
außerung. Es ist inakzeptabel, die Kritik am Islam durch
den Vorwurf der „Islıamophobie oder des „Rassismus’ zu
unterdrücken.

Jüdisches Leben wird in Deutschland nicht nur von
Rechtsextremisten, sondern zunehmend auch von
juden- und israelfeindlichen Muslimen bedroht.
Angriffe auf Juden sowie antisemitische Beleidigungen
müssen konsequent strafrechtlich geahndet werden.
Die Al-Quds-Tage in Berlin, wo Demonstranten die
Zerstörung Israels fordern, sind dauerhaft zu verbieten.

Islam

Die vom Verfassungsschutz als sehr hoch
eingeschätzte Terrorgefahr durch Dschihadisten ist eine
ständige Bedrohung unseres Lebens und Friedens.

Die Bildung von „Parallelgesellschaften‘, in denen
muslimische „Friedensrichter die Rechtsvorschriften
der Scharia anwenden und das staatliche Monopol der
Strafverfolgung und Rechtsprechung unterlaufen,
werden wir unterbinden. Rechtsgrundsätze der Scharia
dürfen in keiner Weise Anwendung in der deutschen
Rechtsprechung finden. Zudem muss der Macht-
anspruch grofser Familienclans durch konsequenten
Einsatz der Ermittlungsbehörden und des Strafrechts
gebrochen werden. Kriminelle Clan-Mitglieder sind
abzuschieben.

Die Finanzierung des Baus und Betriebs von Moscheen
in Deutschland durch islamische Staaten wollen wir ge-
setzlich verbieten. Die Kooperation deutscher Behörden
mit der türkisch-islamischen Anstalt für Religion (DITIB),
die Teil der türkischen Religionsbehörde ist, will die AfD
beenden. Die Verleihung des Status als Körperschaft
öffentlichen Rechts an islamische Organisationen
lehnen wir ab.

Islamische Vereine, die sich gegen das Grundgesetz und
die Völkerverständigung richten, wie z.B. die Muslim-
bruderschaft, müssen gemäß Art. 9 Abs. 2 Grundgesetz
verboten werden. In Deutschland predigende Imame
sollen sich zu unserer Verfassung bekennen und
möglichst in deutscher Sprache predigen.

85

Eine Kritik des Islams
ist wie jede andere
Religionskritik
legitimiert durch das
Grundrecht auf freie
Meinungsäufserung.

86

Der Unterdrückung
muslimischer Frauen
stellt sich die AfD
entgegen und fordert
in allen Bereichen die
Gleichberechtigung von
Mann und Frau.

Islam

Sie müssen bei der Zulassung ein Zertifikat B2 für die
deutsche Sprache des Gemeinsamen Europäischen
Referenzrahmens für Sprachen vorweisen können.

Die islamtheologischen Lehrstühle an deutschen Uni-
versitäten sind abzuschaffen und die Stellen der
bekenntnisneutralen Islamwissenschaft zu übertragen.
Im konfessionsgebundenen Religionsunterricht an
staatlichen Schulen soll kein Islamunterricht stattfinden,
vielmehr eine sachliche Islamkunde im Ethikunterricht.

Minarett und Muezzinruf sind mit einem toleranten
Nebeneinander der Religionen, wie es die christlichen
Kirchen praktizieren, nicht vereinbar.

Der Unterdrückung muslimischer Frauen stellt sich die
AfD entgegen und fordert in allen Bereichen die
Gleichberechtigung von Mann und Frau.

Das Tragen von Burka und Nigab in der Öffentlichkeit
wollen wir auch in Deutschland untersagen, ähnlich wie
in Frankreich, Österreich, Dänemark, den Niederlanden
und der Schweiz.

Das Kopftuch als religiös-politisches Zeichen soll im
Öffentlichen Dienst generell nicht gestattet und in
öffentlichen Schulen weder von Lehrerinnen noch
Schülerinnen getragen werden.

Islam

Um Polygamie und Zwangsheiraten von Muslimen zu
unterbinden, fordert die AfD, das Verbot der religiösen
Trauung ohne vorherige standesamtliche Eheschlie-
ung wieder in Kraft zu setzen. Den Jobcentern sind die
Personenstandsregister der Standesämter zugänglich
zu machen, um missbräuchliche Inanspruchnahme von
Hartz IV durch Zweit- oder Drittfrauen zu verhindern.
Eheverträge zu güter- und unterhaltsrechtlichen
Angelegenheiten sollen nur nach deutschem Recht
geschlossen werden dürfen.

87

Migration, tz!
und Integration

90

Die Hilfe für Menschen
in Not muss von der
Aufnahme in
Deutschland entkoppelt
und in den Krisen-
regionen selbst geleistet
werden.

Migration, Asyl und Integration

Migration, Asyl und Integration:
Human und rational handeln -
Deutschland schützen

Die Fluchtbewegungen des 21. Jahrhunderts können
nicht mittels Aufnahme der Migranten in Europa und
Deutschland bewältigt werden

Noch nie gab es global mehr Flüchtlinge alsin den
letzten Jahren. Neben kriegerischen Auseinanderset-
zungen und staatlichen Verfolgungshandlungen spielt
insbesondere das enorme Bevölkerungswachstum auf
dem afrikanischen Kontinent eine überragende Rolle.
Da dies mit Armut und dem Kampf ums tägliche Über-
leben verbunden ist, wächst der Migrationsdruck auf
Europa in besonderer Weise. Entgegen der Intention des
UN-Migrationspakts kann dieses globale Problem nicht
durch Migration gelöst werden. Selbst kleinere Teile
dieser Entwicklung, die sich in Millionen von Menschen
ausdrücken lassen, Können weder ökonomisch, noch
sozial- und integrationspolitisch in Europa aufgefangen
werden. Deutschland und die anderen europäischen
Länder würden destabilisiert, ohne dass ein wirkungs-
voller Beitrag zur Lösung des globalen Problems geleistet
wird.

Asylmissbrauch beenden

Das derzeitige Asylsystem ist dysfunktional und gleicht
einer Lotterie. Finanzielle Fehlanreize und unterlassene
Abschiebungen führen zu illegaler Migration, Schleuser-
kriminalität und zahlreichen Todesopfern im Mittelmeer.
Die meisten dieser Migranten begeben sich in der

Migration, AsylundIntegration 91

Hoffnung aufein besseres Leben auf den Weg nach
Europa. Die Hilfe für Menschen in Not muss jedoch von
der Aufnahme in Deutschland entkoppelt und in den
Krisenregionen selbst geleistet werden, wo dies weit
effektiver ist und viel mehr wirklich Hilfsbedürftige
erreicht.

Die unlösbaren Folgeprobleme

Die desaströsen Folgen der unregulierten Massenzu-

wanderung seit 2015 sind unübersehbar und verschär-

fen sich weiter: überproportionale Zuwandererkrimi-

nalität, Terroranschläge und islamischer Separatismus

belegen dies ebenso wie dreistellige Milliardenkosten,
Wohnraummangelund die hohe Arbeitslosigkeitunterr

den Zuwanderern. Erfolgreiche Integration kann und Eine existentielle Frage

wird so nicht gelingen. wie die Zuwanderung
muss in demokratischer

Selbstbestimmung
Unsere Lösung: Ein Schutzsystem des 21. Jahrhunderts in «u nationaler Ebene

. entschieden werden.
Anlehnung an das australische Modell.

Eine existentielle Frage wie die Zuwanderung mussin
demokratischer Selbstbestimmung auf nationaler Ebene
entschieden werden. Unser Modell für das 21. Jahrhun-
dert hatneben der Ablösung der überholten internatio-
nalen Regelwerke, wie der Genfer Flüchtlingskonvention
von 1951, folgende Kernpunkte:

» Fokus aufhumanitäre Hilfe in Krisenregionen selbst
(intrakontinentale Fluchtalternativen)

» Zurückweisung von Asylsuchenden bei Einreise an
der Grenze gemäfs Art. 16a Abs. 2S.1GG

» Ausschliefslich qualifizierte Einwanderung nach
japanischem Vorbild

92 Migration, Asyl und Integration

» Humanitäre Aufnahme nur für vom Bundestag
ausgewählte, besonders schutzbedürftige Personen,
für deren Auswahl ein mit der deutschen Werte- und
Gesellschaftsordnung vereinbarer kultureller und
religiöser Hintergrund ein wichtiges Kriterium ist.

Solange ein dauerhafter und wirksamer Schutz der
EU-Außengrenzen fehlt und die untaugliche über-
staatliche Rechtslage fortbesteht, muss Deutschland
die Kontrolle der Grenzen und die Unterbindung
illegaler Grenzüberschreitungen selbst in die Hand
nehmen.

Wir fordern daher:

» Einreisen darf künftig nurnoch, wem dies erlaubt
ist. Zurückweisungen an der Grenze müssen wieder
als selbstverständliches Recht souveräner Staaten
aufgefasst werden.

» Die Grenzen müssen zur Abwehr unerlaubter Zuwan-
derung und zur Kriminalitätsbekämpfung mit

Deutschland muss die einermodernen und wirksamen Sicherung ausge-
Kontrolle der Grenzen stattet werden. Dabei muss grenzüberschreitender
und die Unterbindung dl laub dw kehr oh

illegaler Grenzüber- Pendler-, Urlaubs- und Warenverkehr ohne unzu-

schreitungen selbst in mutbare Einschränkungen gewährleistet sein.

ne Hand nehmen » Um die „Grüne Grenze‘ mit derselben Effektivität

überwachen zu können, wird ein umfassendes und
modernes Grenzsicherungskonzept erarbeitet.
Hierzu Können auch physische Barrieren wie z.B.
Grenzzäune zur Sicherung der Staatsgrenzen
gehören.

Migration, AsylundIntegration 93

Asylparadies Deutschland schließen

Seit 2015 trägt Deutschland die Hauptlast der

illegalen Zuwanderung nach Europa, verursacht durch
eine Vielzahl selbstverschuldeter Ursachen. Die AfD hat
einen umfassenden Mafßsnahmenkatalog zur Wieder-
herstellung geordneter Verhältnisse im Asylverfahren
erarbeitet und wird dessen Umsetzung konsequent
einfordern:

1. Keine pauschalen Zuwanderungskontingente und
Umverteilungsquoten, egal ob im nationalstaatlichen,
europäischen oder außereuropäischen Rahmen (z. B.

Resettlement).

2. Keine Aufnahme von Migranten im Rahmen
einer „europäischen Lösung), die durch Miss- Die AfD hat einen
brauch der Seenotrettung nach Europa gelangen. umfassenden Maß-

. .. . . nahmenkatalog zur
Vielmehr muss gemäfs dem internationalen Seerecht yedernersielnung

grundsätzlich eine Ausschiffung der Betroffenen am _ geordneter Verhält-
nächstgelegenen geeigneten Seehafen erfolgen. Die nn verfahren
heute praktizierte Form der Seenotrettung wirkt häu-
fig als tödlich endender Pull-Faktor.
3. Beantragung von Asyl und Eröffnung des Antragsver-
fahrens nur bei nachgewiesener Identität und Staats-
angehörigkeit.
A. Strenge Bestrafung von Falschangaben im
Asylverfahren.
5. Ausweitung der Zahl sicherer Herkunftsstaaten.
6. Bekämpfung des Missbrauchs legal erteilter Visa zum
zweck der Asylantragstellung;
7. Verlust des Schutzstatus nach Reisen von Asylbe-
rechtigten in ihr Herkunftsland. Wer sein „Verfolger-
land besucht hat, ist beim Versuch der Wiederein-
reise an der deutschen Grenze zurückzuweisen.

94

Die AfD fordert die
Wiederherstellung
rechtsstaatlicher
Zustände bei
Abschiebungen.

Migration, Asyl und Integration

10.

11.

12.

13.

14.

15.

Ablehnung jeglichen Familiennachzuges für
Flüchtlinge.

Abschaffung des „Spurwechsels’ abgelehnter und
ausreisepflichtiger Asylbewerber. Keine Belohnung
für illegale Einreise und Verweigerung der Ausreise.
Niederlassungserlaubnis für anerkannte Asyl-
bewerber erst nach 10 Jahren.

Beendigung des Wildwuchses bei der Asylverfahrens
beratung und -betreuung. Die „Beratung und Betreu-
ung erfolgt meist durch nichtstaatliche, freie Träger,
die ein finanzielles Interesse an steigenden
Flüchtlingszahlen und am Verbleib möglichst vieler
Abgelehnter haben. Diese Organisationen zählen
zur „Asyl-Industrie‘, die sich am Asylsystem berei-
chert. Aus Verfahrens- und Rückkehrberatung wird
nahezu immer eine Verweigerungs- und Bleibe-
beratung. Verfahrensberatung darfnurnoch nach

8 12 a Asylgesetz und nur durch staatliche Stellen
erfolgen.

Keine Arbeitserlaubnis mehr für Asylantragsteller
und abgelehnte Asylbewerber.

Jährliche Prüfung des Fortbestands von Asyl-
gründen über zehn Jahre, da Asyl ein Recht auf Zeit ist
und keine Anwartschaft auf dauerhafte
Einwanderung begründet.

Ausnahmslose Abschiebung von Gefährdern und
Straftätern, auch nach Afghanistan, Irak und Syrien.
Rückkehrpflichtige sollen eine handwerklich-prak-
tische Grundausbildung erhalten Können, die sie zum
Wiederaufbau ihrer Heimat und zur dortigen
Existenzgründung befähigt („fit for return‘).

Migration, Asyl und Integration

Abschiebung und Rückkehr

Die unterlassene Aufenthaltsbeendigung abgelehnter
und ausreisepflichtiger Asylbewerber ist das Gegenteil
von Rechtsstaatlichkeit: Sie ist Ausdruck fortgesetzten
Staatsversagens. Die Zahl der Ausreisepflichtigen ist da-
bei kontinuierlich gestiegen und hat im zweiten Quartal
2021 die Zahl von ca. 300.000 Personen erreicht -
während die Zahl freiwilliger Ausreisen und erfolgrei-
cher Abschiebungen kontinuierlich sinkt.

Die Politik hat vor der Aufgabe, diesen Widerspruch
aufzulösen, Kapituliert - getrieben von einer mächtigen
Anti-Abschiebe-Industrie. Es ist zynisch und inhuman,
Menschen mit der faktischen Garantie auf dauerhaften
Verbleib im „reichen’ Deutschland anzulocken und dabei
deren möglichen Tod im Mittelmeer in Kaufzu nehmen.

Freiwillige Rückkehr ist einer Abschiebung aus finanzi-
ellen, organisatorischen und humanitären Gründen im-
mer vorzuziehen, weshalb Rückkehrbereitschaft deutlich
mehr als bisher gefördert werden muss.

Die AfD fordert daher zur Wiederherstellung rechts-
staatlicher Zustände:

1. Eine Abschiebeoffensive zum Abbau des Überhangs
an Ausreisepflichtigen.

2. Eine nationale und eine supranationale „Remigra-
tionsagenda’ als Schutzgewährung in Herkunfts- und
Transitregionen nach dem Grundsatz „Hilfe vor Ort‘

3. Keine Duldungen mehr, sondern Bescheinigungen
über die Ausreisepflicht.

95

Die AfD fordert eine
nationale und eine
supranationale
„Remigrationsagenda'
als Schutzgewährung in
Herkunfts- und Transit-
regionen.

96 Migration, Asyl und Integration

A. Zuständigkeitsübertragung für den Vollzug von
Abschiebungen an die Bundespolizei und deren
entsprechende Ausstattung mit Personal und
Kompetenzen.

5. Verhinderung von Abschiebungen zu Straftatbestand
erklären.

6. Nutzung freier Platzkapazitäten bei Transportflügen
der Bundeswehr, um die Anzahl der Abschiebungen
zu erhöhen.

7. Aufhebung der Kirchenasyl-Vereinbarung zwischen
BAMF und Kirchen. Es ist nicht hinnehmbar, dass die
Kirchen unter Missbrauch des Kirchenasyls die
Fristen für Abschiebungen in EU-Länder aushebeln.

Leben retten und soziale Gerechtigkeit herstellen

Im internationalen Vergleich sind die Leistungen
Deutschlands für Asylbewerber sehr hoch. Anerkannte
erwartet eine lebenslängliche materielle Absicherung.
Die Mittel dafür werden aufgebracht von Steuerzahlern,
die schwer dafür arbeiten müssen, aber keine
Wirtschaftsflüchtlinge eingeladen haben.

Die AfD fordert die

Wahrung sozialer Die Grofßszügigkeit des deutschen Staates verursacht eine
Gerechtigkeit, indem . . . .

nur Sachleistungen als erhebliche Sogwirkung, die zum Tod vieler Menschen
Sozialleistungen für führt. Die Bundesregierung lädt damit schwere

Asylbewerber und Aus-
reisepflichtige erbracht
werden.

moralische Schuld auf sich.

Die Asylanerkennung führt zu einer sofortigen Gleich-
stellung in allen sozialen Sicherungssystemen, ohne jede
vorangegangene Teilnahme am Solidarsystem.

Das ist sozial ungerecht.

Migration, Asylund Integration 97

Die AfD fordert daher zur Senkung von Anreizen und zur
Wahrung der sozialen Gerechtigkeit:

» Sozialleistungen für Asylbewerber und Ausreise
pflichtige sind ausnahmslos als Sachleistungen zu
erbringen.

» Asylanerkennung darf nicht länger den automati-
schen, sofortigen, zeitlich und inhaltlich unbegrenz-
ten Anspruch auf alle Leistungen des Wohlfahrts-
staates bedeuten. Nach österreichischem Vorbild
könnte eine Leistungsgewährung in voller Höhe auch
vom Erreichen guter Sprachkenntnisse abhängig
gemacht werden.

» Eines der Hauptmotive für die gegenwärtige
Massenmigration nach Deutschland und Europa ist
die Möglichkeit von Migranten, in Deutschland
bezogene Transferleistungen und auch legal oder
illegal verdientes Geld zurück in ihre Heimat-
länder zu überweisen. Um diesen Migrationsanreiz
zu beseitigen, befürworten wir die Einführung einer
Steuer auf Rücküberweisungen, die den
Europäischen Wirtschaftsraum (EWR) verlassen.

Hilfe vor Ort statt Tod auf dem Weg nach Europa

Hilfe vor Ort hat für die AfD höchste Priorität. Ihr ist

absoluter Vorrang zu geben vor einer Aufnahme von

Asylbewerbern in Deutschland. Zugleich werden damit

wichtige Fluchtgründe abgebaut. Die AfD will die Zuwan-

derung wieder national

Ein Teil der im Rahmen der Asylpolitik eingesetzten regein

zweistelligen Milliardensummen kann heimatnah eine
vielfache Wirkung entfalten und so helfen, den
Migrationsdruck auch nachhaltig zu senken.

98 Migration, Asyl und Integration

Zuwanderung wieder nationalregeln

Die zuwanderungspolitische Souveränität Deutschlands
ist schon heute nicht mehr gegeben. Hauptgrund hierfür
ist eine weitreichende Abgabe von Hoheitsrechten an die
EU und der Beitritt zuinternationalen Vereinbarungen,
die allesamt eine linksextreme „One-world-Utopie‘
verfolgen.

Die AfD stellt sich dem entgegen. Sie tritt für die Rück-
gabe der Hoheitsrechte zur Rechtssetzung von Asyl
und Zuwanderung ein und schlägt dafür die folgenden
Mafßsnahmen vor:

» Die Vergemeinschaftung des Aufenthalts- und
Asylrechts wird rückgängig gemacht.

Die europäische Grenzsicherung verbleibt dabeiin
europäischer Verantwortung (Frontex) parallel zur
nationalstaatlichen Verantwortung für die eigene
Grenzsicherung.

» Deutschland kündigt unverzüglich den UN-Migrati-
onspakt und den UN-Flüchtlingspakt auf, da diese
trotz ihrer angeblichen Unverbindlichkeit in
nationales Recht eingehen werden.

_ » Derneue, von der Europäischen Kommission ange-

Die AfD bekennt sich BR
ie AfD bekennt sic strebte EU-Migrationspakt soll verhindert werden
dazu, die kulturelle

Identität Deutschlands bzw. der Beitritt Deutschlands unterbleiben.
en Fe » Kein ‚Gemeinsames europäisches Asylsystem‘
Ir [fordern die AUSAT- j
beitung eines (GEAS) mit von der EU festgelegten Aufnahmequoten
umfassenden für Flüchtlinge.

migrationspolitischen
Ansatzes nach
Japanischem Vorbild.

Migration, Asyl und Integration

Qualifizierte Zuwanderung

Das aktuelle Einwanderungssystem schadet
Deutschland kulturell sowie wirtschaftlich und ist
dringend reformbedürftig. Die AfD bekennt sich dazu,
die kulturelle Identität Deutschlands zu wahren.

Vorbild einer identitätswahrenden Migrationspolitik
können für Deutschland nicht klassische Einwande-
rungsländer von der Gröfse Kanadas oder Australiens
sein, sondern eher Länder wie Japan, die eine ihrer
Landesstruktur entsprechende Begrenzung und Steue-
rung der Migration verfolgen.

Durch Lockdown-Krise, Automatisierung und Digita-
lisierung sind die Zeiten rekordhoher Beschäftigung
absehbar vorbei. Massenmigration erzeugt Lohndruck
bis in die Mittelschicht hinein und führt zu Konkurrenz
um Sozialleistungen.

Wir fordern die Ausarbeitung eines umfassenden migra-
tionspolitischen Ansatzes nach japanischem Vorbild mit
dem Primärziel der Interessenwahrung der Deutschen
und bereits zugewanderter Bürger, die sich zur
kulturellen Identität Deutschlands bekennen.

Die Behauptung einiger Wirtschaftsverbände und
Lobbyisten, in Deutschland herrsche Fachkräftemangel
und damit ein Bedarf an qualifizierter Einwanderung,
dient nur vordergründig dem Gemeinwohl und darf
deshalb nicht maßgeblich sein. Der schädliche Einfluss
wirtschaftlicher Partikularinteressen oder gar damit
kaschierter antideutscher Ressentiments muss beendet
werden.

99

Integration gelingt nur,
wenn die Regeln,
Traditionen und Werte
unserer Gesellschaft
akzeptiert werden.
Unverzichtbar ist der
Erwerb der deutschen
Sprache.

100 Migration, Asyl und Integration

Integration

Bei der Integration sehen wir Zuwanderer und Personen,
denen wir einen vorübergehenden Schutz gewähren,
in der Pflicht. Integration gelingt nur, wenn die Regeln,
Traditionen und Werte unserer Gesellschaft akzeptiert
werden. Unverzichtbar ist der Erwerb der deutschen
Sprache.

Die AfD fordert:

» Beiqualifizierten Zuwanderen setzen wir eine in-
tensive Integrationsbereitschaft voraus. Der Fokus
muss auf dem Spracherwerb liegen, den wir bis zum
Niveau B2 unterstützen wollen.

» Die Bevorzugung von Migranten in Form einer Quo-
tierung, unter Missachtung des Leistungsprinzips,
muss ausgeschlossen werden. Abstriche hin-
sichtlich der Anforderungen bei der Ausbildung oder
bei der Einstellung in den Öffentlichen Dienst lehnen
wir ab.

» Integrationsprojekte sind einer staatlichen Evalua-
tions- und Erfolgsprüfung zu unterziehen.

Die AfD fordert Das Profitgeschäft von NGOs und Wohlfahrts-
Integrationsprojekte verbänden ist zu beenden.

einer staatlichen a
Evaluations- und » Die AfD verlangt, dass das Kopftuch als religiös-
Erfolgsprüfung zu politisches Zeichen im Öffentlichen Dienst generell

hen .
nn nicht gestattet und in öffentlichen Schulen weder

von Lehrerinnen noch Schülerinnen getragen wird.
Dies betrachten wir als eine der wichtigsten
Bedingungen für eine gelingende Integration. Das
Kopftuchverbot trägt zur Wahrung des Schulfriedens
bei und ermöglicht jungen Mädchen eine freie
Persönlichkeitsentwicklung und Selbstbestimmung.

Migration, Asylund Integration 101

Die deutsche Staatsangehörigkeit ist
ein kostbares Gut

Das Geburtsortprinzip (Erwerb der deutschen Staats-
angehörigkeit durch Geburt in Deutschland, auch wenn

kein Elternteil Deutscher ist) wollen wir wieder aus dem
Gesetz streichen und zum Abstammungsprinzip zurück-
kehren, wie es bis zum Jahr 2000 galt. Die Anforderun-

gen - besonders auch in zeitlicher Hinsicht - für eine
Ermessenseinbürgerung müssen erheblich steigen und
einen Anspruch auf Einbürgerung darf es nicht mehr

geben. Die Verleihung der deutschen Staats-

angehörigkeit soll als Erfolg eigener Anstrengungerlebt _

werden und die Betroffenen mit Stolz erfüllenkönnen. Pwertdie
Rückkehr zum Abstam-
mungsprinzip für die
Staatsbürgerschaft, wie
es bis zum Jahr 2000
galt.

KAPITEL 11

Familienpolitik
U

104

Die AfD bekennt sich zur
Familie als Keimzelle
unserer Gesellschaft.

Familienpolitik

Familienpolitik

Familien stärken und fördern

Die AfD bekennt sich zur Familie als Keimzelle unserer
Gesellschaft. Sie besteht aus Vater, Mutter und Kindern.
Familie bedeutet Sicherheit, Obhut, Heimat, Liebe und
Glück. Dieses Werte- und Bezugssystem wird von Gene-
ration zu Generation weitergegeben.

Von linksgrüner Seite jedoch wird die Institution Familie
ausideologischer Motivation heraus diskreditiert, um sie
durch andere Leitbilder zu ersetzen.

Wir fordern dagegen die Wiederherstellung des grund-
gesetzlich garantierten, besonderen Schutzes der
Familie.

In der Shell-Jugendstudie 2019 gaben 68% der Jugendli-
chen an, später einmal eine Familie gründen zu wollen,
19% waren noch unentschlossen. In der Vergangenheit
konnten allerdings viele junge Erwachsene ihren
Kinderwunsch nicht wie gewünscht verwirklichen.

Dies unterstreicht die Notwendigkeit einer aktivieren-
den, also geburtenfördernden Familienpolitik. Eine Fort-
setzung der herrschenden, familienzersetzenden Politik
wird die demografische Katastrophe, in die wir geraten
sind, weiter verschlimmern. Am Ende dieses Prozesses
steht auch der Zusammenbruch der sozialen Siche-
rungssysteme und letztlich unserer kulturellen Identität.
Das können wir nicht wollen! Die AfD ist entschlossen,
dem familienfeindlichen Zeitgeist eine positive,
familienfreundliche Sozialpolitik entgegenzustellen.

Familienpolitik 105

Ein Schlüssel zu glücklicheren und stabileren Familien
liegt dabei in aktivierenden familienpolitischen Mafsnah-
men und in einer größeren Wertschätzung der Famili-
enarbeit.

Ursachen der demografischen Katastrophe

Die Geburtenrate liegt in Deutschland seit Mitte der 70er
Jahre weit unterhalb des Niveaus zur Bestandserhaltung.
Bei einer Geburtenrate von 1,5 Kindern schrumpft ein
Volk um ca. 30% pro Generation.

Die Ursachen für den Geburtenrückgang sind vielfältig.

Durch den Ausbau des deutschen Sozialstaates mit

Hilfeleistungen für alle Lebenslagen hat sich eine funda-

mental veränderte Einstellung der Bürgerzumeigenen wir fordern die Wieder-
generativen Verhalten entwickelt. Die Wahrnehmung herstellung des grund-
des Staates als Helfer in allen Lebenslagen a
hat die Vorstellung von der eigenen Familie auch als der Familie.
generationsübergreifende Wirtschafts- und

Versorgungsgemeinschaft ersetzt. Vor diesem Hinter-

grund wurde der Hang zur vorrangigen ökonomischen
Wohlstandsoptimierung während der Lebens-

erwerbsphase zum Leitbild einer materialistischen

Gesellschaft. Immer mehr Paare haben daher den

ideellen Wert einer Familie als Sozialisationsraum und

emotionale intergenerative Lebensgemeinschaft aus

den Augen verloren. Der Gedanke einer vorrangigen

„selbstverwirklichung” hat dazu geführt, zugunsten von

Einkommen und Karriere Kinderwünsche zurück-

zustellen oder gänzlich aus der eigenen Lebensplanung

zu verdrängen.

106

Die AfD will Familien
entlasten, indem ihnen
Rentenbeiträge in Höhe
von 20.000 € pro Kind
freigestellt werden.

Familienpolitik

Eltern, die für Familienarbeit ganz oder teilweise zu
Hause bleiben, werden in unserer Gesellschaft
abgewertet. Insbesondere bei doppelberufstätigen Eltern
besteht durch die strukturelle Benachteiligung von
Familien in der Arbeitswelt eine mangelnde Vereinbar-
keit von Familie und Beruf. Hinzu kommen die kinder-
bezogenen Mehrkosten für Wohnraum, Versorgung und
Ausbildung, die bei der Bemessung von Steuern und
Sozialversicherungen nur minimal ausgeglichen
werden. Das alles führt zu durchschnittlich kleineren
Familien und teilweise auch zu einer gewollten Kinder-
losigkeit.

Leistungsfähigkeit der Familien erhalten

Vor allem Familien der mittleren Einkommensschicht
werden in Deutschland steuerlich stärker belastet als in
fast allen anderen europäischen Ländern. Die Geburten-
rate ist bei diesen Familien erschreckend gering. Die AfD
will hier mit dem steuerlichen Familiensplitting einen
Paradigmenwechsel einleiten, verbunden mit einer
Anhebung des Kinderfreibetrages. Als weitere steuerli-
che Mafsnahmen fordern wir die vollständige steuerliche
Absetzbarkeit von kinderbezogenen Ausgaben und eine
Absenkung der Mehrwertsteuer für Artikel des Kinder-
bedarfs auf den reduzierten Satz.

Ergänzend sollen Eltern zur Herstellung der Lastenge-
rechtigkeit bei der Geburt jedes Kindes eine Rückzah-
lung bereits entrichteter Rentenbeiträge in Höhe von
20.000 € erhalten bzw. von zukünftigen Beiträgen in
entsprechender Höhe freigestellt werden, ohne dass die
spätere Leistung gekürzt wird.

Familienpolitik

Familien unterstützen

Umfragen belegen immer wieder, dass sich viele

junge Eltern in Deutschland statt Mehrfachbelastung
und Rollenkonflikten mehr Zeit für ihre Kinder wün-
schen. Kinder unter drei Jahren fühlen sich am wohlsten,
wenn sie durch die eigenen Eltern betreut werden. Es
muss wieder möglich sein, dass eine Familie mit kleinen
Kindern von einem Gehalt leben kann, so dass die Eltern
frei zwischen Berufstätigkeit oder Erwerbspause
entscheiden können.

Eltern, die ihre Kleinkinder selbst betreuen, erleben
deren Entwicklung unmittelbar, vermitteln ihnen
Herzensbildung und erhalten damit ihre körperliche und
seelische Gesundheit, während gerade bei Kindern unter
drei Jahren eine Fremdbetreuung mit Entwicklungs-
risiken verbunden ist (Großstudie ‚The NICHD Study

of Early Child Care and Youth Development‘, 1991 und
2007). Die AfD will daher für die ersten drei Jahre ein
Betreuungsgeld für Eltern bzw. Grofßseltern einfüren,
welches sich als Lohnersatzleistung am bisherigen
Aurchschnittlichen Nettolohn der letzten drei Jahre vor
Geburt des ersten Kindes orientiert, gedeckelt auf die
Höhe des allgemeinen durchschnittlichen Nettogehalts.

Das Kindergeld soll beibehalten, Missbrauchsmöglich-
keiten müssen aber unterbunden werden. So ist zum
Beispiel für im Ausland lebende Kinder nur der dort
übliche Betrag zu zahlen.

Für junge Familien sollen die Erstanschaffungen bei
Familiengründung durch einen Ehe-Start-Kredit
unterstützt werden.

107

Es muss wieder möglich
sein, dass eine Familie
mit kleinen Kindern
von einem Gehalt leben
kann.

108

Wir wollen eine kinder-
freundliche Gesellschaft
als Staatsziel ins Grund-
gesetz aufnehmen.

Familienpolitik

Der Ehe-Startkredit soll Deutschen im Sinne des Grund-
gesetzes gewährt werden sowie EU-Bürgern, die seit
mindestens 20 Jahren in Deutschland leben. Mit jedem
Kind wird ein Teil des Kredits erlassen. BAföG-Empfän-
gern soll bei einer Geburt während der Ausbildung bzw.
während des Studiums die Rückzahlung bei erfolgrei-
chem Abschluss erlassen werden. Für alle Studenten mit
Kindern soll die Regelstudienzeit auf Antrag um bis zu
sechs Semester verlängert werden.

Der Wiedereinstieg von Eltern nach der Babypause soll
vereinfacht werden, indem nach Kinderzahl gestaffelte
Lohnsubventionen an die Arbeitgeber gezahlt werden.

Familien brauchen bezahlbaren Wohnraum. Neben der
Schaffung günstiger Mietwohnungen wollen wir auch
möglichst viele Familien in eigene vier Wände bringen.
Dazu wollen wir Bauland vergünstigt an Familien ab-
geben und die Konditionen der KfW verbessern: Statt
teurer Förderung von Energieeffizienzhäusern wollen
wir gestaffelte Teilerlasse nach Kinderzahl.

Familienfreundliches Deutschland

Wir wollen eine kinderfreundliche Gesellschaft als
Staatsziel ins Grundgesetz aufnehmen. Das stärkt
Familien und verpflichtet die Politik zu echter Fami-
lienfreundlichkeit - im Gegensatz zur Aufnahme von
„Kinderrechten ins Grundgesetz, die dem Staat nur „die
Lufthoheit über den Kinderbetten’ sichern soll. Kinder
sind schon jetzt Grundrechtsträger. Gesonderte
„Kinderrechte schwächen deshalb nur die Elternrechte
nach Art. 6 GG.

Familienpolitik 109

Statt Gleichstellungsbeauftragten wollen wir Familien-
beauftragte, die familienfreundliche Entscheidungen
sicherstellen und somit die aktivierende Familienpolitik
unterstützen. Staatliche Leistungen und Verwaltungs-
abläufe sind systematisch auf Familienfreundlichkeit zu
prüfen und anzupassen. Über soziales Marketing soll für
Familiengründungen und das Leitbild der 3-Kind-
Familie geworben sowie über die damit verbundenen
Angebote informiert werden.

Mehrgenerationenfamilien sind stärker zu fördern. Für
Arbeitgeber müssen Anreize für eine bessere Vereinbar-
keit von Familie und Beruf geschaffen werden.

Kinder in den Familien belassen,
Kindesentzug nur in Ausnahmefällen

Einige Familien bedürfen der staatlichen Unterstützung

durch Beratung und Hilfe bei der Erziehung. Allerdings Die AfD fordert Fami-

darf dies nur in äußersten Ausnahmefällen zu einem lien- statt Gleichstel-

Kindesentzug durch das Jugendamt führen. Leiderhat _ngasbeauftragte, um
. . . familienfreundliche

sich in den letzten Jahren um die Inobhutnahme von Entscheidungen sicher-

Kindern eine regelrechte Industrie entwickelt, dieeines zustellen

ständigen Nachschubs an Kindern bedarf. Die Anzahl

der Inobhutnahmen ist seit 1995 von 23.432 Kindern bis

2018 auf 52.590 Kinder bzw. um 124% gestiegen. Bis 2019

haben sich die Ausgaben dafür von ca. 15 Milliarden Euro

(1995) auf 50,6 Milliarden Euro vervielfacht. In Extrem-

fällen musste sogar die „übermäßige Liebe der Mutter‘

als Entzugsgrund herhalten.

110

Die Jugendpolitik muss
auf eine Förderung der
Familien ausgerichtet
werden und das Ziel
verfolgen, dass die
Kinder in ihren Familien
bleiben.

Familienpolitik

Hier fordert die AfD einen radikalen Kurswechsel. Wir
wollen Fehlentscheidungen vermeiden, die als Willkür
empfunden werden. Die Jugendpolitik muss auf eine
Förderung der Familien ausgerichtet werden und das
Ziel verfolgen, dass die Kinder inihren Familien bleiben.

Dazu müssen die Beratung und Sanktionierung von
Familien innerhalb der Jugendämter auf zwei unab-
hangige Stellen verteilt und strikt getrennt werden. Es

ist seitens der Fachaufsicht und aller Leitungsebenen
auf die Einhaltung der Gesetze zu achten, insbesonde-
re muss kostenlose Akteneinsicht gewährt werden, es
muss eine Angemessenheitsprüfung aller Leistungen
stattfinden, ein Kindesentzug darf nur für eine kurze Zeit
angeordnet werden und das Wunsch- und Wahlrecht
für Leistungen des Jugendamtes nach 816 SGB VIII muss
respektiert werden.

Wir wollen darüber hinaus eine unabhängige Kontrolle
der Jugendämter und der freien Träger installieren, um
dem dort herrschenden Wildwuchs Einhalt zu gebieten.
Die Ombudsstelle muss mit Entscheidungsbefugnissen
ausgestattet und bekannt gemacht werden, damit sich
Eltern auch an diese wenden können.

Willkommenskultur für Kinder

Ungeborene Kinder haben ein Recht auf Leben. Viel zu
oft wird dieses Recht jedoch Zukunftsängsten unter-
geordnet. Solchen Ängsten will die AfD durch konkrete
Hilfen für Familien in allen Lebenslagen vorbeugen,
insbesondere den lebensrettenden Ausweg der
Adoption erleichtern und fördern. Wir wollen die
Möglichkeiten der anonymen bzw. vertraulichen Geburt

Familienpolitik 11

weiter ausbauen und stärken sowie die Verfügbarkeit
von Babyklappen sicherstellen.

Die Gesellschaft muss in Familien, Schule und Medien
den Respekt vor dem Leben und ein positives Bild von
Ehe und Elternschaft vermitteln. Diesen Bedarf erkennt
man daran, dass seit Jahren in Deutschland jahrlich rund
100.000 ungeborene Kinder getötet werden, was der
Zahl der Einwohner einer Grofßsstadt entspricht.

Die verpflichtende Schwangerschaftskontfliktberatung
istin vielen Fällen zu einem formalen Verwaltungsakt
verkümmert und befördert eine Bagatellisierung von
Abtreibungen. Sie muss stattdessen dem Schutz des —
Lebens dienen. Wie vom Bundesverfassungsgerichtzur Wnaeschalis

. . a . . . beratungen müssen
Bedingung gestellt, ist regelmäfßsig die Wirksamkeit der mit transparenten
Beratungsscheinregelung zu überprüfen. Gegebenen- Qualitätsstandards
falls ist durch gesetzliche Korrekturen ein wirksamer und mög’ichst unter

.. . Einbeziehung der Väter
Lebensschutz zu gewährleisten. durchgeführt werden.

Schwangerschaftsberatungen müssen mit
transparenten Qualitätsstandards und möglichst unter
Einbeziehung der Väter durchgeführt werden.
Abtreibungen, speziell aus sozialen und familiären
Gründen, müssen dabei die Ausnahme bleiben. Bei der
Beratung muss explizit über den Entwicklungsstand der
ungeborenen Kinder und mögliche Spätfolgen einer
Abtreibung aufgeklärt werden, wie z.B. das Post-
Abortion-Syndrom (PAS). Eine Abtreibung ist
meldepflichtig, Verstöfse dagegen sind zukünftig zu
ahnden. Die Abtreibungsstatistik soll dadurch die
wirkliche Zahl der Abtreibungen wiedergeben. Die
Entscheidung über eine Abtreibung muss natürlich bei
der Mutter bzw. bei den Eltern liegen, dennoch lehnen
wir alle Bestrebungen ab, die Tötung Ungeborener zu

112

Die AfD fordert auch zur
Vermeidung von Tren-
nungen eine familien-
freundliche Steuer- und
Abgabenpolitik.

Familienpolitikt

einem Menschenrecht zu erklären. Wirwenden uns
strikt gegen die Kommerzialisierung des Mutterleibes
durch bezahlte Leihmutterschaften und gegen die Ver-
marktung von Gewebe getöteter ungeborener Kinder.

Trennungsfolgen mindern

Kinder sind die Hauptleidtragenden, wenn sich die
Eltern trennen. Die daraus resultierenden Schäden
müssen reduziert werden, indem die Problematik aus
der Sicht der Kinder betrachtet wird. Dazu gehört auch
die umfassende Unterstützung und Beratung getrennter
Eltern.

Abstiegsängste und finanzieller Druck belasten Fami-
lien stark und führen oft zu instabilen Ehen und Tren-
nungen. Die AfD fordert daher auch zur Vermeidung
von Trennungen eine familienfreundliche Steuer- und
Abgabenpolitik, die es Familien ermöglicht, auf Wunsch
auch mit nur einem Erwerbseinkommen eine Familie zu
ernähren, ohne gravierende Einbufßsen des Lebensstan-
dards. Auch für Eltern, die beide berufstätig sein wollen,
müssen die weiteren Rahmenbedingungen so ausgelegt
werden, dass Familie und Beruf möglichst gut vereinbar
sind.

Da die Mehrheit der Trennungskinder bei den Müttern
aufwächst, ist ein besonderes Augenmerk auf die Ein-
beziehung der Väter in die Erziehung zu legen. Kinder
brauchen Vater und Mutter, die jeweils unterschiedliche
Aspekte für die Entwicklung einbringen. Kinder dürfen
nicht als Druckmittel oder Instrument der Rache dienen,
indem der Kontakt dem anderen Partner erschwert oder

Familienpolitik 113

verwehrt wird. Die Umgangsrechte und-pflichten beider
Elternteile müssen entsprechend gestärkt werden. Wir
fordern auch bei nnichtehelichen Kindern ein
gemeinsames Sorgerecht ab Geburt.

Die Reform des Familienrechts von 1977 führte dazu,

dass selbst Straftaten und schwerwiegendes Fehlverhal-

ten gegen den Ehepartner bei der Bemessung finanziel-

ler Ansprüche nach Trennung und Scheidung oftohne

Auswirkung bleiben. Eine derartige Rechtsprechung

ist nicht geeignet, die Partner zu ehelicher Solidarität

anzuhalten und beeinträchtigt die Stabilität bestehender

Ehen. Ohne das Schuldprinzip wiedereinzuführen, muss

aber schwerwiegendes Fehlverhalten gegen die eheliche

Solidarität bei den Scheidungsfolgen wieder verbindlich —

.. . . Die AfD fordert, Kitas
berücksichtigt werden. und Schulen von Angst
und Hysterie frei-
zuhalten und eine
positive Lebens-

einstellung zu
vermitteln.

Verantwortungsvolle Kinderbetreuung

Kinder sollten frei von Indoktrination aufwachsen, bis
siein der Familie gefestigt und alt genug sind, sich den
Problemen unserer Welt zu stellen. Ideologen
verschiedener Richtungen versuchen aber, Kinder
möglichst früh zu beeinflussen. Das wurde bereits im
Dritten Reich und in der DDR in grofsem Stil praktiziert.
Politische Ideologien, wie z.B. Genderwahn und
Klimahysterie, werden den Kindern heute schon im Vor-
schulalter nähergebracht.

Die AfD fordert, Kitas und Schulen von Angst und
Hysterie freizuhalten und eine positive Lebens-
einstellung zu vermitteln.

114

Um die Vereinbarkeit
von Familie und Beruf
zu vereinfachen, sind
Betriebskindergärten
stärker als bisher zu
fördern.

Familienpolitik

Häufig wird die politische Beeinflussung von einer
Frühsexualisierung im Sinne „diverser Geschlechter-
rollen begleitet. Die „Sexualpädagogik der Vielfalt‘
versucht, Kinder in Bezug auf ihre sexuelle Identität zu
verunsichern und Geschlechterrollen aufzulösen. Sie
werden dadurch massiv in ihrer Entwicklung gestört.
Kinder haben das Recht auf Schutz ihrer Intimsphäre,
damit sie ihre Sexualität selbstbestimmt herausbilden
können.

In Kindergärten muss Deutsch gesprochen werden.
Darauf müssen Erzieher, Träger und Behörden aktiv
hinwirken.

Um die Vereinbarkeit von Familie und Beruf zu verein-
fachen, sind Betriebskindergärten stärker als bisher zu
fördern. Sie reduzieren die Fahrtwege und vereinfachen
die Abstimmung von familiären und beruflichen Ver-
pflichtungen.

Das Geschlecht ist eine biologische Tatsache

Die menschliche Spezies besteht aus zwei Geschlech-
tern, dem männlichen und dem weiblichen. Diese Zwei-
geschlechtlichkeit wird nicht dadurch aufgehoben, dass
bei wenigen einzelnen Personen eine Zuordnung zum
männlichen oder weiblichen Geschlecht schwer oder
gar nicht möglich ist. Es versteht sich von selbst, dass
diese Menschen vollwertiger Teil unserer Gesellschaft
sind und mit derihnen gebührenden Achtung behandelt
werden müssen. Wir halten es aber für unangemessen,
das biologische Regel-Ausnahme-Verhältnis umzukeh-
ren und für eine sehr geringe Zahl von Menschen neue
Kategorien der geschlechtlichen Klassifizierung (z.B.
durch Änderung der Toilettenordnung) einzuführen.

Familienpolitik 115

Das biologische Geschlecht wirkt sich unmittelbar auf
viele Verhaltensaspekte von Männern und Frauen aus.
Die Geschlechtsrollenbilder in den verschiedenen Kul-
turen bauen darauf auf. Sie können variieren, allerdings
darf der Einfluss kultureller und sozialer Aspekte nicht
überschätzt werden. Der Mensch ist kein beliebig um-
formbares Geschöpf, sondern bewegt sich stetsin den
von der Natur gesetzten Grenzen.

Die Gleichberechtigung von Mann und Frau ist ein hohes
Gut und im Grundgesetz verankert. Die biologische
Geschlechterbindung des Menschen betrachten wir
nicht als Last, sondern als Geschenk. Die AfD fordert
daher die Würdigung auch traditioneller Lebensentwür-
fe und die Wertschätzung der Lebensleistung von Frau-
en, die Familien gründen und Kinder grofsziehen. Frauen 11. app fordor die
sollen genauso wie Männer entscheiden dürfen, welchen würdigung traditio-
Lebensweg sie einschlagen, ohne mit gesellschaftlicher ""senwwure
.. \ und die Wertschätzung
Achtung rechnen zu müssen, wie es aktuell im linken der Lebensleistung von
Zeitgeist verankert ist. Frauenquoten betrachten wir Frauen, die Familien

. 2: . gründen und Kinder
dementsprechend nicht als fortschrittlich. Sie stellen großziehen.
eine Form der Diskriminierung dar und verfestigen
das Bild, dass ein erfülltes und anerkanntes Leben für
Frauen nur durch eine berufliche Karriere erreicht
werden könne.

Das Geschlecht wird durch die
Geschlechtschromosomen bestimmt. Wir sehen es
kritisch, dass „Geschlechtsumwandlungen’ zunehmend
bagatellisiert werden. Geschlechtsumwandlungen

bei Kindern und Jugendlichen lehnen wir ab. Eine so
schwerwiegende Entscheidung darf nur mit der Reife
und Einsichtsfähigkeit des Erwachsenenalters getroffen
werden.

KAPITEL 12

Arbeits- und
SJeyArliste)iiai..

118

Die AfD lehnt die von
der EU vorangetriebene
Aufweichung des Ein-
stimmigkeitsprinzips im
Bereich der Sozialpolitik
ab.

Arbeits- und Sozialpolitik

Arbeits- und Sozialpolitik

Zuwanderung von EU-Ausländern in unsere
Sozialsysteme begrenzen

Hunderttausende EU-Ausländer werden als
Regelleistungsberechtigte in der Grundsicherung für
Arbeitsuchende geführt. Die AfD fordert, die nationalen
Vorschriften zum Leistungsbezug der Grundsicherung
für Arbeitsuchende dahingehend abzuändern, dass ein
Leistungsanspruch nur für solche EU-Ausländer besteht,
die unter Aufnahme einer existenzsichernden Tätigkeit
einreisten beziehungsweise ihren gewöhnlichen
Aufenthalt begonnen und diese über einen
angemessenen Zeitraum ausgeübt haben. Der Bezug
von Leistungen der Grundsicherung für Arbeitsuchende
im Anschluss an eine existenzsichernde Tätigkeit bzw.
über den Bezug von Arbeitslosengeld hinaus soll auf
maximal ein Jahr begrenzt werden.

Nationale Gesetzgebungshoheit im
Sozialrecht verteidigen

Ein stabiles, leistungsgerechtes Sozialsystem gehörtin
die nationale Verantwortung. Eine dauerhafte Abkehr
vom Einstimmigkeitsprinzip auf EU-Ebene im Bereich
der Sozialpolitik zugunsten von Mehrheitsentscheidun-
gen wäre faktisch eine weitere Abgabe von Hoheitsrech-
ten an die Europäische Union. Die AfD lehnt aus diesem
Grund die von der EU vorangetriebene Aufweichung des
Einstimmigkeitsprinzips im Bereich der Sozialpolitik ab.

Arbeits- und Sozialpolitik 119

Keine Auszahlungen von Sozialleistungen auf
ausländische Konten

Sozialleistungen wie Hartz IV oder Sozialhilfe werden je
nach Wunsch des Berechtigten auch auf ausländische
Konten überwiesen. Überzahlungen oder Fehlzahlungen
können nur unter erschwerten Bedingungen zurück-
geholt werden, wodurch dem deutschen Sozialsystem
ein erheblicher finanzieller Schaden entsteht. Dieser
belastet jeden Steuerzahler und die Solidargemeinschaft
zusätzlich. Deshalb fordert die AfD, Sozialleistungen nur
noch aufinländische Konten zur Auszahlung zu bringen.

Lohndumping in der Leiharbeit verhindern

Leiharbeitnehmer werden vergleichbaren Stammarbeit-

nehmern hinsichtlich der Entlohnung ab dem ersten

Arbeitstag mindestens gleichgestellt. Um dem erhöh- eabeinenmer
ten Kündigungsrisiko und der erwarteten Flexibilität sollen vergleichbaren
der Leiharbeitnehmer Rechnung zu tragen, wird eine u
Flexibilitätsprämie eingeführt. Sie wird als prozentualer  onnung ab dem ersten
Aufschlag auf das mafsgebliche Arbeitsentgelt geleistet. Arbeitstag mindestens
Die bisherige Möglichkeit der stufenweisen Heran- gieichgestelt werden
führung des Entgeltes entfällt. Wirksam geschlossene

(Branchen-) Zuschlagstarifverträge behalten bis zum

vereinbarten Ablauf ihre Gültigkeit. Eine Verlängerung

ist ausgeschlossen.

Mindestlohn beibehalten

Der gesetzliche Mindestlohn ist mit dem Wesen der
Sozialen Marktwirtschaft eng verbunden. Er korrigiert im
Bereich der Entlohnung die Position der
Niedriglohnempfänger als schwache Marktteilnehmer

120

Die AfD setzt sich für
eine Bezugsdauer von
Arbeitslosengeld Iin
Abhängigkeit von der

Dauer der Vorbeschäfti-

gungein.

Arbeits- und Sozialpolitik

gegenüber den Interessen der Arbeitgeber als
vergleichsweise starke Marktteilnehmer. Er schützt sie
auch vor dem durch die derzeitige Massenmigration zu
erwartenden Lohndruck. Insbesondere erlaubt der
Mindestlohn eine Existenz jenseits der Armutsgrenze
und die Finanzierung einer, wenn auch bescheidenen,
Altersversorgung, die ansonsten im Wege staatlicher
Unterstützung von der Gesellschaft zu tragen ware.
Mindestlöhne verhindern somit die Privatisierung von
Gewinnen bei gleichzeitiger Sozialisierung der Kosten.

Arbeitsleben würdigen -
Arbeitslosengeld I gerecht gestalten

Ein Arbeitnehmer, der nach langen Jahren der Erwerbs-
tätigkeit arbeitslos wird, hatim Regelfallnur Anspruch
auf ein Jahr Arbeitslosengeld I. Anschliefend erhälter
Arbeitslosengeld II wie ein Arbeitsloser, der noch nie
zuvor in die Arbeitslosenversicherung eingezahlt hat.

Dies stellt eine Geringschätzung der langjährigen Bei-
tragszahler dar. Die AfD setzt sich für eine Bezugsdauer
von Arbeitslosengeld Iin Abhängigkeit von der

Dauer der Vorbeschäftigung ein. Der Selbstbehalt bei der
Berechnungvon Arbeitslosengeld Il ist sanktionsfrei zu
erhöhen. Dies ist ein Gebot der Gerechtigkeit, um unter-
schiedliche Erwerbsbiografien bei Arbeitslosigkeit auch
differenziert zu behandeln.

Arbeits- undSozialpolitik 121

Reform der Arbeitslosenversicherung

Der Beitragssatz in der Arbeitslosenversicherung soll
abgesenkt werden, um die Einkommen der Arbeitneh-
mer zu erhöhen und den Produktionsfaktor Arbeit zu
entlasten. Dazu soll die Arbeitslosenversicherung von
versicherungsfremden Leistungen entlastet werden und
der Kreis der Leistungsberechtigten striktan dem Kreis
der Beitragszahler ausgerichtet werden. Die Versicher-
ten sollen beim Bezug des beitragsfinanzierten Arbeits-
losengeldes I nicht mehr mit Sanktionen gegängelt
werden.

Aktivierende Grundsicherung - Arbeit,diesichloehnt * /@.«@ss
— 0000000000000 dem Erwerbstätigen

Die AfD will eine „Aktivierende Grundsicherung? als stets ein spürbarer

A . its] | Anteil des eigenen Ver-
ternative zum Arbeitslosengeld II (sogenanntes „Hartz  nstes verbleibt und

IV“). Das erzielte Einkommen soll nicht wie bisher das erzielte Einkommen

vollständig mit dem Unterstützungsbetrag verrechnet nicht mehr volständig
mit dem Unterstüt-

werden. Stattdessen verbleibt dem Erwerbstätigen stetS _zungsbeitrag verrechnet
ein spürbarer Anteil des eigenen Verdienstes. Dadurch  werdensoll
entstehen Arbeitsanreize. Wer arbeitet, wird aufjeden

Fallmehr Geld zur Verfügung haben als derjenige, der

nicht arbeitet, aber arbeitsfähig ist (Lohnabstandsgebot).
Missbrauchsmöglichkeiten sind auszuschliefsen.

Finanzkontrolle Schwarzarbeit stärken

Die Bekämpfung von Schwarzarbeit, illegaler Beschäf-
tigung und Verstöfsen gegen das Mindestlohngesetz ist
Aufgabe der Finanzkontrolle Schwarzarbeit (FKS).

122

Die AfD fordert die
Schaffung von Anreizen
in Form eines Bonussys-
tems für die Einrichtung
von mehr Arbeitsplätzen
mit fairer Entlohnung
für Menschen mit
Behinderung.

Arbeits- und Sozialpolitik

Die AfD fordert, die Finanzkontrolle Schwarzarbeit beim
Personalaufbau zu stärken, indem u.a. die
Personalgewinnung über eine neue eigenständige
zentrale Organisationseinheit organisiert und die
Ausbildung der FKS-Mitarbeiter neu geordnet wird.

Arbeitsrecht reformieren und vereinfachen

Es soll ein Arbeitsgesetzbuch geschaffen werden, wel-
ches die zahlreichen Einzelgesetze systematisch zusam-
menführt und ergänzt und damit Klarheit und Rechts-
sicherheit schafft.

Gezielte Maßnahmen gegen Wohnungs-
und Obdachlosigkeit

Das Problem der Wohnungs- und Obdachlosigkeit wurde
bislang gesellschaftlich eher am Rande wahrgenommen.
Die steigende Zahl der Wohnungs- und Obdachlosen
zeigt jedoch die mangelhafte Planung im sozialen
Wohnungsbau, die ihre Ursache in der fehlenden kon-
kreten Bedarfsermittlung hat. Die AfD fordert daher

die sofortige Einführung einer bundesweiten zentralen
Statistik zur Erfassung der Wohnungs- und
Obdachlosigkeit.

Mehr Arbeitsplätze mit fairer Entlohnung für
schwerbehinderte Menschen

Trotz der gesetzlichen Verpflichtung, Arbeitsplätze

für behinderte Menschen vorzuhalten, ist die Zahl der
arbeitslosen schwerbehinderten Menschen deutlich zu
hoch. Um echte Teilhabe für behinderte Menschen am

Arbeits- und Sozialpolitik

Arbeitsleben zu verwirklichen, fordert die AfD die
Schaffung von Anreizen in Form eines Bonussystems für
alle Arbeitgeber für die Einrichtung von mehr Arbeits-
plätzen für Menschen mit Behinderung, gekoppelt mit
einer fairen Entlohnung.

Verbesserungen für Menschen mit Behinderungen in
Krankenhäusern

Menschen mit Behinderungen sollen in Krankenhäusern
und Reha-Einrichtungen besser betreut werden. Dafür
sollen die Voraussetzungen für eine soziale Assistenz
geschaffen werden, wodurch dann auch die Angehörigen
entlastet würden.

123

KAPITEL 13

Das Renten-
konzept der AfD

126

Wir wollen unseren
Eltern und Grofseltern
im Rentenalter einen
Lebensabend in Würde
ermöglichen.

Das Rentenkonzept der AfD

Das Rentenkonzept der AfD

Als Partei der Leistungsgerechtigkeit istes der AfD ein
besonderes Anliegen, unseren Eltern und Großseltern
im Rentenalter einen Lebensabend in Würde zu ermög-
lichen. Sie haben unser Land mit ihrer Arbeit zu einer
der führenden Industrienationen der Welt aufgebaut.
Die drohende Altersarmut für Millionen Leistungsträger
ist für uns schon aus diesem Grund unerträglich. Der
entwürdigenden und ungerechten Rentenpolitik der
Altparteien erteilen wir daher eine deutliche Absage.
Zugleich müssen wir das Rentensystem für die Jungen
vor dem Hintergrund der demographischen Entwicklung
bezahlbar halten.

Renten zukunftsfest

Es braucht einen ausgewogenen Ansatz, der die Belange
aller Betroffenen einbezieht. Der drohenden
Überlastung der Beitragszahler muss durch einen
höheren Steuerzuschuss in der Rentenfinanzierung
entgegengewirkt werden, versicherungsfremde
Leistungen sind aus Steuermitteln zu begleichen. Dieser
höhere Steueraufwand darf jedoch nicht durch
Steuererhöhungen finanziert werden. Vielmehr sind die
Steuerzuschüsse zur Rente durch konsequente Strei-
chungen von ideologischen Politikmafsnahmen, bei-
spielsweise in der Migrations-, Klima- und EU-

Politik, gegenzufinanzieren. Gute Renten sind künftig
nur dann finanzierbar, wenn wir die richtigen haushalts-
politischen Prioritäten setzen. Statt linksgrüner
„Weltrettungsprojekte” braucht es marktwirtschaftliche
Freiheit und Öffentliche Investitionen in Bildung und

Das Rentenkonzept der AfD 127

Wissenschaft, um über eine prosperierende Wirtschaft
auch hohe soziale Standards finanzieren zu Können.

Freiheit beim Renteneintritt

Unser Anliegen ist nicht nur ein höheres Mafs an

Leistungsgerechtigkeit in der Rentenversicherung, den

Rentenberechtigten sollen auch mehr Freiheitsrechte

zugestanden werden. Unserem Freiheitsverständnis

nach gehört die Entscheidung über den Eintritt in die

Rentenphase in die Hand des Einzelnen. Die Betroffenen

haben nach einem langen Berufsleben viele

Erfahrungen gesammelt und wissen selbst am besten,

wann sie aus dem Erwerbsleben ausscheiden wollen.

Manche gehen in ihrer Arbeit auf, haben grofse Freude nu

daran und wollen so lange wie möglich arbeiten. Andere _ Soziale Gerechtigkeit ist
haben für ihren Lebensabend andere Pläne, wollen mehr ” nn
Zeit mit dem Partner verbringen, aufReisengehenoder muss aber immer auch
sich um die Enkelkinder kümmern. Wir wollen deshalb Zur Eigenverantwortung
jedem ermöglichen, länger zu arbeiten und im Einver- nn

nehmen mit dem jeweiligen Arbeitgeber zum

individuellen Wunschzeitpunkt den Ruhestand

anzutreten. Die Rentenhöhe hängt dabei von den

eingezahlten Beiträgen und dem Renteneintritt ab. Wer

länger arbeitet, bekommt entsprechend mehr Rente.

Altersarmut verhindern

Unabhängig vom Zeitpunkt des Renteneintritts muss
immer gelten: Wer lange in die Rentenkasse einge-
zahlt hat, sollte auch bei einem geringeren Einkommen
bessergestellt werden als Personen, die weniger ein-
gezahlt haben. Dem wird die AfD gerecht, indem 25%
der Altersrente nicht auf die Grundsicherung im Alter

128 Das Rentenkonzept der AfD

angerechnet werden. Auf diese Weise wird Altersarmut
verhindert oder zumindest deutlich verringert. Wahrend
der Erwerbsphase lohnt es sich daher, auch Tätigkeiten
mit geringem Einkommen anzunehmen, aus denen sich
haufig auch höher bezahlte Beschäftigungsverhältnisse
ergeben. Soziale Gerechtigkeit ist ein integraler Bestand-
teil der AfD-Politik, muss aber immer auch zur Eigenver-
antwortung anregen.

Abschaffung der Politikerpensionen

— Diese Eigenverantwortung müssen wir auch von
Politiker sollen wie Politik inford Esij . d ttelb
andere Arbeitnehmer olitikern einfordern. Es ist niemandem vermittelbar,
auch in die gesetzliche dass Berufspolitiker bereits nach wenigen Jahren hohe
Kentenversicherung  Pensionsansprüche erwerben. Das ist vor allem kritisch,
einzahlen. Auch dies ist on .
Ausdruck unseres weil die Nutzniefßser dieser Gesetze selbst über solche
Verständnisses von Regelungen gesetzgeberisch entschieden haben. Damit
sozialer Gerechtigkeit. . 2: .

muss Schluss sein. Politiker sollen wie andere

Arbeitnehmer auch in die gesetzliche

Rentenversicherung einzahlen. Auch dies ist Ausdruck

unseres Verständnisses von sozialer Gerechtigkeit.

Reduzierung des Beamtenstatus auf
originär hoheitliche Aufgaben

Zu diesem Verständnis gehört auch ein neuer Umgang
in der Frage „Staatsbedienstete und Renten-
versicherung‘. Unsere Beamten sind das Rückgrat der
deutschen Verwaltung und eine wesentliche Stütze für
das im internationalen Vergleich gut funktionierende
Staatswesen. Die hohe Loyalität der Beamten beruht
auch auf dem Prinzip der Fürsorgeptlicht des Diens-
therrn. Darauf beruht die beamtenrechtliche Altersver-
sorgung.

Das Rentenkonzept der AfD 129

Die AfD möchte sie deshalb beibehalten, allerdings die
Verbeamtung auf hoheitliche Aufgaben beschränken,
wie z.B. bei Bundeswehr, Zoll, Polizei, Finanzverwaltung
und Justiz. Auf diese Weise wird ein Grofsteil der
künftigen Staatsbediensteten in die gesetzliche
Rentenversicherung aufgenommen und deren
Beitragsaufkommen gestärkt.

Lastengerechtigkeit zwischen Familien
und Kinderlosen herstellen

Besondere Beachtung verdienen im
Sozialversicherungssystem die Familien. Eltern tragen
die Lasten der Kindererziehung. Die späteren
Leistungen der Kinder, insbesondere in der
Rentenversicherung, kommen aber allen Rentnern
zugute, auch den Kinderlosen. Familien mit Kindern
tragen daher Sonderlasten für die Allgemeinheit.

. .. . . . Die AfD will die Familien
Die AfD möchte hier einen Ausgleich herstellen, indem entlasten. Für jedes

Familien für jedes Kind 20.000 Euro Beiträge der Eltern nd bekommen die
. . Eltern 20.000 Euro der
zur Rentenversicherung aus Steuermitteln erstattetbe- zunagesurRenten-
kommen, ohne dass sich die Rentenansprüche dadurch _ versicherung erstattet
verringern. Wurden zum Zeitpunkt der Geburten noch
keine Beiträge in entsprechender Höhe gezahlt, erfolgt
eine Anrechnung auf zukünftige Rentenbeiträge. Die
Eltern werden dadurch in dem Lebensabschnitt wir-
kungsvoll unterstützt, in dem die Kosten für die Kinder
anfallen und gegebenenfalls ein Elternteil vollständig
oder teilweise auf das Erwerbseinkommen verzichtet.
Dies wird nicht nur die Entscheidung für ein Kind oder
weitere Kinder erleichtern, sondern insgesamt
Leistungsdruck aus den Familien nehmen und damit die
Trennungsquote verringern.

130

Durch die Anhebung
des Steuerfreibetrags
wollen wir insbesondere
niedrige Renten vor der
Besteuerung bewahren.

Das Rentenkonzept der AfD

Ungerechtigkeiten beider
Überleitung der Ostrenten beseitigen

Neben den Ungerechtigkeiten gegenüber Familien
bestehen auch 31Jahre nach der deutschen
Wiedervereinigung noch immer Ungerechtigkeiten bei
der Überleitung der Ostrenten. Beiderinden
neunziger Jahren erfolgten Rentenüberleitung mit dem
Renten-Überleitungsgesetz und dem Anspruchs- und
Anwartschaftsüberführungsgesetz ist es zu Ungereimt-
heiten gekommen. Die im Alterssicherungssystem der
„DDR enthaltenen Regelungen für besondere Berufs-
gruppen wurden nur teilweise umgesetzt. Nach dem
Auslaufen von Übergangsregelungen ergeben sich
erhebliche Unterschiede je nach Rentenbeginn. Für die
Härtefälle und groben Unbilligkeiten im Rentenüber-
leitungsprozess werden wir einen Ausgleich durch eine
Fondslösung schaffen. Im Rahmen der Fondslösung
sind den Betroffenen pauschalierte Einmalzahlungen in
angemessener Höhe zu gewähren. Bei der Bemessung
der Einmalzahlungen soll - soweit dies möglich ist- an
die zurückgelegte Betriebszugehörigkeit angeknüpft
werden.

Keine Doppelbesteuerung, Steuerfreibetrag anheben

Neben den besonderen Ungerechtigkeiten gegenüber
Ostrentnern und Familien in ganz Deutschland gibt es
auch grundsätzliche Ungerechtigkeiten bei der Renten-
besteuerung. Viele Rentner, die seit 2005 aufihre Renten
Steuern zahlen, unterliegen einer Doppelbesteuerung.
Die rechtlichen Regelungen sind so anzupassen, dass
dieser verfassungswidrige Zustand abgeschafft wird.
Generell ist die Besteuerung von Renten aus sozialer

Das Rentenkonzept der AfD

Sicht kritisch zu beurteilen. Wir wirken der Einkommen-
steuerlast dadurch entgegen, dass wir den Steuerfrei-
betrag - wie im steuerpolitischen Abschnitt dargestellt

- für alle anheben und so zumindest niedrige Renten vor
der Besteuerung bewahren.

Umlagefinanzierte Rente stärken

Mit unserem ausgewogenen Ansatz und den daraus
abgeleiteten Mafßnahmen werden wir das umlagefinan-
zierte Rentensystem für den Eintritt der Babyboomer ins
Rentensystem tauglich machen. Wir erhöhen die
Leistungsgerechtigkeit deutlich, ohne die nach-
kommenden Generationen zu überlasten. Wir zeigen,
dass Gegenwartsinteressen und generative
Verantwortung in Einklang zu bringen sind.

Wehrdienst rentenpolitisch würdigen

Die AfD hat sich zum Ziel gesetzt, die ausgesetzte
Wehrpflicht wiedereinzuführen. In der gesetzlichen
Rentenversicherung wird die Zeit des Wehrdienstes
folglich so in Ansatz gebracht, als habe der
Wehrdienstleistende Beiträge und Verdienste im
Umfang eines durchschnittlichen Einkommens erzielt.
Je Monat der Dienstzeit werden somit 1/12 Entgeltpunkte
dem Rentenversicherungskonto gutgeschrieben. Diese
Regelung kommt auch den Wehrersatzdienstleistenden
(Zivildienstleistenden), die den Dienst an der Waffe aus
Gewissensgründen ablehnen, zugute.

131

Für Wehr- und Zivil-
dienstleistende wird je
Monat der Dienstzeit
1/12 Entgeltpunkte dem
Rentenversicherungs-
konto gutgeschrieben.

KAPITEL 14

Gesundheits-
politik

134

Unser Ziel isteine am
Menschen orientierte
und wohnortnahe medi-
zinische Versorgung.

Gesundheitspolitik

Gesundheitspolitik

Die Gesundheitspolitik der AfD baut auf dem
bestehenden deutschen Gesundheitssystem auf.

Es zeigen sich jedoch zunehmend Fehlentwicklungen,
denen sich die Alternative für Deutschland mit Reform-
vorschlägen annimmt. Unser Ziel ist eine am Menschen
orientierte und wohnortnahe medizinische Versorgung.
Die Effizienz unseres Gesundheitswesens soll neu
bewertet werden, Eigenverantwortlichkeit, natürliche
Prävention sowie Therapiefreiheit gefördert und der Ein-
flussnahme von Lobbyisten strengere Grenzen gesetzt
werden.

Schluss mit den unverhältnismäßigen
Corona-Maßnahmen

Die Verunsicherung der Bürger mit willkürlichen
Grenzwerten, nicht nachvollziehbaren Mafsnahmen und
Einschränkungen von Grundrechten durch
Verwaltungsermächtigungen unter Umgehung der
Parlamente sind für uns inakzeptabel. Der Schwerpunkt
freiwilliger Hygienemafsnahmen und Schutz-
bestimmungen muss auf gefährdeten Bevölkerungs-
gruppen liegen.

Das Tragen von Masken in Kindertagesstätten, Horten
und Schulen lehnen wir ab. Die unverhältnismäfsigen
Lockdown-Mafsnahmen sind unverzüglich zu beenden.
Die Pflicht zum Tragen einer Maske lehnen wir ab.

Eine verpflichtende Impfung (direkt und indirekt),
Immunitätsausweise, sowie Tracking Apps oder sonstige
Überwachungsmaßnahmen lehnen wir ab.

Gesundheitspolitik 135

Wir wollen dafür sorgen, dass der öffentliche und
wissenschaftliche Diskurs sowie die Achtung des
Meinungspluralismus gewahrt bleiben.

Wir fordern die Rückkehr zu bewährten wissenschaftli-
chen Diagnosemethoden zur Feststellung einer
Infektion. Ein Schnelltest- oder PCR-Test, der laut
Entwickler und WHO allein nicht zum Infektionsnach-
weis geeignet istund dessen Falschpositiv-Rate zu
künstlich überhöhten Inzidenzwerten führt, darf nicht
länger die Grundlage für Freiheitsbeschränkungen
durch staatlich verordnete Lockdowns, Zugangsbe-
schränkungen und Quarantäne sein.

Impfen muss freiwillig bleiben — |
Der Schwerpunkt frei-

Die AfD verteidigt das Grundrecht auf körperliche nen \
Unversehrtheit und die Verantwortung jedes Einzelnen _ senutzbestimmungen

für seinen Körper. muss auf gefährdeten
Bevölkerungsgruppen

liegen.
Einsetzung eines parlamentarischen

Corona-Untersuchungsausschusses

Wir setzen uns dafür ein, dass im Rahmen eines
Corona-Untersuchungsausschusses die gesamte
Corona-Politik der Bundesregierung, die Ausrufung
der Epidemie von nationaler Tragweite, dernichtim
Grundgesetz verankerte Corona-Gipfel, der Einsatz
und die Aussagekraft des PCR-Tests, die beliebig festge-
setzten Grenzwerte, die Informationspolitik, fehlender
wissenschaftlicher Diskurs, die Verhältnismäfßsigkeit
der Mafsnahmen, die Einschränkung der Grundrechte,
verpflichtende Tests und Quarantäne sowie die körper-
lichen, psychischen und psychosozialen Schäden durch
die MalSnahmen, die Maskenpflicht ohne Gesundheits-

136

Die AfD fordert die
Einführung eines
Individualbudgets
für Krankenhäuser,
um auch in struktur-
schwachen Gebieten

wohnortnahe medizi-

nische Versorgung zu
ermöglichen.

Gesundheitspolitik

prüfung und die per Notfallzulassung eingesetzten
Impfstoffe genauestens untersucht werden. Ein beson-
deres Augenmerk ist auf die persönliche Bereicherung
von Abgeordneten und Regierungsmitgliedern im
Kontext der Corona-Politik zu legen.

Krankenhäuser und Kliniken brauchen
Planungssicherheit und eine individuell
auskömmliche Finanzierung

Die bestehende Finanzierung in dem DRG-
Fallpauschalensystem hat sich als Zuteilungs- und
Sparsystem mit der „Heckenschere‘ erwiesen. Die
Steuerungsfunktion zur Bereinigung von angeblichen
Überkapazitäten potenziert Verwerfungen und
beschleunigt Kliniksterben insbesondere im ländlichen
Raum.

Die AfD fordert die Einführung eines Individualbudgets
für Krankenhäuser, um auch in strukturschwachen
Gebieten wohnortnah beispielsweise Notfalleinrichtun-
gen, Abteilungen für Geburtshilfe und insbesondere die
stationäre Behandlung von Kindern zu ermöglichen.

In die individuelle Finanzierungsvereinbarung zwischen
den Krankenkassen und den jeweiligen Kliniken fließen
das klinische Leistungsgeschehen, die Prüfungs-
ergebnisse des erweiterten neuen Medizinischen
Dienstes im Gesundheitswesen (MDG), der tatsächliche
Bedarf in der Bevölkerung vor Ort sowie die wirtschaft-
liche Leistungsfähigkeit des jeweiligen Krankenhauses
ein.

Die AfD fordert mit Blick auf die Trägervielfalt und die
Zurverfügungsstellung leistungsfähiger Krankenhaus-
strukturen eine Begrenzung privater Träger im Kranken-
hausbereich bei max. 60%.

Gesundheitspolitik

Serviceorientierte ambulante Versorgung durch
Beendigung der Budgetierung

Die Hälfte der niedergelassenen Ärzte erreichtin den
nächsten Jahren das Rentenalter.

Die Motivation zum Führen einer eigenen Praxis sowie
zu einer längeren Berufsausübung über die übliche
Altersgrenze hinaus ist unter den bestehenden Restrik-
tionen gering. Das bestehende System einer leistungs-
unabhängigen Budgetierung führt zu monatelangen
Wartezeiten auf Facharztbehandlungen, überbordender
Bürokratie, vorzeitigen Praxisschliefsungen und stark
verringertem Wunsch nach beruflicher Selbst-
ständigkeit.

Nicht Kopfpauschalen, Budgetierung und willkürliche
Honorarkürzungen innerhalb der GKV, sondern eine
leistungsgerechte Bezahlung der Mediziner sichert eine
bedarfsorientierte Betreuung des einzelnen Patienten.
Eine unverhältnismäfßsige Leistungsausweitung wird
durch flankierende Mafsnahmen vermieden.

1. Ein weiterentwickelter Medizinischer Dienst im
Gesundheitswesen (MDG), derim Gegensatz zum
bestehenden System paritätisch besetzt und
finanziert sein wird, soll sich aus allen im
Gesundheitssystem Beteiligten zusammensetzen.

Er wird zukünftig nicht nur ein Instrument der
Kostenersparnis für Kostenträger sein, sondern auch
ein Mittel zur Leistungsgerechtigkeit für die Patienten
und für die Behandlungsqualität.

2. Ein mehrstufiges Bonussystem für Beitragszahler,
das notwendige Arztkontakte nicht verhindert, aber
von leichtfertigen Besuchen abhaält, wird eingeführt.

137

Die AfD tritt ein für eine
leistungsorientierte Be-
zahlung der Mediziner.
Damit wollen wir auch
die bedarfsorientierte
Betreuung des einzelnen
Patienten sichern.

138 Gesundheitspolitik

Medizinische Versorgung auf dem Land
wiederherstellen und sichern

Die AfD betrachtet die Förderung der ärztlichen Versor-
gungim ländlichen Raum als eine der aktuell dringends-
ten Aufgaben.

Folgende Mafsnahmen werden die Versorgung dort
stärken:

1. Aufhebung der Kopfpauschalen-Vergütung und der
Budgetierung der ärztlichen Honorierung.

2. Beendigung der Deckelung im Abrechnungssystem
(Degression).

3. Finanzielle und organisatorische Niederlassungs-
hilfen.

A. Abbau der Hürden bei der Anstellung von ärztlichem
Personal, wie z.B. der Jobsharing-Limitation.

5. Weiteren Ausbau von Arztpraxen / Polikliniken/MVZ
mit angestellten Ärzten auch unter der Trägerschaft
der Kommunen, aber unter ärztlicher Leitung.

6. Förderung von Medizinstudenten, die sich nach dem

Die AfD betrachtet Studium für einen gewissen Zeitraum zu einer

die Förderung der Berufstätigkeit in strukturschwachen Gebieten

ärztlichen Versorgung .

im ländlichen Raum als verpflichten.

eine der aktuell 7. Bereitstellung von günstigen Studiendarlehen für

dringendsten Aufgaben. Medizinstudenten, die mit einer - auch anteiligen -
Berufstätigkeit in unterversorgten Landkreisen
sukzessiv erlassen werden.

8. Konsequente Wahrnehmung des Sicherstellungs-
auftrages für eine flächendeckende Versorgungs-
dichte durch die Kassen(zahn-Järztlichen
Vereinigungen.

Gesundheitspolitik

Zugang zu Heil- und Hilfsmitteln entbürokratisieren

Die Genehmigungsptlicht für Hilfsmittel durch die
Gesetzliche Krankenversicherung (GKV) schafft Versor-
gungslücken für den Patienten und verursacht hohen
bürokratischen Aufwand.

Sie ist durch eine Festbetragsregelung zu ersetzen.

Kurzzeitpflegeplätze in Krankenhäusern finanzieren

Im Anschluss an einer Krankenhausbehandlung führt
das Fehlen von direkt anschliefsenden Pflege-
möglichkeiten oftmals zu einer unwürdigen Übergangs-
situation. Die AfD fordert, dass Kurzzeitpflegeplätze in
den Krankenhäusern durch die Pflegeversicherung
finanziert werden. So wird der Druck auf pflege-
bedürftige Menschen vermieden, die haufig befürchten,
dass eine Folgebetreuung in anschließenden Pflege-
einrichtungen oder Zuhause nicht sichergestellt ist.

Beendigung des Pflegenotstands

Steigende Patientenzahlen, Abnahme von qualifiziertem
Pflegepersonal, Überlastung vorhandener Pflegekräfte
und eine Ausgabendeckelung der Kranken- und
Pflegekassen bei der Entlohnung der Pflege sorgen für
eine zunehmende Schieflage im Pflegebereich.

139

Die AfD fordert, dass
Kurzzeitpflegeplätze in
den Krankenhäusern
durch die Pflegever-
sicherung finanziert
werden.

140

Die AfD fordert eine an-
gemessene Bezahlung
der Pflegekräfte über
einen Flächentarifver-
trag und eine bundes-
einheitliche gesetzliche
Personaluntergrenze für
Pflegeeinrichtungen.

Gesundheitspolitik

Daher fordert die AfD:

1. Eine leistungsgerechte angemessene Bezahlung der
Pflegekräfte über einen Flächentarifvertrag mit
steuerfreien Nacht-, Sonn- und Feiertagszuschlägen.

2. Eine bundeseinheitliche gesetzliche Personalunter-
grenze für Pflegeeinrichtungen mit einer Puffer-
regelung bei deren kurzzeitiger Unterschreitung.

3. Regelmäßige Überprüfung der Ergebnis- und
Abrechnungsqualität in Pflegeeinrichtungen.

A. Förderung und Finanzierung der Ausbildung zur
Pflegefachkraft und nicht nur der Pflegehilfskräfte
über das Jobcenter.

Die AfD fordert das Fortbestehen der verschiedenen
Versorgungsformen und die Förderung altersgerechter
Wohnformen. Wir befürworten weiter die getrennte
Berufsausbildung von Gesundheits- und Kranken-
pflege, Kinderkrankenpflege und Altenpflege, die sich
über Jahrzehnte bewährt hat. Die generalisierte
Pflegeausbildung halten wir für eine Fehlentwicklung zu
Lasten der Pflegebedürftigen.

Vorfahrt für die häusliche Pflege

Über 70% der Pflegebedürftigen bevorzugen die
Unterbringung zu Hause. Sie ist nicht nur aus sozialen,
sondern auch aus finanziellen Gründen vorzuziehen.
Die stationäre Pflegebedürftigkeit ist soweit wie möglich
hinauszuschieben. Die Unterstützung von pflegenden
Angehörigen ist durch die weitgehende Angleichung des
Pflegegeldes an die Pflegesachleistungen zu fördern.

Gesundheitspolitik 141

Vereinfachung und bessere
Übersichtlichkeit in der sozialen Pflege- und
gesetzlichen Krankenversicherung

Die AfD schlägt eine Zusammenlegung von sozialer
Pflegeversicherung und gesetzlicher Kranken-
versicherung vor, um häufig auftretende Schnittstellen-
probleme bei der Versorgung von Pflegebedürftigen, die
gleichzeitig häufig auch multimorbide sind,

zu vermeiden.

Der Leistungsumfang der sozialen Pflegeversicherung
soll dabei dem Versicherungsprinzip in der Kranken- —
versicherung angeglichen werden. Da eigene Kinderfür ""uwung von

on . . j pflegenden Angehöri-
das Funktionieren des Sozialstaates unabdingbar sind, gen ist durch die weit-
muss berücksichtigt werden, dass kinderlosen gehende Angleichung
. . des Pflegegeldes an die
Versicherten aufgrund des Wegfalls der Kinder- fiegeg

Pflegesachleistungen zu
erziehung mit all den damit verbundenen Kosten im unterstützen.

höheren Mafßse die Bildung eigener Rücklagen für den
Pflegefall zumutbar ist.

Arzneimittelversorgung verlässlich machen

Die zunehmende Wartezeit insbesondere auflebens-
wichtige Medikamente sowie eine Gefährdung gleich-
bleibender Versorgungsqualität weisen Reformbedarf
auf.

Folgende Forderungen der AfD ergeben sich daraus:

Der überwiegenden Monopolsituation und einseitigen
Abhängigkeit vom Ausland wird durch Abschaffung der
Rabattverträge, Ausweitung des Festbetragsmarktes,
Anpassung der Herstellerrabatte und Abschaffung der
Reimportquote entgegengewirkt.

142

Für die Gewährleistung
einer flächendeckenden
und qualifizierten
Versorgung mit Arznei-
mitteln müssen die
inhabergeführten
Apotheken erhalten
bleiben.

Gesundheitspolitik

Zur Vermeidung einer Überlastung des Arzneimittel-
budgets werden diese Mafsnahmen durch eine
Ausweitung der Nutzenbewertung auch für den
patentgeschützten Anteil der Medikamente und durch
die Absenkung der Umsatzsteuer für Medikamente von
19% auf7 %, finanziert. Inanderen EU - Ländern gilt
bereits für Medikamente ein abgesenkter Steuersatz
oder sie sind ganz steuerbefreit.

Eine Bevorratung von rezeptpflichtigen Medikamenten
für mindestens zwei Monate muss von den Herstellern
sichergestellt werden.

Für die Gewährleistung einer flächendeckenden und
qualifizierten Versorgung mit Arzneimitteln müssen die
inhabergeführten Apotheken erhalten bleiben, die mit
ihrem Liefer-, Nacht-, Sonn- und Feiertagsservice an
365 Tagen die Versorgung der Bürger vor Ort
sicherstellen.

Organspende, Sterbehilfe

Eine Verpflichtung zur Organspende lehnt die AfD ab.
Der Prozess des Sterbens ist durch die bewährte
Palliativmedizin und eine passive Sterbehilfe zu
begleiten.

Cannabis nur in der Medizin

Für medizinische Indikationen sollen unter ärztlicher
Aufsicht Präparate mit dem Hauptwirkstoff zur Verfü-
gung stehen. Wir befürworten den Ausbau der suchtpsy-
chiatrischen Versorgung für eine dauerhafte Abstinenz
von Drogen.

Gesundheitspolitik 143

Antibiotikaresistenzen vermeiden
und konsequent bekämpfen

Wir fordern verpflichtende Untersuchungen auf
multiresistente Keime bei jedem stationären Kranken-
hausaufenthalt. Jede Klinik mit einer Intensivstation
muss verpflichtend einen klinischen Mikrobiologen be-
schäftigen. Desinfektionsfufsmatten, Handedesinfektion
an Haupt- und Stationseingängen, die Umsetzung fester,
verbindlicher Besuchszeiten und Abklatschabstriche in
Kliniken, Pflegeheimen und ambulanten operativen
Einrichtungen werden der lebensbedrohlichen
Vermehrung multiresistenter Keime entgegenwirken.
Unangekündigte Qualitätskontrollen in Kliniken und
Pflegeheimen, Beratungen und Sanktionen der

Kassen(zahn-Järztlichen Vereinigungen bei Wir fordern
überdurchschnittlichem Antibiotikaverbrauch in verpflichtende

. Untersuchungen auf
Praxen runden diese Maßnahmen ab. multiresistente Keime

Die Entwicklung von Reserveantibiotika in Deutschland _DPeiledem stationaren

. . . . u. Krankenhausaufenthalt.
sowie der Phagentherapie soll eine staatliche Förderung '
erfahren.

Den Beruf des Heilpraktikers erhalten

Ausbildungs- und Prüfungsstandards brauchen eine
einheitliche, verbindliche Definition. Im Rahmen der
freien Berufswahl muss es auch Ärzten ermöglicht
werden, den Beruf des Heilpraktikers neben der
ärztlichen Tätigkeit auszuüben.

144 Gesundheitspolitik

Anforderungen an ausländisches Personal im
Gesundheitswesen

Medizinisches Fachpersonal muss generell mindestens
über Sprachkenntnisse auf dem Sprachniveau C1 verfü-
gen. Die fachliche Qualifikation muss uneingeschränkt
dem deutschen Standard genügen. Studien- und Aus-
bildungsbewerber, die sich auf den Einsatz in unterver-
sorgten Gebieten festlegen, sind zu bevorzugen.

Souveränität des Patienten über seine
Daten herstellen

Medizinische Behandlungsdaten zeichnen sich durch
besonders hohe Sensibilität aus. Die AfD lehnt die
Schaffung einer zentralen Datenbank mit der Anbindung
von Kliniken, Praxen, therapeutischen Einrichtungen
und Apotheken zur Speicherung vertraulicher
Patientendaten ab. Wir befürworten eine Speicherung
von Notfalldatensätzen, einer Medikamentenübersicht
oder einer Patientenverfügung direkt aufeiner
Krankenversicherungskarte zum Nutzen des Patienten,
über deren Nutzung dieser eigenverantwortlich

Die afD lehnt diescnap entscheidet.

fung einer zentralen
Datenbank zur Spei-
cherung vertraulicher
Patientendaten ab.

Gesundheitspolitik

WHO grundlegend reformieren

Die Finanzierung sowie die personelle Struktur der
Weltgesundheitsorganisation (WHO) ist grundlegend

zu reformieren, um die Unabhängigkeit und Neutralität
der WHO wiederherzustellen und dem Anspruch einer
internationalen, also zwischenstaatlichen Organisation
gerecht zu werden. Die notwendigen Struktur-
anpassungen sollen zu einer Verschlankung der WHO
und damit zu einer Absenkung der aus Steuermitteln
finanzierten Beiträge und Spenden führen. Wir setzen
uns zudem dafür ein, dass die WHO die begrifflichen
Voraussetzungen für das Vorliegen einer Pandemie
wieder um den bis 2009 gültigen Zusatz „eine enorme
Anzahl von Todes- und Krankheitsfällen' erweitert. Sollte
eine grundlegende Reform nach diesen Mafsstäben nicht
dAurchsetzbar sein, sprechen wir uns für einen Austritt
Deutschlands aus der Weltgesundheitsorganisation aus.

145

KAPITEL 15

Bildung,
Wissenschaft
und Forschung

148

Wir befürworten ein
differenziertes, aber
durchlässiges
Schulsystem, das den
unterschiedlichen
Begabungen der Schüler
gerecht wird.

Bildung, Wissenschaft und Forschung

Bildung, Wissenschaft und
Forschung

Mut zur Leistung statt Akademisierungswahn

Ein leistungsorientiertes, differenziertes Bildungswesen
ist die Grundlage unseres Wohlstands und wesentlicher
Bestandteil unserer Kultur. Während seit Jahrzehnten
jedoch die Abiturientenquote immer weiter steigt, fehlen
den Auszubildenden und Studienanfängern
grundlegende Kenntnisse und Fähigkeiten. Seit dem
PISA-Schock vor 20 Jahren jagt eine Reform die andere,
mit ständiger weiterer Absenkung des Niveaus.

Die Bildungsstandards aller Schulformen und Bildungs-
einrichtungen müssen wieder auf das Niveau einer
führenden Wissenschafts- und Industrienation gehoben
werden. Wir fordern die Abkehr von ausschließlicher
Kompetenzorientierung und eine stärkere fachwissen-
schaftliche Ausrichtung des Unterrichts.

Mehrgliedriges Schulsystem erhalten

Bildungsgerechtigkeit erfordert Differenzierung, nicht
Gleichmacherei. Effizientes Lehren und Lernen ist nur
möglich, wenn die Leistungsunterschiede zwischen den
Schülern innerhalb einer Schulform begrenzt bleiben.

Wir befürworten daher ein differenziertes, aber durch-
lässiges Schulsystem, das den unterschiedlichen
Begabungen der Schüler gerecht wird. Das Abitur muss
wieder zum Ausweis der Studierfähigkeit werden,

der Haupt- oder Realschulabschluss zu qualifizierter

Bildung, WissenschaftundForschung 149

Berufsausbildung befähigen. Schulen in freier Träger-
schaft (Privatschulen) sind eine sinnvolle Ergänzung des
staatlichen Bildungswesens.

Berufliche Bildung stärken: Meister statt Master

Die duale Ausbildung in Unternehmen und Berufs-

schulen ist ein Erfolgsmodell. Jedoch gefährdet das

Streben nach immer höheren Abiturientenquoten den
Nachwuchs in den Ausbildungsberufen. Zahlreiche
Lehrstellen können aus Mangel an ausreichend —

qualifizierten Bewerbern nicht besetzt werden. Wir wollen das berufli-
che Bildungs- und Aus-

bildungssystem stärken.
Wir wollen das berufliche Bildungs- und Ausbildungs- Der Wert der beruflichen

.. . . Bildung muss stärker
system stärken. Der Wert der beruflichen Bildung muss rauachoben werden.
stärker herausgehoben werden.

Digitalisierung ist kein Selbstzweck

Grundsätzlich benötigen Schulen eine moderne,
zeitgemäfse IT-Ausstattung. Dies ist vor allem für den
Informatikunterricht und für die Berufsausbildung in
technischen Fächern notwendig.

Allerdings muss Digitalisierung stets unter den
Prämissen der Sinnhaftigkeit und der Arbeits-
erleichterung betrachtet werden.

Dazu sind verbindliche Kriterien für Rahmenlehr-

pläne und schulinterne Curricula zu definieren, wie sich
digitale Medien innerhalb des Lehr-Lern-Prozesses
didaktisch-methodisch und altersgerecht sinnvoll sowie
datenschutzkonform einsetzen lassen.

150

Die AfD drängt darauf,
bei einer zukünftig
rational gesteuerten
Einwanderungspolitik
den Bildungsstand als
ein wesentliches
Entscheidungs-
kriterium zu
berücksichtigen.

Bildung, Wissenschaft und Forschung

Die ersten vier Schuljahre sollten vorwiegend digitalfreie
Raume sein, da sie der Aneignung der grundlegenden
Kulturtechniken Lesen, Rechnen und Schreiben dienen.

Keine ideologisch motivierte Inklusion:
Förder- und Sonderschulen erhalten

Die Forderung der Vereinten Nationen, behinderten
Kindern Teilhabe am Bildungssystem zu garantieren, ist
in Deutschland bereits erfüllt. Kinder mit besonderem
Förderbedarf erhalten in der Förderschule eine
umfassende Unterstützung, die die Regelschule nicht
leisten kann.

Die AfD setzt sich deshalb für den Erhalt der Förder- und
Sonderschulen ein. Die Förderschule muss wieder zum
Regelfall für Schüler mit sonderpädagogischem Förder-
bedarf werden.

Bildung und Migration

Die Integration von Migranten ist eine Herausforderung
für das deutsche Bildungssystem, da viele von ihnen
anderen Kulturkreisen und zudem oft bildungsfernen
Schichten entstammen. Fehlende Vorbildung und
mangelnde Sprachkenntnisse verhindern in vielen
Fällen eine erfolgreiche Qualifizierung für den
deutschen Arbeitsmarkt. Diese Menschen drohenin
Deutschland persönlich zu scheitern und zu einer
dauerhaften Belastung für den Sozialstaat zu werden.

Die AfD drängt darauf, bei einer zukünftig rational
gesteuerten Einwanderungspolitik den Bildungsstand

Bildung, Wissenschaftund Forschung 151

als ein wesentliches Entscheidungskriterium zu
berücksichtigen. Dafür können die Erfahrungen aus
Japan wegweisend sein.

Für schulpflichtige Asylbewerber muss es Ziel der
Beschulung sein, diese auch für die Möglichkeit ihrer
Rückkehr in ihr Heimatland vorzubereiten. Der Lernfort-
schritt einheimischer Schüler darf nicht

beeinträchtigt werden. Solange die Migranten nicht
hinreichend Deutsch sprechen, um am Regelunterricht
teilzunehmen, ist ein Unterricht in ihrer Muttersprache

eine Option. Die deutschen Kultur-
güter, Traditionen sowie
die Geschichte müssen
bereits ab der Grund-
schule zum Pflichtstoff
des Unterrichts gehören.

Neutralität der Schule wiederherstellen

Das Klassenzimmer darf kein Ort der politischen Indok-
trination sein. An deutschen Schulen wird oft nicht die
Bildung einer eigenen Meinung gefördert, sondern die
unkritische Übernahme ideologischer Vorgaben. Leitbild
der schulischen Bildung muss immer der selbstständig
denkende Bürger sein.

Kompetenzorientierung entwertet Bildung

Mafßsgeblich von Lobbygruppen initiiert, hat die Reduzie-
rung der Bildungsanforderungen auf einen inhaltsleeren
Kompetenzbegriff zu einer empirisch nachweisbaren
Niveausenkung geführt. Geschwätzig formulierte Kom-
petenzraster verdrängen Lehrinhalte. Bildung und das
Streben nach Erkenntnis treten in den Hintergrund.

Wir fordern, den Einfluss von Lobbygruppen auf unser
Bildungswesen zu reduzieren.

152

Die AfD fordert die
flächendeckende
Überprüfung aller
Moscheevereine mit
Koranschulen auf
Verfassungs-
feindlichkeit..

Bildung, Wissenschaft und Forschung

Deutsches Kulturgut im Unterricht vermitteln

Die deutschen Kulturgüter, Traditionen sowie die Ge-
schichte sind immanenter Bestandteil der deutschen
Identität. Die AfD wird allen Bestrebungen, dieses aus
den Lehrplänen zu entfernen, zu verfälschen oder zu
reduzieren, entgegenwirken. Bereits ab der Grundschule
müssen diese Inhalte zum Pflichtstoff des Unterrichts
gehören. Das fördert Heimatliebe und Traditions-
bewusstsein.

Kein bekenntnisgebundener Islamunterricht an
deutschen Schulen

Islamunterricht an deutschen Schulen dient derzeit
nicht der Integration, sondern fördert ein Verharren in
Einstellungen, die echte Integration verhindern. Da die
islamischen Gemeinschaften in Deutschland keine
kirchenähnliche Struktur aufweisen, kann ihnen das
Privileg eines „bekenntnisgebundenen Religionsunter-
richts an staatlichen Schulen nicht zugestanden werden.

Keine Sonderrechte für Muslime an unseren Schulen.

Muslimische Schüler müssen genauso wie alle anderen
Schüler auch am Sport- und Schwimmunterricht sowie
an Klassenfahrten teilnehmen. Nur so Können wir der
Herausbildung von Parallelgesellschaften entgegenwir-
ken und den Schülern Teilhabechancen eröffnen, damit
die Anpassung an unsere deutsche Kultur gelingt.

Bildung, Wissenschaft und Forschung 153

Kein Koranunterricht durch verfassungsfeindliche
Moscheevereine

Die Koranschulen in Deutschland werden zumeist von
Moscheevereinen betrieben. Viele dieser Vereine stehen
unserer freiheitlich-demokratischen Grundordnung
ablehnend gegenüber. Wir fordern deshalb eine flächen-
deckende Überprüfung aller Moscheevereine auf
Verfassungsfeindlichkeit. Auch in Koranschulen muss
die Lehrsprache Deutsch sein.

Keine Konfuzius-Institute an öffentlichen
Bildungseinrichtungen

Über die sogenannten Konfuzius-Institute übt die
kommunistische Partei Chinas Einfluss auf das _

. . BE Wir wollen eine höhere
Bildungsangebot an deutschen Universitäten und zu- | Grundfinanzierung
nehmend auch an allgemeinbildenden Schulen aus. Dies der Hochschulen
steht im Widerspruch zum Neutralitätsgebot und zum einführen, um deren

. a . . oo Abhängigkeit von Dritt-
Demokratieprinzip öffentlicher Bildungseinrichtungen. mitten zu verringern
Wir wollen deshalb die Konfuzius-Institute aus den

Hochschulen wieder herauslösen.

Bologna ist gescheitert:
Diplom und Magister wieder einführen

Die EU-weite Harmonisierung von Studiengängen
(Bologna-Prozess) hat zu einer Verschulung des
Studiums geführt. Die akademische Freiheit wurde
beschnitten, ein Studienplatzwechsel erschwert.
Arbeitgeber kritisieren die geringe Qualifikation von
Bachelor-Absolventen.

154

Die AfD fordert auf
Gender-Ideologie
beruhende Lehre und

Forschung zu streichen.

Für die Besetzung von
Stellen muss allein die
fachliche Qualifikation
der Bewerber
entscheiden.

Bildung, Wissenschaft und Forschung

Die AfD will eine Abkehr von Modularisierung und
Kompetenz-Orientierung und die bewährten Diplom-
und Magisterstudiengänge wieder einführen.

Autonomie der Hochschulen stärken,
Freiheit von Forschung und Lehre bewahren

Deutschland muss wieder ein Land der Spitzen-
forschung werden. Wir wollen eine höhere Grund-
finanzierung der Hochschulen einführen, um deren
Abhängigkeit von Drittmitteln zu verringern. Hoch-
schulen sollen das Recht besitzen, Bewerber durch
Aufnahmeprüfungen auszuwählen. Der durch politische
Zielvorgaben zu Studentenzahlen, Studienerfolg und
anderen Quoten erzeugte Zwang zur Nivellierung ist zu
beenden.

Kein Geld für „Gender Studies“, keine Gender-Quoten

Kern der Gender-Ideologie ist die Leugnung von biologi-
schen Unterschieden zwischen Mann und Frau, verbun-
den mit der Behauptung, dass Geschlechter nur soziale
Konstrukte seien. Unterschiede etwa bei der Auswahl
und Ausübung von Berufen werden als Beweis einer
vermeintlichen Diskriminierung betrachtet und deren
Beseitigung gefordert. Behauptungen der
Gender-Ideologie stehen im Widerspruch zu
Erkenntnissen der Biologie.

Alle Fördermittel für die auf der Gender-Ideologie
beruhende Lehre und Forschung sind zu streichen.
Politisch korrekte Sprachvorgaben zur Durchsetzung der
Gender-Ideologie lehnen wir ab. Gleichstellungs-
beauftragte sind abzuschaffen. Bei der Besetzung von
Stellen muss allein die fachliche Qualifikation des
Bewerbers entscheiden.

155

KAPITEL]1

Kultur
Er <=

158

Als zentrales Element
deutscher Identität will
die AfD die deutsche
Sprache als Staats-
sprache im Grundgesetz
festschreiben.

Kultur

Kultur

Deutsche Leitkultur statt „Multikulturalismus“

Unsere Identität ist geprägt durch unsere deutsche
Sprache, unsere Werte, unsere Geschichte und unsere
Kultur. Letztere sind eng verbunden mit dem
Christentum, der Aufklärung, unseren künstlerischen
und wissenschaftlichen Werken. Unsere Identität
bestimmt die grundlegenden Werte, die von
Generation zu Generation weitergegeben werden.

Die deutsche Leitkultur beschreibt unseren
Wertekonsens, der für unser Volk identitätsbildend ist
und uns von anderen unterscheidet. Sie sorgt für den
Zusammenhalt der Gesellschaft und ist Voraussetzung
für das Funktionieren unseres Staates. Die gemein-
schaftsstiftende Wirkung der deutschen Kultur ist
Fundament unseres Grundgesetzes und kann nicht
durch einen Verfassungspatriotismus ersetzt werden.

Kulturrelativismus und Multikulturalismus führen zu
einem Neben- und Gegeneinander von Parallel-
gesellschaften, denen es an gemeinsamen Werten für
das Zusammenleben fehlt. In einer derart
fragmentierten Gesellschaft entstehen Konflikte, die
kaum noch beherrschbar sind. Die AfD wird nicht
zulassen, dass Deutschland aus falsch verstandener
Toleranz vor dem Islam seine tradierte Kultur verliert.

Kultur

Die deutsche Sprache als Zentrum unserer Identität

Die Nationalsprache ist das Herz einer Kulturnation.
Als zentrales Element deutscher Identität will die AfD
die deutsche Sprache als Staatssprache im Grundgesetz
festschreiben.

An deutschen Bildungseinrichtungen darf das Deutsche
nicht durch andere Sprachen zurückgedrängt werden.
Deutsch muss als Lehr- und Wissenschaftssprache
erhalten bleiben.

Die sogenannte „gendergerechte Sprache’ ist eine
groteske Verunstaltung der deutschen Sprache.
Sie schafft keine Gleichberechtigung.

Sprache darf kein Spielball ideologischer Interessen sein.
Wir lehnen daher insbesondere die sogenannte
„gendergerechte Sprache‘ ab und sprechen uns gegen
jegliche Verpflichtung aus, sie verwenden zu müssen.
Die Verwendung der „leichten Sprache’ sollte auf
kognitiv beeinträchtigte Personen beschränkt bleiben.

Kulturförderung

Die Förderung von Kultureinrichtungen muss transpa-
rent sein. Die staatliche Kulturförderung darf nicht dazu
missbraucht werden, die politische Willensbildung zu
beeinflussen. Vereine und Stiftungen, die staatlich geför-
dert werden, müssen sich zur freiheitlich-
demokratischen Grundordnung bekennen.

Die AfD bekennt sich zur Kulturhoheit der Bundesländer
und will die kulturpolitischen Aktivitäten des
Bundes begrenzen.

159

Die AfD bekennt sich
zur Kulturhoheit der
Bundesländer und will
die kulturpolitischen
Aktivitäten des Bundes
begrenzen.

160

Die AfD fordert, die
Schönheit historischer
Innenstädte zu
bewahren und bei
Bedarf durch
Rekonstruktionen
wiederherzustellen.

Kultur

In der Filmförderung muss die künstlerische Freiheit
gewahrt bleiben. Identitätspolitische Vorgaben der
staatlichen Filmförderung sind abzulehnen.

Brauchtum und Gedenken

Brauchtum wirkt identitätsstiftend und gemeinschafts-
bildend. Deshalb wollen wir unser Brauchtum und
unsere Mundarten pflegen, wozu lokale Kulturvereine
einen wichtigen Beitrag leisten. Das Leid der Heimat-
vertriebenen darf nicht vergessen werden.

Baudenkmäler und heimische Architektur prägen
Heimatgefühl und kulturelle Identität. Die Schönheit
historischer Innenstädte muss bewahrt und bei Bedarf
durch Rekonstruktionen wiederhergestellt werden.

Die deutsche Geschichte ist in ihrer Gänze zu würdigen.
Die offizielle Erinnerungskultur darf sich nicht nur auf
die Tiefpunkte unserer Geschichte konzentrieren, sie
muss auch die Höhepunkte im Blick haben. Ein Volk
ohne Nationalbewusstsein kann auf die Dauer nicht
bestehen.

Die AfD setzt sich gegen die Schmähung des Deutschen
Kaiserreichs ein, das unzutreffend als rückständiger
Unrechtsstaat diffamiert wird. Noch heute zehren wir in
der Bundesrepublik von den kulturellen,
technologischen und wirtschaftlichen Errungenschaften
des Kaiserreichs.

Kultur 161

Wir wenden uns gegen die zunehmend aggressiven

Versuche einer ideologisch geprägten, moralisierenden

Umdeutung der Geschichte, die sich an der Schleifung

von Denkmälern und Umbenennung von Strafsen

festmacht. Die Aggressivität, die sich mittlerweile gegen

das Kaiserreich wie auch gegen die Denkmäler

Bismarcks und der deutschen Kaiser richtet, gilt

nicht nur diesem vergangenen Staat, sondern der Die AfD stellt sich als

deutschen Nation an sich. einzige politische Kraft
der Demontage unserer

historisch-kulturellen
Die Debatte um eine angeblich notwendige Identität entgegen.

„Dekolonisierung' unserer Kultur, die miteiner
Verteufelung des „weilfsen Mannes einhergeht, stellt
das Selbstverständnis unserer kulturellen Identität
insgesamt infrage. Die AfD stellt sich als einzige
politische Kraft dieser Demontage unserer historisch-
kulturellen Identität entgegen. Die sogenannte
“Critical Race Theory’, wonach Rassismus gegen
Weifßse nicht möglich sei, lehnen wir ab.

Die AfD wendet sich auch gegen eine pauschale
Rückgabe von Sammlungsgütern aus kolonialen
Kontexten.

Wir setzen uns ebenfalls dafür ein, den

Beschluss des Bundestages, ein Mahnmal und ein
Informationszentrum für die Opfer der
kommunistischen Gewaltherrschaft zu errichten,
endlich umzusetzen.

KAPITEL 17

Medien
Er <<

164

Wir wollen das
öffentlich-rechtliche
Rundfunksystem grund-
legend reformieren: Die
Rundfunkstaatsverträge
sind in jedem
Bundesland zu
kündigen. Zwangs-
beiträge und Werbung
entfallen. Dafür wird ein
Grundfunk geschaffen.

Medien

Medien

Den öffentlich-rechtlichen Rundfunk neu gestalten

Freier Funk für freie Bürger!

Das öffentlich-rechtliche Rundfunksystem ist überholt.
Die Rundfunkstaatsverträge sind injedem Bundesland
zu kündigen, um eine grundlegende Reform zu
ermöglichen. Am Ende soll ein stark reduzierter
Anbieter stehen, der ca. ein Zehntel des bisherigen
Umfangs haben soll. Dieser „Grundfunk hat nur die
Aufgabe, die Bürger flächendeckend mit neutralen
Inhalten aus den Sparten Information, Kultur und
Bildung zu versorgen. Wichtig bleiben regionale
Inhalte - ein schlanker „Heimatfunk als Schaufenster
der Regionen.

Zwangsbeiträge und Werbung entfallen. Die Finanzie-
rung erfolgt durch eine Abgabe, die insbesondere Tech-
nologiekonzerne, die audiovisuelle Inhalte verbreiten,
sowie Video-Streaming-Dienste zu leisten haben. Diese
dominieren den deutschen Markt, ohne dafür in
angemessener Weise Steuern oder Abgaben zu zahlen.

„Cancel Culture“ bekämpfen

Jede Zensur von Meinungsäufßserungen ist ein Angriff
auf das Grundrecht der Meinungsfreiheit. Eine Vor-
machtstellung in den sozialen Medien und im
Bildungswesen darf nicht dazu missbraucht werden, die
politische Willensbildung einseitig zu beeinflussen.

Als Quasi-Oligopol wollen wir die Anbieter großer

Medien 165

sozialer Medien wie Facebook, Twitter oder Instagram
dazu verpflichten, die Meinungsfreiheit ihrer Nutzer zu
respektieren.

Das Internet muss als Ort der freien Meinungsäußerung
erhalten bleiben. „Faktenprüfer‘ und Meinungswächter
dürfen keine staatliche Finanzierung erhalten.

Über Subventionen, Unternehmensbeteiligungen und
regierungspolitische Werbefinanzierung beeinflusst die
Regierung auch die privaten Medien.

Diese parteipolitische Medienbeeinflussung lehnen wir
ab. Beteiligungen von Parteien an Medienunternehmen
wollen wir verbieten.

Das Internet muss

Meinungsfreiheit statt Tugendterror als Ort der freien
nu Meinungsäußerung
Diffuse Vorstellungen von „politischer Korrektheit erhalten bleiben.

ersticken die öffentliche Diskussion durch Sprach- und
Denkverbote. Tatsachen werden verdreht und
kontroverse Themen tabuisiert. Das Zusammenrücken
der Altparteien zu einem politischen Meinungskartell
hat die linke Dominanz im Öffentlich-rechtlichen
Rundfunk und in privaten Mainstream-Medien
verfestigt. Auch die Freiheit von Wissenschaft,
Forschung und Lehre ist gefährdet. Berufliche
Sanktionen, soziale Ächtung und gewaltsame Übergriffe
lassen die verfassungsrechtlich garantierten Freiheits-
rechte zur Theorie verkümmern. Dies hat sich in der
Corona-Krise in erschreckender Weise gezeigt.

Bauen, Wohnen,
Mieten
U

168

Wir benötigen eine
bundesweite
Vereinfachung und
Vereinheitlichung des
Baurechtes

und eine
Beschleunigung von
Verfahren durch
Bürokratieabbau
und Digitalisierung.

Bauen, Wohnen, Mieten

Bauen, Wohnen, Mieten

Fehlender Wohnungsneubau, Landflucht, die
zunehmende Zahl der Ein-Personen-Haushalte sowie
die ungezügelte und gesetzeswidrige Migration führen
dazu, dass es für untere und mittlere Einkommens-
gruppen in angespannten Märkten zusehends schwerer
wird, eine bezahlbare Wohnung zu finden. Um diesen
Tendenzen entgegenzuwirken, müssen nicht nur die
Voraussetzungen für bedarfsgerechten Wohnungs-
neubau z. B. durch die Ausweisung von Bauland
geschaffen, sondern auch die Kosten substanziell
gesenkt werden. Die AfD fordert daher die Reduktion der
überzogenen Standards und Vorgaben, die Streichung
der Energieeinsparverordnung sowie den Brand-, den
Wärme- und den Schallschutz auf ein notwendiges
Mindestmaß zurückzufahren. Wir benötigen eine
bundesweite Vereinfachung und Vereinheitlichung des
Baurechtes und eine Beschleunigung von Verfahren
durch Bürokratieabbau und Digitalisierung.

Die Grunderwerbsteuer auf selbstgenutzte Wohnim-
mobilien ist ersatzlos zu streichen. Damit Einheimische
besser auf das vorhandene Angebot zugreifen können,
ist der Erwerb von Wohnimmobilien durch Käufer ohne
deutsche Staatsbürgerschaft, deren Hauptwohnsitz im
Ausland liegt, über eine Erhöhung der Grunderwerb-
steuer auf 20% zu erschweren. Die Grundsteuer und
Gebühren wie die EEG-Umlage und die GEZ-Gebühren
(Haushaltsabgabe) sind zu streichen. Der Ausgleich der
Grundsteuer an die Gemeinden ist vom Bund zu leisten.

Bauen, Wohnen, Mieten 169

Wohneigentumsförderung

Wohneigentum befreit den Bürger von der Miet-
zahlung. Es trägt zum Vermögensaufbau bei, minimiert
die Belastung durch Wohnkosten im Alter und vermin-
dert damit Altersarmut. Daneben ist aber Eigentum auch
ein sozialer Faktor, der vielfach unterschätzt wird. Wohn-
gebiete mit einer hohen Eigentumsquote sind sozial sta-
bil. Eigentum führt zu einem verantwortlichen Umgang
mit einer Wohnung und stabilisiert das soziale Umfeld.

Die AfD will die Schaffung von Wohneigentum durch

folgende Mafsnahmen fördern: Staatliche

Wohnungsunternehmen sollen Mietern ihre Wohnung

zum Kauf anbieten. Insbesondere für junge Familien

ist das Erbbaurecht als kostengünstiger Einstieg in das

Eigentum auszubauen. Wohnungsbaugenossenschaften

sollen bei Vergabe von Wohnbauland bevorzugt werden,

um mehr Bürgern den Einstieg in das Wohneigentum zu

erleichtern. Staatliche Bürgschaften als Eigen-

kapitalersatz für bis zu 10% des Objektwertes sollen —

den Kauf von Wohnraum erleichtern. Wir fordern eine . nn \
steuerliche Sonderabschreibung für die eigengenutzte Mietrecht und lehnt

Immobilie. staatliche Überregulie-
rungen sowie Investiti-

onshemmnisse wie die
Mietpreisbremse oder
den Mietendeckel ab.

170

Bauen, Wohnen, Mieten

Mehr Wohngeld statt Sozialer Wohnungsbau

Der bisherige soziale Wohnungsbau ist gescheitert, er
kann nur einen Bruchteil der Berechtigten erreichen. Er
führt zu Fehlbelegungen und verursacht hohe Kosten für
den Steuerzahler. Es ist besser, einkommensschwache
Mieter vermehrt mit Wohngeld zu unterstützen, wo-
durch eine soziale Durchmischung gewährleistet wird.
Reicht das Wohngeld nicht aus, um sich eine Wohnung
zu sichern, ist den Bürgern zusätzlich ein kommunales
Wohngeld zu zahlen. Die AfD steht für ein ausgewogenes
Mietrecht und lehnt staatliche Uberregulierungen sowie
Investitionshemmnisse wie die Mietpreisbremse oder
den Mietendeckel ab.

171

KAPITEL 19

Klima, Energie,

Technik un

Digitalisierung
U

174

Das Ziel der Bundes-
regierung, die
CO,-Emissionen
faktisch auf null zu

senken, lehnt die AfD ab.

Klima, Energie, Technik und Digitalisierung

Klima, Energie, Technik und
Digitalisierung

Dem Klimawandel positiv begegnen

Das Ziel der Bundesregierung, die CO,-Emissionen
faktisch auf null zu senken, führt zu einem radikalen
Umbau von Industrie und Gesellschaft (‚Die Große
Transformation‘ / ‚The Great Reset‘) und bedroht unsere
Freiheit in einem immer beängstigenderen Ausmaß.
Die AfD lehnt dieses Ziel und den damit verbundenen
Gesellschaftsumbau ab.

Begründet wird der radikale Umbau von Industrie

und Gesellschaft mit der Behauptung, dass durch eine
Dekarbonisierung, also den Verzicht auf die Nutzung
von Kohle, Öl und Gas, eine vom Menschen gemachte
„Klimakatastrophe' noch verhindert werden könne. Wir
halten diese These und die daraus abgeleiteten Mafsnah-
men für falsch.

Das Spurengas CO, ist als Voraussetzung für alles Leben
unverzichtbar. Der Anstieg der Konzentration von CO, in
der Atmosphäre hat in den letzten Jahrzehnten zu einem
Ergrünen der Erde beigetragen.

Niemand streitet die jüngste globale Erwärmung ab. Die
AfD bezweifelt aber, dass diese nur negative Folgen hat.
Statteinen aussichtslosen Kampf gegen den Wandel des
Klimas zu führen, sollten wir uns an die veränderten Be-
dingungen anpassen, so wie es Pflanzen und Tiere auch
tun. Die Menschheitsgeschichte belegt, dass Warmzeiten
immer zu einer Blüte des Lebens und der Kulturen führ-

Klima, Energie, Technik und Digitalisierung 175

ten, während Kaltzeiten mit Not, Hunger und Kriegen
verbunden waren. Es ist bis heute nicht nachgewiesen,
dass der Mensch, insbesondere die Industrie, für den
Wandel des Klimas maßgeblich verantwortlich ist. Die
jüngste Erwärmung liegt im Bereich natürlicher Klima-
schwankungen, wie wir sie auch aus der vorindustriellen
Vergangenheit kennen.

Kernforderungen zur Klimapolitik

» Das Klima kann keinen durch den Menschen
postulierten und schon gar nicht garantierten
Bestandsschutz haben. Das Klima ist per se nicht
schutzfaähig.

» Den Klimaschutzplan 2050 der Bundesregierung und
Dekarbonisierungsmafsnahmen lehnt die AfD ab.

» Das Pariser Klimaabkommen vom 12.12.2015 ist zu
kündigen. Deutschland muss aus allen staatlichen
und privaten „Klimaschutz -Organisationen
austreten und ihnen jede Unterstützung entziehen.

» Die AfD fordert einen Öffentlichen, freien Diskurs
über die Ursachen von Klimaveränderungen und die
verheerenden Folgen einer Dekarbonisierung.

» Jegliche Form der CO,-Besteuerung ist abzuschaffen.

. . .. Die AfD fordert einen
Technischen Fortschritt begrüßen öffentlichen, freien

. . . . Diskurs über die
Technischer Fortschritt wirdin Deutschland zunehmend _ irsachen von kKlima-

nicht mehr als Chance für unser rohstoffarmes Land veränderungen und die
.. verheerenden Folgen

gesehen, sondern aufgrund von Ängsten be- oder einer Dekarbonisierung,

verhindert. Die AfD stellt sich gegen diese Entwicklung

und versucht, in Gesellschaft und Parlamenten eine

positive Zukunftserwartung zu vermitteln. Nicht

Verzicht, Rückschritt und Sozialismus haben die

176

Gesicherte und günstige
Energieversorgung Ist
eine der Grund-
voraussetzungen von
Wohlstand, Sicherheit
und Gesundheit.

Die AfD setzt sich dafür
ein, den Heraus-
forderungen der
Gegenwart
technologieoffen zu
begegnen.

Klima, Energie, Technik und Digitalisierung

Umweltbelastungen seit Jahrzehnten reduziert, sondern
technischer Fortschritt und eine auf klaren Regeln
basierende Marktwirtschaft.

Techniken müssen dem Wohl der Bürger, der Wirtschaft
und auch unserer Heimat, d.h. der Umwelt, zugute
kommen. Aufgabe der Politik ist es, Rahmenbedingun-
gen auf der Grundlage des aktuellen Standes von
Wissenschaft und Technik zu formulieren. Sicherheit
und Wirtschaftlichkeit jeder Technik sind durch die
Anwender sicherzustellen.

Darum fordert die AfD, den Herausforderungen der
Gegenwart technologieoffen zu begegnen.

Gesicherte und günstige Energieversorgung ist eine der
Grundvoraussetzungen von Wohlstand, Sicherheit und
Gesundheit. Diese sicherzustellen ist Kernaufgabe des
Staates. Energiepolitik bedeutet, Rahmenbedingungen
so zu setzen, dass die Energieversorgung gesichert, um-
weltfreundlich und kostengünstig im Rahmen des freien
Marktes gewährleistet ist. Teure und knappe Energie
lässt Industrien abwandern und schränkt den Lebens-
standard breiter Teile der Bevölkerung, gerade auch der
unteren Einkommensgruppen, drastisch ein.

Energie - Nein zum „Green Deal“

Wir lehnen den „Green Deal’ der EU sowie jegliche
weitere Formen von Planwirtschaft ab.

Wir wollen das EEG ersatzlos streichen und die Vorrang-
einspeisung beenden. Die sichere Energieversorgung ei-
nes Industrielandes erfordert einen breiten Energiemix.

Klima, Energie, Technik und Digitalisierung 177

Eine komplette Umstellung unserer Energieversorgung
aufvolatile „erneuerbare Energielieferanten ist unöko-
logisch unrealistisch und daher abzulehnen.

Die AfD lehnt das neue GEG (Gebäudeenergiegesetz) ab,
denn es führt zu weiteren Mietsteigerungen ohne ange-
messene Ökologische Verbesserungen.

In Wäldern und Schutzgebieten dürfen keine Wind- und
Solaranlagen errichtet werden. Wir setzen uns für die
Bewahrung des heimatlichen Landschaftsbildes, unserer
Umwelt und Tierwelt ein.

Vorrangflächen für Windenergieanlagen sollen nur
bei breiter Zustimmung der betroffenen Bürger —

Wir lehnen den „Green
ausgewiesen werden. Deal‘ der EU sowie

jegliche weitere Formen

. . . von Planwirtschaft ab.
Der Mindestabstand von Windenergieanlagen zur

Wohnbebauung muss das 10-fache der Gesamthöhe,
mindestens jedoch 2,5 km, betragen.

Anlagen, die nach 20 Jahren aus der Förderung gefallen
sind, sollen ohne Subventionen und Einspeisevorrang
am Markt teilnehmen. Ein Rückbau von Windenergie-
anlagen hat vollständig zu erfolgen, inklusive der
Betonfundamente.

Sogenannte Energiepflanzen wollen wir nicht mehr
fördern und Ackerflächen stattdessen wieder für den
Nahrungsmittelanbau nutzen. Alternativ können diese
Flächen der Natur zurückgegeben werden.

178

Die AfD setzt sich für
die Verstromung von
Braun- und Steinkohle
als grundlast- und
regelfähige Energie-
quelle ein.

Klima, Energie, Technik und Digitalisierung

Die Bevorzugung und Förderung einer WasserstofT-
wirtschaft lehnen wir ebenso ab wie die einseitige
Förderung der Elektromobilität. Technikoffenheit gilt
auch für die Nutzung von Wasserstoff und Elektromo-
bilität. Beide erfordern ein Ökologisch, energetisch und
ökonomisch unsinniges paralleles Infrastruktursystem.
Aufgrund hoher und günstiger Verfügbarkeit aus
Erdgas besteht kein Bedarf an durch volatile
„erneuerbare Energieträger erzeugtem Wasserstoff,
eine grofstechnische Herstellung daraus ist bisher gar
nicht möglich.

Für die Sicherstellung der Energieversorgung benötigen
wir weitere flexible Gaskraftwerke. Dazu ist die Gasver-
sorgung durch die Fertigstellung des Projektes Nord
Stream 2 zu sichern. Auch die Anlandung von Flüssiggas
muss ermöglicht werden.

Die AfD setzt sich für die Verstromung von Braun- und
Steinkohle als grundlast- und regelfähige Energiequelle
ein. Deutschland hat die weltweit saubersten und
effizientesten Kohlekraftwerke. Daher lehnen wir die
Ausstiegspläne aus der Kohleverstromung ab.

Die Laufzeit der in Deutschland noch verbliebenen sechs
aktiven Kernkraftwerke muss sich nach der technischen
Nutzungsdauer und ökonomischen Kriterien richten.
Eine Abschaltung aus anderen Gründen lehnen wir ab.
Deutsche Kernkraftwerke gehören zu den sichersten der
Welt.

Verwertbare Kernkraftwerk-Reststoffe sollten für eine
zukünftige Nutzung rückholbar gelagert werden
(„geschlossener Brennstoffkreislauf‘).

Klima, Energie, Technik und Digitalisierung 179

Da mit Gorleben ein vollständig erforschtes und unein-
geschränkt geeignetes Endlager zur Verfügung steht, ist
die Suche nach einem anderen geologischen Endlager
einzustellen. Stattdessen soll die Erforschung und Nutz-
barmachung von Technologien forciert werden, die eine
Lagerung von hochradioaktiven Reststoffen in
geologischen Zeiträumen nicht mehr erforderlich
machen. Die Endlagerfrage wird von Kernkraftgegnern
als politisch aufgebauschtes Problem ins Feld geführt,
um die friedliche Nutzung der minimalinvasiven und
emissionsarmen Kernenergie auf ewig zu verhindern. —

Die AfD tritt ein für die
Neueinrichtung von si-

Die AfD fordert die Wiedereinrichtung von Kern- cheren Kernkraftwerken,
f h d £ icheB ; um Energieknappheit
orschungszentren und umfangreiche Beteiligung an für den Industriestand-

internationalen Kernforschungsprojekten zur Sicher- ort Deutschland zu
stellung des notwendigen Wissens zum Betrieb und vermeiden

Bau von Kernreaktoren, zur Erforschung fortschrittli-

cher Reaktorkonzepte der Generation III+ und IV sowie

von Brutreaktoren und Fusionsenergie. Die AfD trittein

für die Neueinrichtung von sicheren Kernkraft-

werken, um Energieknappheit für den Industriestandort

Deutschland und seine Bürger zu vermeiden.

Im Bereich der privatwirtschaftlichen Luft- und Raum-
fahrt setzt sich die AfD für Wettbewerb ein und will den
Rahmen für federführende Unternehmen und eine
europäische Zulieferindustrie schaffen, um konkurrenz-
fähig zu bleiben.

Wir unterstützen die Fortsetzung und den Ausbau
nationaler und internationaler Raumfahrtprogramme
zur weiteren Erschliefsung des Weltraums für die
Wissenschaft, als Rohstoffquelle und als möglicher
neuer Lebensraum für den Menschen.

130

Als freiheitliche und
demokratische Partei
wendet sich die AfD
gegen jeglichen
Missbrauch digitaler
Techniken, die zur
Überwachung oder
Bevormundung

von Bürgern und Unter-

nehmen führen.

Klima, Energie, Technik und Digitalisierung

Wir wollen eine Aufklärungsinitiative über den heute
schon sichtbaren Nutzen der Gentechnik ins Leben
rufen und einen pragmatischen Ordnungsrahmen für
diese Technik schaffen. Generelle Verbote von Kunst-
stoffen lehnen wir ab und setzten uns für deren intelli-
gente Nutzung und verstärkte Wiederverwertung ein.

Die AfD setzt sich für die Schaffung von Kompetenz-
zentren zur Erforschung von Abfallbeseitigungs-und
Rohstoffrückgewinnungstechniken ein. Deutschland
ist bei der umweltgerechten Beseitigung von Abfällen
und Wiedergewinnung von Rohstoffen gut aufgestellt,
entsprechendes Know-How ist weltweit gefragt. Dies gilt
auch für Kunststoffe, da der überwiegende Teil zur
Energieerzeugung wiedergewonnen oder dem Stoff-
kreislauf wieder zugeführt wird. International muss
jedoch das Problem der Verschmutzung der Meere vor
allem durch die Verringerung des Eintrages in den Ent-
wicklungsländern gelöst werden. Mit wettbewerbsfähi-
gen Wiederaufbereitungstechnologien könnte
Deutschland hier einen wichtigen Beitrag leisten.

Digitalisierung muss uns Menschen dienen

In der Digitalisierung von Wirtschaft und Gesellschaft
sehen wir eine Chance für die Verbesserung des Lebens
aller Bürger. Als freiheitliche und demokratische Partei
wendet sich die AfD jedoch gegen jeglichen Missbrauch
digitaler Techniken, die zur Überwachung oder Bevor-
mundung von Bürgern und Unternehmen führen.

Wir fordern auch hier die Einhaltung des Grundgesetzes
und aller rechtsstaatlichen Prinzipien bei der Erstellung
und Nutzung von digitalen Diensten. Grundrechte und
Meinungsfreiheit gelten auch im Internet.

Klima, Energie, Technik und Digitalisierung

Im Einzelnen verlangen wir:

» Verzichtauf Anwendung von Digitalisierung, die
totalitäre Strukturen befördert.

» Abschaffung des NetzDG.

» Verhinderung von Upload-Filtern, denn sie
beschneiden die Meinungsfreiheit und
führen zu Zensur.

Defizite im EU-Urheberrecht sind entsprechend
zu korrigieren. Die AfD steht für ein freies und
offenes Internet.

» Einhaltung rechtsstaatlicher Prinzipien auch im
digitalen Bereich (Internet, soziale Medien und
Plattformen).

» Verpflichtung der Anbieter von sozialen Plattformen,
einen diskriminierungsfreien Zugang zu
ermöglichen und inhaltliche Neutralität zu
wahren. Entscheidungen über die Rechtmäfsigkeit
von Inhalten auf Plattformen sollen regelmäßig nicht
mehr beim Betreiber liegen. Die Justiz allein
hat die Befugnis und die Fähigkeiten, über die
Zulässigkeit von Inhalten zu entscheiden.

» Eine zentrale Meldestelle, an die sich von Rechts-
verletzungen auf Plattformen betroffene Bürger und
Einrichtungen wenden können. AufLänderebene soll
es Schwerpunkt-Justizzentren für die Rechts-
dAurchsetzung im Netz geben.

181

Die AfD steht für ein
freies und offenes
Internet. Wir fordern die
Einhaltung rechtsstaat-
licher Prinzipien auch
im digitalen Bereich.

132 Klima, Energie, Technik und Digitalisierung

Datenschutz und Datensicherheit

Wir stehen für Datenschutz und Datensicherheit ohne

ausufernde Bürokratie:

» Die AfD fordert die Abschaffung der DSGVO und
seine Ersetzung durch ein neues, schlankes Daten
schutzgesetz zur Wahrung informationeller Selbst-
bestimmung. Einwilligungen zur Datenverarbeitung
müssen jederzeit und wirksam widerrufbar sein.

» Datenschutzbehörden will die AfD in ihrer Rolle
bestärken und auch gegenüber staatlichen Stellen
sanktionsfähig machen, da der Staat dieselben
Datenschutzregeln wie der Bürger einzuhalten hat.

» Die Ende-zu-Ende-Verschlüsselung in der

— elektronischen Kommunikation sowie Datenschutz

Die AfD fordert die durch Technikgestaltung und Voreinstellungen

Abschaffung der DSGVO . \ .

und seine Ersetzung („by design“ und „by default“) müssen gesetzlich als

durch ein neues, schlan- Standard verankert werden.

NE > Öffentliche Ausgaben für die Erforschung und
Anwendung von Quanten-Kryptographie müssen aus
Gründen der nationalen Sicherheit, der Vermeidung
von Wirtschaftsspionage und für die IT-Sicherheit der
Bürger umgehend und deutlich aufgestockt werden.

» Ein zentrales, bereichsübergreifendes Personen
kennzeichen bei der Modernisierung der
Verwaltungsregister von Bund und Ländern lehnen
wir ab, daes die Erstellung von Persönlichkeits-
profilen zulässt und damit verfassungswidrig ist.

» Die zahlreichen IT-Sicherheitsgesetze und
-strategien in Deutschland müssen umgehend
aktualisiert und konsolidiert werden, da die hohe
Komplexität ihres Zusammenwirkens IT-Sicherheit
eher gefährdet, statt die zu fördern.

» Behörden dürfen keine Software-Hintertüren nutzen,
Sicherheitslücken sind sofort zu melden und
konsequent zu schliefsen.

Klima, Energie, Technik und Digitalisierung 183

Auch kommunale kritische Infrastrukturen und
Einrichtungen müssen besser vor Hacker-Angriffen
geschützt werden.

Wir wollen einen einheitlichen Rechtsrahmen zur
Standardisierung und Zertifizierung von
IT-Infrastrukturen, -Produkten, -Dienstleistungen
und -Systemen.

Das Bundesamt für Sicherheit in der Informations
technik (BSI) soll zu einer Behörde des digitalen
Verbraucherschutzes ausgebaut werden.

Behörden der polizeilichen und nichtpolizeilichen
Gefahrenabwehr müssen unverzüglich auf den Stand
der digitalen Technik gebracht und ihre Verfahrens-
weisen entsprechend angepasst werden. Instrumente
an der Schnittstelle zum Bürger, wie z.B. die NINA-
Warn-App, müssen nicht nur mit den notwendigen
finanziellen Ressourcen ausgestattet, sondern auch
anwenderfreundlich ausgestaltet werden.

Infrastruktur ist die Grundlage für den weiteren
Ausbau der Digitalisierung

>

Die AfD fordert die Förderung regionaler Strukturen —

. Die AfD fordert die
auch beim Glasfaser- und Funknetzausbau nach dem Förderung regionaler

Vorbild des „nationalen Roamings‘. Der Netzübergang Strukturen auch beim
zwischen Betreibern muss für Nutzer unbemerkt und a nn
kostenneutral erfolgen.

Der 5G-Netzausbau muss bei künftiger Nutzung

höherer Frequenzen im Hinblick auf gesundheitliche

Risiken auch weiterhin durch laufende wissenschaft

liche Untersuchungen begleitet und die Bürger

müssen über deren Ergebnisse umfassend informiert

werden.

184 Klima, Energie, Technik und Digitalisierung

Digitalisierung der öffentlichen Verwaltung

Die Digitalisierung der öffentlichen Verwaltung hat
sowohl intern als auch in der Interaktion mit den
Bürgern noch viel Potential. Die AfD fordert die bessere
finanzielle und beratende Unterstützung von
Kommunen bei OZG (OnlineZugangsGesetz) - Projekten,
zu denen sie vom Bund verpflichtet wurden.

» Die AfD fordert, digitale Verwaltungsprozesse zu
verschlanken und zu vereinheitlichen.
Das E-Government ist weiter auszubauen, dem
Bürger aber eine gleichwertige, niederschwellige,
persönliche Kontaktaufnahme zu ermöglichen.
Grundsätzlich soll jeder Behördengang von Bürgern
in Zukunft nach seiner Wahl digital oder persönlich
abgewickelt werden können. Bezahlung von
Gebühren jeder Art muss mit Bargeld erfolgen
können.
» Die AfD fordert den bevorzugten Einsatz (ggf. zu
. entwickelnder) quelloffener Software für digitale
Die AfD fordert, digitale Verwaltungsprozesse. Dies gilt auch für den Betrieb
Verwaltungsprozesse von Servern und Arbeitsplatzrechnern.

zu verschlanken und zu
vereinheitlichen.

Klima, Energie, Technik und Digitalisierung 185

Digitale Souveränität

Zur Verbesserung der Sicherheit digitaler Verwaltungs-
systeme fordert die AfD die Bündelung europäischer
IT-Kompetenzen im Rahmen von Forschungs- und
Entwicklungskooperationen mit dem Ziel, europäische
Hard-& Systemsoftware zu entwickeln, um von
Lösungen aufsereuropäischer Anbieter unabhängiger
zu werden.

Die AfD will Behörden verpflichten, das Beschaffungs-
wesen an langfristigen strategischen und sicherheits-
politischen Erwägungen auszurichten.

Künstliche Intelligenz ist eine Technik der Zukunft mit
einem hohen wirtschaftlichem Potential; die
Konkurrenz durch multinationale Konzerne in diesem
Bereich ist jedoch grofs. Daher wollen wir die
Kompetenzen im Bereich Kl in der Bundesrepublik
besser bündeln und nationale Kooperationen stärker
fördern.

KAPITEL 20

Mobilität und
Infrastruktur

138

Die AfD spricht sich für
eineanden
Bedürfnissen der Bürger
orientierte
Verkehrspolitik aus.

Eine ideologisch
geleitete Verbotspolitik,
die bestimmte Verkehrs-
mittel bevorzugt oder
diskriminiert, lehnt die
AfD ab.

Mobilität und Infrastruktur

Mobilität und Infrastruktur

Verkehrspolitik

Die AfD spricht sich für eine an den Bedürfnissen der
Bürger orientierte Verkehrspolitik aus. Eine ideologisch
geleitete Verbotspolitik, die bestimmte Verkehrs-

mittel bevorzugt oder diskriminiert, lehnt die AfD ab. Im
Vordergrund steht für uns die Freiheit der Bürger in der
Wahl des Verkehrsmittels. Individuelle Mobilität muss
bezahlbar bleiben.

Motorisierten Individualverkehr schützen

Die AfD unterstützt und fördert den motorisierten Indi-
vidualverkehr als beliebteste Möglichkeit der
Fortbewegung. Intelligente Technik und stauvermei-
dende Verkehrsführung schützen die Umwelt. Wissen-
schaftlich mangelhaft belegte Mafßnahmen, wie z.B.
Dieselfahrverbote oder Umweltspuren, erreichen das
nicht. Gleichzeitig fordert die AfD den Erhalt und Ausbau
voninnerstädtischen Fahrspuren und Parkraum zur Ver-
meidung von wachsendem Verkehrschaos sowie zum
Schutz der Attraktivität des Einzelhandels.

Die AfD lehnt zudem Kollektivrmafsnahmen wie
Fahrverbote für Motorräder aufgrund des Fehlverhaltens
Einzelner ab. Grundsätzlich setzt die AfD auf das
Verantwortungsbewusstsein der Bürger.

MobilitätundInfrastruktur 189

Bundesautobahnen entlasten - Transitverkehr auf
Schiene und Wasserwege verlagern

Die AfD lehnt ein generelles Tempolimit auf Bundes-

autobahnen strikt ab. Starre Tempolimits müssen regel-

mäfsig überprüft werden und im Fall der Unbegründet-

heit wegfallen. Temporaär sind flexible, situations-

gerechte Streckenbeeinflussungsanlagen die Alternative.

Viele Autobahnen und Brücken in Deutschland müs-

sen seit langem dringend saniert und instand gesetzt

werden. Das geht nur mit effizienten Mafßnahmen in

der Planung und Ausführung von Baustellen und mit

schneller Arbeit der Verwaltung. Zudem fordert die AfD

mehr Lkw-Parkplätze sowie Konsequente Anwendung —
bestehender Lkw-Überholverbote zur Sicherheit aller ln
Verkehrsteilnehmer. Das Konzept der „RollendenLand- _ aufBundesautobahnen
straße” und des Wechselbrückenumschlages soll fürdie «ab
Transitverbindungen durch Deutschland mit

regelmäfsigen Taktungen etabliert werden. Das Netz an

Verladestellen und multimodalen Güterverkehrszentren

(Straße, Schiene und Wasserwege) ist zu verdichten.

Sichererer, sauberer und verlässlicher
Schienenverkehr

Für den Schienenpersonenverkehr muss Deutschland
ein besser ausgebautes und abgestimmtes Öffentliches
Nah- und Fernverkehrsnetz erhalten, das sich am ein-
fachen und zuverlässigen Modell der Schweiz orientiert.
Pünktlichkeit, Sicherheit und Sauberkeit sowie eine
optimale Taktung von Bus-, Bahn- und Vernetzung mit
dem Flugverkehr müssen gewährleistet werden.

190

Die AfD befürwortet
die Abschaffung der
Luftverkehrsteuer und
richtet sich gegen
unrealistische
CO,-Reduktionsziele
derEU.

Mobilität und Infrastruktur

Die AfD unterstützt den Ausbau des zu lange
vernachlässigten Schienennetzes - insbesondere

die Beseitigung von Engpässen und die Erweiterung
des Hochgeschwindigkeitsnetzes. Beim angestrebten
Deutschlandtakt muss eine Verkürzung der Reisezeiten
auf allen Verbindungen das oberste Ziel sein.

Flugverkehr als Wirtschaftsfaktor stärken

Der globale Flugverkehr ist von elementarer Bedeutung
für den Wirtschaftsstandort Deutschland und darf nicht
kurzsichtig einer unwissenschaftlichen Klima-Hysterie
geopfert werden. Deutschlands Flughäfen sind als
Wirtschaftsfaktor zu stärken. Künftiges Wachstum soll
unter innovativer Einbeziehung der umliegenden und
schwächer ausgelasteten Verkehrsflughäfen auch inner-
halb des Sicherheitsbereichs (Gate-to-Gate) geschehen.
Die AfD befürwortet die Abschaffung der Luftverkehr-
steuer und richtet sich gegen unrealistische CO2-
Reduktionsziele derEU.

Im Rahmen eines „Luftverkehrskonzepts 2021 sind
Deutschlands Flughäfen als Wirtschaftsfaktor zu stärken.

Die deutsche Automobilindustrie als
Leitindustrie erhalten

Die AfD sieht die strategische Bedeutung der
deutschen Automobil- und Zulieferindustrie.

Gerade der für den Grofßsteil der Arbeitsplätze
zuständige Mittelstand, insbesondere der metall-
verarbeitenden Industrie, ist abhängig vom Fortbestand
des Verbrennungsmotors. Es ist daher die Aufgabe der
Bundesregierung, auf nationaler und europäischer

Mobilitätund Infrastruktur 191

Ebene für eine technologieoffene Gesetzgebung zu
sorgen. Die heutige einseitige Bevorzugung von
Elektromobilität ist auf Grund mangelnder Strom-
kapazitäten und der globalen Umweltbelastung bei der
Batterieproduktion sofort zu stoppen. Ob der
Verbrennungsmotor eines Tages durch andere
Antriebstormen abgelöst wird oder weiter existiert,
muss eine Frage des technischen Fortschritts sowie des

Marktes werden und darf nicht auf der Verbotspolitik Die AfD sieht die stra-
der EU basieren. tegische Bedeutung der
eu deutschen Automobil-
Wenn die bestehende, realitätsfremde EU-CO,-
. j 2 und Zulieferindustrie.
Reduktionsgesetzgebung im Verkehrssektor nicht Es ist daher die Aufgabe

verhindert werden kann, sind synthetische Kraftstoffe  @ Pundesreglerung,
als zukunftsfähiger Treibstoff für Verbrennungsmotoren en für
zu berücksichtigen. Fahrzeuge mit Verbrennungsmotor, eine technologie-
welche mit synthetischem Kraftstoff angetrieben u nn
werden, müssen hinsichtlich Ihrer Abgabenlast gegen-

über elektrisch angetriebenen Fahrzeugen eine Gleich-

behandlung erfahren. Dies gilt sowohl für den Energie-

träger selbst als auch für das Fahrzeug insgesamt.

Stadt und Land gleichrangig entwickeln

Seit Jahrzehnten prägen Landflucht und Verstädterung
das Bild unseres Landes. Es braucht einen grundlegen-
den Politikwechsel, um diese Entwicklung umzukehren.
Wir wollen gleichwertige Lebens- und Arbeits-
bedingungen in ganz Deutschland, die Wahrung und
zukunftsfähige Weiterentwicklung regionaler Identität
in allen Teilräumen des Landes. Die Lebensweisen und
Werte von Stadt und Land sind gleichrangig und
ergänzen sich.

192

Mit einer traditionsbe-
zogenen,
familienfreundlichen
und selbst-

bewussten Gestaltung
des ländlichen Lebens-

raumes wollen wir unse-

re Heimat wiederins
Gleichgewicht bringen.

Mobilität und Infrastruktur

Mit einer traditionsbezogenen, familienfreundlichen
und selbstbewussten Gestaltung des ländlichen Lebens-
raumes wollen wir unsere Heimat wieder ins Gleich-
gewicht bringen. Zur Sicherung der kommunalen Eigen-
verantwortung werden wir das Finanzvolumen und die
Handlungsspielräume der Gemeinden deutlich
vergröfsern.

Entscheidungen näher zum Bürger bringen

Die Regionen und Landkreise wollen wir durch
Wirtschaftsförderung, Bewahrung regionaler Kultur,
Daseinsvorsorge, Verkehr und Digitalisierung stark
machen. Die EU, Bund und Land sollen weniger
regulierend in die kommunale Selbstverwaltung und
die Gestaltungsmöglichkeiten der Regionen eingreifen.

Landschaft als Teil regionaler Identität

Bei Umwelt- und Landschaftsschutz wollen wir eine
starke regionale Verankerung. Für die heimatliche Land-
schaft wollen wir die Privilegierung von energieerzeu-
genden Anlagen ausschließen. Die Weiterentwicklung
bestehender Siedlungstlächen ist zu ermöglichen. Bei
der notwendigen Inanspruchnahme von Flächen setzen
wir aufschonende Landnutzung in Eigenverantwortung
der Regionen und Kommunen.

Unser Leitbild ist die polyzentrische Stadt und der
lebenswerte, fufsläufige, familienfreundliche Stadtteil.
Wir stehen für den Erhalt des kulturellen Erbes,
ortsübliches Bauen und sorgfältige Rekonstruktion
von historischen Ortskernen und Einzelgebäuden.

Mobilität undInfrastruktur 193

Grofßsstädte und ihr ländliches Umland sind als Einheit
zu denken. Wir wollen sie entlang der Verkehrsachsen
räumlich weiterentwickeln, die Siedlungsentwicklung im
Umland fördern und die Verdichtungsräume entlasten.

Die Kommunen müssen

Infrastruktur bei der Erfüllung
ihrer Pflichtaufgabe

Soziale und technische Infrastrukturen sind die Grund- zur Daseinsvorsorge
lage unseres heutigen Zusammenlebens. Obwohl wir 1 un |
sie als selbstverständlich wahrnehmen, steigt für die länderübergreifend in
Kommunen der Investitionsstau für die bedarfsgerechte "anungen eingebunden
Bereitstellung immer weiter an. [_

Die AfD ist sich dessen bewusst, dass intakte und

flächendeckend ausgebaute Infrastrukturen die

Voraussetzung für eine prosperierende Wirtschaft sind

und ein sehr wichtiges Mittel gegen den fort-

schreitenden Bevölkerungsrückgang auf dem

Land darstellen.

Kommunen stärken -
Erfüllung der Pflichtaufgaben sicherstellen

Die Kommunen müssen bei der Erfüllung ihrer Pflicht-
aufgabe zur Daseinsvorsorge finanziell wesentlich
besser ausgestattet und länderübergreifend in
Planungen eingebunden werden. Daseinsvorsorge ist als
hoheitliche Aufgabe von Bund, Ländern und Kommu-
nen zu betrachten und soll daher in öffentlicher Hand
bleiben.

194

Die AfD fordert, den
Ausbau der digitalen
Infrastruktur durch eine
bessere und flächende-
ckende Koordination zu
beschleunigen.

Mobilität und Infrastruktur

Ausbau digitaler Infrastrukturen beschleunigen

Aus Sicht der AfD gehört hierzu auch die flächen-
deckende, anbieteroffene Bereitstellung von Glasfaser-
anschlüssen und eines leistungsfähigen, flächen-
deckenden Mobilfunknetzes. Die derzeitige Praxis der
Mobilfunk-Frequenzbandversteigerung kann zum
Hindernis für den flächendeckenden Ausbau und
Preistreiber für den Endkunden werden und muss
deshalb grundlegend überdacht werden.

Der derzeitige Ausbau des Glasfasernetzes in
Deutschland ist von Insellösungen und „Weifsen Flecken
in den ländlichen Gebieten und Parallelstrukturen in den
Ballungsräumen geprägt. Eine bessere und flächende-
ckende Koordination ist hier zwingend erforderlich.

Nord Stream 2 endlich fertigstellen

Die AfD unterstützt den Ausbau der Nord Stream? -
Pipeline. Durch deren Anbindung an Deutschland kann
unseren Bürgern ein kostengünstiger Primärenergie-
träger zur Verfügung gestellt werden.

195

KAPITEL 21

Landwirtschaft,
Umwelt- und
Verbraucher-
schutz
Er

198

Reine Luft und sauberes
Wasser sowie
naturnahe Lebens-
räume sind lebens-
wichtige Allgemein-
güter, deren Schutz zu
den Prioritäten der AfD
zählt.

Landwirtschaft, Umwelt- und Verbraucherschutz

Landwirtschaft, Umwelt- und
Verbraucherschutz

Reine Luft und sauberes Wasser, gesunde und verant-
wortungsbewusst erzeugte Lebensmittel sowie natur-
nahe Lebensräume sind lebenswichtige Allgemeingüter,
deren Schutz zu den Prioritäten der AfD zählt. Unsere
Politik orientiert sich dabei an den Zielen der Nach-
haltigkeit und Versorgungssicherheit als konservative
Prinzipien.

Bei Verbrauchern wie Erzeugern wollen wir eigenver-
antwortliches Handeln fördern. Falls möglich, sollten die
Verbraucher auf regional erzeugte Lebensmittel zurück-
greifen können. Dabei kommt es uns nicht darauf an, ob
sie aus biologischer oder konventioneller

Produktion kommen, sondern entscheidend ist die
Qualität der Produkte.

Die heimische Land- und Forstwirtschaft ist unverzicht-
bar für die Versorgung mit wichtigen Allgemeingütern.
Wir wollen mehr Selbstversorgung in Deutschland mit
heimischen Nahrungsmitteln möglich machen. Die fort-
schreitende Entmündigung von Landwirten, Jägern und
Verbrauchern durch immer mehr europäische
Regulierungen muss ein Ende haben. Entscheidungen
müssen unter Einbeziehung aller Beteiligten auf der
Grundlage rationaler Faktenanalysen statt ideologischen
Wunschdenkens getroffen werden.

Landwirtschaft, Umwelt- und Verbraucherschutz 199

EU-Agrarpolitik renationalisieren

Die Gemeinsame Agrarpolitik (GAP) der EU hat

bisher mehr Schaden als Nutzen gebracht. Wir wollen
die Zuständigkeit für die Agrarpolitik deshalb wieder in
die alleinige Verantwortung der Nationalstaaten legen.

Deutschland ist der gröfste Nettoeinzahler in den

EU-Haushalt. Landwirte anderer Staaten sollen nicht mit

deutschem Steuergeld subventioniert werden. Um bis

zur Renationalisierung gleiche Wettbewerbsbedingun-

gen herzustellen, die Landschaftspflege zu belohnen und

die extensive Nutztierhaltung zu fördern, setzt sich die

AfD für die Einführung einer bundesweiten Weidetier-

prämie ein. Darüber hinaus müssen Agrarumweltmaß-

nahmen derart gestaltet werden, dass sie lebensfähige Wir wollen die
ländliche Strukturen in unserer Heimat erhalten und Zuständigkeit für die

. . . Agrarpolitik wieder in
den agrartechnischen Fortschritt sichern. die alleinige

Verantwortung der

.. .. Nationalstaaten legen.
Grüne Berufe wertschätzen 2

Die verschiedenen Sparten der Flächen-
bewirtschaftung - Ackerbau, Viehzucht, Weinbau,
Forstwirtschaft, Gartenbau und Imkerei - tragen zur
Pflege der Kulturlandschaft, der Schaffung regionaler
Arbeitsplätze und zur Versorgung der Bevölkerung mit
hochwertigen Lebensmitteln in unserem Land bei.

Wir setzen uns ein für eine realitätsbezogene Bildung
zum Thema „Grüne Berufe‘ und die sich daraus
ergebende Wertschätzung für Leistungen und Produkte
der Landwirte. Bevor agrarpolitische Mafsnahmen
eingeführt werden, muss geprüft werden, wie sie sich
auf das Einkommen der Bauern auswirken und welchen
ökologischen Nutzen sie erbringen.

200

Die AfD fordert die
Revision der Dünge-
verordnung und die
Aufhebung der
pauschalen Reduktion
der Düngemengen.

Landwirtschaft, Umwelt- und Verbraucherschutz

Familienbetriebe in den Mittelpunkt stellen

Die Agrarförderung muss die Leistungen kleinerer und
mittlerer Familienbetriebe berücksichtigen und die Hof-
nachfolge vereinfachen. Die Landwirtschaft ist zu entbü-
rokratisieren, um die Bauern zu entlasten.

Weder deutsche noch EU-Behörden sollen sich in deren
Wirtschaftsweise einmischen, etwa bei der Wahl der
Fruchtfolge. Um unsere vielfältigen Agrarstrukturen zu
erhalten, wollen wir Junglandwirte besserstellen, die ihre
Agrarflächen selbst bewirtschaften. Wir setzen uns dafür
ein, den Aufkauf von Flächen durch landwirtschafts-
fremde Investoren zu erschweren. Landwirtschaftliche
Existenzgründer wollen wir von der Grunderwerbsteuer
befreien.

Düngung am Bedarf der Pflanze orientieren

Eine ausreichende Versorgung von Kulturpflanzen mit
notwendigen Düngemitteln ist für das Pflanzenwachs-
tum, die Bodenfruchtbarkeit und Ernährungssicherheit
unverzichtbar. Die verschärfte neue Düngeverordnung
widerspricht jedoch der guten landwirtschaftlichen
Praxis und gefährdet den Anbau von Kulturpflanzen mit
hohem Nitratbedart, wie z. B. Kartoffeln, Getreide und
Mais. Die AfD fordert die Revision der Düngeverordnung,
die Aufhebung der pauschalen Reduktion der Dünge-
mengen und die präzise Ermittlung von Eintragsquellen
nach dem Verursacherprinzip unter Beachtung von
Kostenwirksamkeit und Verhältnismäßigkeit.

Landwirtschaft, Umwelt- und Verbraucherschutz 201

Praxiskonformen Pflanzenschutz etablieren

Ein wirksamer Pflanzenschutz nach guter fachlicher

Praxis ist die Grundlage jeglicher landwirtschaftlichen

Produktion. Für unsere Landwirte muss dabei die

Zulassung, die Nutzung und der Handel von alten

Kultursorten möglich sein, um die Sortenvielfalt für den

Verbraucher zu erhöhen. Die bewährten Zulassungs- Die afD steht für eine

standards im Hinblick auf Umwelt-, Gesundheits- und konsequente
Umsetzung der

Verbraucherschutz dürfen dabei weder vernachlässigt Tierschutzgesetze im
noch praxisfern verschärft werden. Die fachliche und Sinne unserer

. . oo: Verantwortung für
wissenschaftliche Nutzen- und Risikobewertung von

. . j Tiere als fühlende
Wirkstoffen sowie der Umgang mit Giftpflanzen und Mitgeschöpfe.

Gentechnik müssen unabhängige Forschungs-
einrichtungen übernehmen. Das Bundesinstitut für
Risikobewertung muss für diese Aufgabe strukturell
und personell besser ausgestattet werden.

Tierwohl, Haltung und Transport weiterentwickeln

Die AfD steht für eine konsequente Umsetzung der Tier-
schutzgesetze im Sinne unserer Verantwortung für Tiere
als fühlende Mitgeschöpfe. Wir wollen in der Nutztier-
haltung artgerechte Haltungssysteme fördern, die an die
Tierart angepasst sind und verlangen die Kopplung der
Tierhaltung an die Fläche. Dazu gehört auch, dass Lager-
und Stallanlagen, die den Umwelt- und Tierschutz ver-
bessern, innerhalb von drei Monaten und mit weniger
Auflagen genehmigt werden müssen. Aufserdem

setzen wir uns für ein Netz regionaler Schlachtbetriebe
in Deutschland ein, um Tiertransporte so kurz wie
möglich zu gestalten.

202

Die AfD fordert den
Einstieg in eine
Bestandsregulierung
und die Ausweisung von
wolfsfreien Gebieten.

Landwirtschaft, Umwelt- und Verbraucherschutz

Die Dauer der Betäubung muss für den gesamten
Schlachtvorgang gewährleistet sein - Ausnahmen hier-
von sind unzulässig. Darüber hinaus fordert die AfD eine
Einschränkung der Exporte von lebenden Schlachttieren
in Länder mit bekannten Verstößen gegen den Tier-
schutz.

Tierschutzgesetze müssen für alle gleichermafsen
gelten. Deshalb lehnen wir auch den Handel, die Bewer-
bung und die Einfuhr von Fleisch aus tierquälerischer
Schlachtung (Schächtung) ab. Es ist unsere ethisch
gebotene Pflicht zum Wohl der Tiere, den Lebendtrans-
port von Schlachttieren zu reduzieren und in Drittländer,
wo nachweislich gegen das Tierschutzrecht verstoßen
wird, auszusetzen.

Schutz der heimischen Ökosysteme verbessern

Im Schlepptau des weltweiten Warenverkehrs dringen
entlang der grofsen Fernhandelsstrafsen und ausgehend
von internationalen Häfen mit ihren großen Waren-
umschlagplätzen immer mehr gebietsfremde Arten in
heimische Ökosysteme vor und stören das ökologische
Gleichgewicht. Es zeigt sich, dass gebietstremde Arten
nicht nur negative Auswirkungen auf unsere hiesige
Natur haben, sondern auch konkrete finanzielle und
gesundheitliche Schäden verursachen. Wir müssen
größere Anstrengungen unternehmen, um die
Einschleppung invasiver Tier- und Pflanzenarten zu
verhindern und ihre Ausbreitung einzudämmen.

Eine invasive Eigenschaft ist als Entnahmegrund im
Tierschutzgesetz zu verankern.

Landwirtschaft, Umwelt- und Verbraucherschutzt

Zeitgemäßes Wildtiermanagement umsetzen

Über die Kombination aus aktivem Wildtiermanage-
ment und artenschutzrechtlicher Ausnahmeverordnung
sollen problematische Wildtierbestandsgröfsen regional
und zeitnah überwacht und korrigiert werden können.
Das Ziel ist die Balance zwischen den Ansprüchen von
Wildtieren sowie den Interessen von Landwirten, Teich-
wirten und Weidetieren. Schäden durch Wildtiere sind
unbürokratisch und zeitnah durch Geldleistungen zu
ersetzen. Jedoch wird eine bedingungslose Wiederan-
siedlung des Wolfs von den Behörden unter Berufung
auf die Zielsetzung der EU-Richtlinien zum Natur- und
Artenschutz betrieben. Der Wolf ist ein Raubtier, das zu
Nutztierverlusten bei Landwirten führt. Die Unsicher-
heiten und Ängste der Menschen, speziell in ländlichen
Regionen, müssen ernst genommen werden. Weidende
Schafe, Ziegen, Rinder und Pferde sind besonders
betroffen. Unsere Fürsorgeptflicht gilt für unsere Weide-
tiere in gleichem Maße wie für den Wolf. Die AfD fordert
den Einstieg in eine Bestandsregulierung und die
Ausweisung von wolfsfreien Gebieten.

Unsere Wälder schützen und bewahren

Etwa ein Drittel der Gesamtfläche unseres Landes ist
bewaldet. Um einen gesunden Wald mit Wild zu
erhalten, braucht es waldbauliche Mafßnahmen im
Einklang mit Ökonomie, Ökologie und Naherholung.
Um eine nachhaltige Bewirtschaftung und Sicherung
der Wälder zu erreichen, setzt sich die AfD für eine
Förderung des Baumaterials und Energieträgers Holz
ein.

203

Der Wald soll von
Windindustrieanlagen
freigehalten werden, um
die wildlebenden

Arten zu schützen und
unseren Kindern

eine intakte Natur zu
hinterlassen.

204

Die Versorgung der

eigenen Bevölkerung so-

wie die Wirtschaftlich-
keit und Wertschöpfung
heimischer Lebensmit-
telproduktion muss
gewährleistet bleiben.

Landwirtschaft, Umwelt- und Verbraucherschutz

Wir stehen für einen Abbau der ausufernden

Bürokratie bei der Dokumentation in der Forstwirtschaft.
Der Wald soll von Windindustrieanlagen freigehalten
werden, um die wildlebenden Arten zu schützen und
unseren Kindern eine intakte Natur zu hinterlassen.

Gesunde Lebensmittel aus heimischer
Produktion fördern

Die AfD unterstützt das Leitbild des mündigen
Verbrauchers. Er soll in seinem Konsumverhalten nicht
staatlich bevormundet werden. Deshalb lehnen wir jede
Form der gesonderten Lebensmittelbesteuerung, wie
eine Fleisch- oder Zuckersteuer, ab. Dazu gehört die
Aufklärung über gesundheitliche Risiken modischer
Ernährungsformen und die Bewahrung unserer
traditionellen Esskultur in Öffentlichen Einrichtungen.
Wir schätzen regionale und saisonale Lebensmittel

aus Deutschland. Sie sichern unseren produzierenden
und verarbeitenden Betrieben ein hinreichendes Ein-
kommen - bei zugleich hohen Standards im Tier- und
Umweeltschutz. Die Versorgung der eigenen Bevölkerung
sowie die Wirtschaftlichkeit und Wertschöpfung
heimischer Lebensmittelproduktion muss gewährleistet
bleiben.

Transparenz im Verbraucherschutz ausbauen

Der Verbraucher muss in die Lage versetzt werden, eine
informierte und eigenverantwortliche Kaufentscheidung
treffen zu können. Produkte sollen deshalb nach
festgelegten Standards gekennzeichnet werden.

Die Auskunft über Inhaltsstoffe und Herkunft der
Produkte ist zu verbessern.

Landwirtschaft, Umwelt- und Verbraucherschutz 205

Auch bei Versicherungsverträgen, Finanzprodukten
und Mobilfunkverträgen wollen wir mehr Transparenz
schaffen. Herstellerangaben und Vertragsklauseln sind
in einer verständlichen Sprache zu verfassen.

Wasserversorgung in öffentlicher Hand belassen

Die AfD befürwortet die regionale Wasserversorgung

durch Kommunen und Zweckverbände. Das Eigen-

tum an den Wasserressourcen, den Förderanlagen, nee wei

der Wasseraufbereitung und am Leitungsnetz mussin Liberalisierung und

öffentlicher Hand und damit in den Händen der Bürger “malserung
des Wassermarkts

verbleiben. Eine EU-weite Liberalisierung und Kommer- jennt die app

zialisierung des Wassermarkts lehnt die AfD entschie- entschieden ab

den ab. Eine generelle Privatisierung des Wassers oder

der Wasserwerke, die durch Richtlinien der EU angeregt

werden, darf es nicht geben. Brunnen privater Grund-

stückseigentümer, die hieraus ihr Trinkwasser

gewinnen, sind hiervon ausgenommen.

Ausbau von erneuerbaren Energien einschränken

Der politisch-ideologisch forcierte und staatlich
subventionierte Ausbau regenerativer Energien ist
nicht nur ein ökonomisches Desaster, sondern belastet
auch unsere Heimat, unser Lebensgefühl und unsere
Natur in erheblichem Mafse. Wir lehnen die undurch-
sichtigen Verfahren ab, die den Ausbau sogenannter
erneuerbarer Energien über die Köpfe der Bürger
hinweg ermöglichen.

206

Die AfD will unsere
Landschaft erhalten
und lehnt deswegen den
Neubau von
Photovoltaikanlagen
aufgrund des hohen
Flächenverbrauchs, der
Entwertung landwirt-
schaftlicher
Produktionsflächen
und der Störung des
Mikroklimas ab.

Landwirtschaft, Umwelt- und Verbraucherschutz

Windenergieanlagen sind nur noch an Standorten
zuzulassen, an denen keine Beeinträchtigungen für
Mensch, Tier oder das Landschaftsbild zu erwarten sind.
Wald- und Schutzgebiete sind generell nicht anzutasten.
Als Mindestabstand zur Wohnbebauung soll das
10-fache der Gesamthöhe, mindestens jedoch 2,5 km,
bundesweit eingehalten werden.

Studien zeigen seit Jahren die verheerende Wirkung

von Windkraftanlagen auf Vögel, Fledermäuse und
Insekten. Gravierend sind auch die gesundheitlichen
Schadwirkungen durch Schattenschlag, Infraschall und
Lärmemissionen. Die massive Subventionierung der
Windindustrie zerstört nicht nur Kulturlandschaften und
Wälder, sondern mindert durch den Bau von Strom-
trassen auch die Grundstückswerte.

Den Neubau von Photovoltaikanlagen auf Freiflächen
lehnen wir aufgrund ihres hohen Flächenverbrauchs
sowie ihrer abschreckenden Wirkung für den Tourismus
ab. Zudem verstärken diese die Aufheizung des
Mikroklimas und entwerten landwirtschaftliche
Produktionsflächen. Photovoltaikanlagen führen
generell zu einer Veränderung des Mikroklimas und
verursachen eine Temperaturerhöhung.
Bestandsanlagen dürfen weiter betrieben werden.

207

208

Index

Abschiebung
Abtreibung
Antifa
Arbeitslosengeld
Arbeitsplätze
Asyl
Ausländer
Autobahnen
Bargeld
Blue-Deal
Bundespolizei
Bundeswehr
Burka

Cancel Culture
Cannabis
China

Corona
Digitalisierung

DITIB 85

Drogen

Eigentum
Einwanderung
Energie
Energieträger
Enteignung
Entwicklungshilfe
Erbschaftssteuer
EU

Euro

EZB
Familien
Flüchtlinge
Gefährder

Gemeinschaftsdienst

Gender

Gesundheitssystem

Grofsbritannien
Grundgesetz
Heimat
Hochschulen
Identität

Impfung
Individualverkehr
Integration
Investitionen
Islam
Islamisierung
Justiz
Kernenergie
Kirchenasyl

77,94, 95

109

76,77,.111
118,120, 121
43,122,190,199
89,90, 91,92, 93, 94, 95, 96, 97,98, 99,100, 101
80,118

189

56,184

43,44

78,79, 96

66, 67,96, 129

65

12, 25, 29, 42, 43, 50, 52, 134, 135, 136, 165

99, 149, 168, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
183, 184, 185, 192

142

169, 205

91,99

173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185
178,191

12, 13, 14, 21, 25, 27, 28, 29, 30, 34, 50, 51, 52, 54, 58, 65, 66,
69, 72, 81,92, 96, 98, 108, 118, 126, 142, 153, 176, 177,181,
190, 191, 192, 199, 200, 203, 205

17, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 68, 109, 129
50, 51, 52, 53, 54, 55, 56, 57, 58

104, 105, 106, 107, 108, 109, 110, 111, 112, 115, 129, 130, 169
90,98

77

68

154

134,137

65

13, 56, 57, 84, 85,108, 115, 135, 153, 159

94, 97,104, 176, 192, 199, 205

153,154

28, 66, 68, 93, 98, 99, 104, 114, 152, 158, 159, 160,

161, 191,192

134

188

89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99,100, 101,150, 152
36, 42, 44, 58, 65,126

33, 84, 85, 86, 87,158

14, 66

15,129, 181

179

96

Klima

Klimawandel
Kriminalität
Kultur
Landwirtschaft
Leitkultur
Linksextremismus
Linksterrorismus
Lobbyismus
Meinungsfreiheit
Migration

Mindestlohn
Mittelstand
Multikulturalismus
Muslimbruderschaft
NATO

Nord Stream 2

Parallelgesellschaften

Parteien

PCR-Test

Pflege
Photovoltaikanlagen
Polizei

Quoten

Rechtsstaat

Rente

Rundfunk
Russland

Scharia

Schulden
seenotrettung
Souveränität
Sozialhilfe
Sozialismus
Staatsangehörigkeit
Steuern
Subventionen
Terrorismus

The Great Reset
Türkei
Umverteilung
UN-Flüchtlingspakt
UN-Migrationspakt
USA
Verbraucherschutz
Verschuldung
Wehrpflicht
Whistleblower
WHO

Wirtschaft

Wissenschaft
Wohngeld

Index

29,126, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
183, 184, 185, 190, 191

174

76, 77,79, 80

22, 71,148, 152, 157, 158, 159, 160, 161, 164, 192
197,198, 199, 200, 201, 202, 203, 204, 205, 206
158

76

76

18, 19

164, 165,181

14, 63, 78, 79, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
100, 101,150, 168

119,120

65, 178,194

85, 152, 158

12, 13, 14, 15, 16, 17, 28, 45, 165

135

65, 68, 139, 140, 141,199

206

79,129

24,154

11,12, 13, 14, 15, 16, 17, 18, 19, 23, 57
126, 127,131

55, 98, 144,185

119

175

68, 77,101

33, 34, 35, 36, 37,38, 39, 51,106, 130, 164
43, 45, 46, 165, 177

63,79

197,198, 199, 200, 201, 202, 204, 205, 206

50,52

66, 68,131

25

135, 145

12, 25, 41,42, 43, 44, 45, 46, 55, 56, 71,

127, 176,180, 193

127, 147,148, 149, 150, 151, 152, 153, 154, 165, 176, 179
170

209

SEICCHRSE unsere
Grundrechte!

Wie? Indem Sie Mitmachen:

afd.de/mitglied werden

Oder Sie unte

für ein 1olo)uan]
Sjelzyote[c} Spen
steuerlich absetzbar.

Bankverbindung:
Alternative für Deutschland
Berliner Volksbank

IBAN: DE94 1loloicKofofeie, MTSWI:PIR) 940)
1:1(6% 1231477019) 3:7:)

Bitte geben Sie im Betreff mit an:
Spende + vollständige Adresse

bzw. AfD-ID

www.afd.de/spenden
Paypal@alternativefuer.de

"""

In [7]:
def change_text(text, additionals=[]):
    text = re.sub(" \d+\n", ".", text)
    text = re.sub("\n\d+", " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub("- ", "", text)
    text = re.sub("[0-9]", "", text)
    text = re.sub("()", "", text)
    for entry in additionals:
        text = re.sub(entry, "", text)
    sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
    return sentences

In [8]:
text1_list = change_text(text1, ["Demokratie und Rechtsstaat"])
text2_list = change_text(text2, ["Migration, Asyl und Integration"])
afd_list = change_text(text_afd)

In [9]:
def list_to_string(liste):
    sentence_string = ""
    for elem in liste:
        sentence_string += elem
        sentence_string += ". "
    return sentence_string

In [10]:
parser1 = PlaintextParser.from_string(list_to_string(text1_list), Tokenizer("german"))
parser2 = PlaintextParser.from_string(list_to_string(text2_list), Tokenizer("german"))
parser_afd = PlaintextParser.from_string(list_to_string(afd_list), Tokenizer("german"))
summarizer = TextRankSummarizer()

In [11]:
summary1 = summarizer(parser1.document, len(text1)//2000)
text1_summary = ""
for sentence in summary1:
    text1_summary+=str(sentence)
print(text1_summary)

Das Volk ist der Souverän  Die Regierungspolitiker in Bund und Ländern haben mit ihrer Flüchtlings-, Europaund Corona-Politik die Prinzipien der deutschen Staatlichkeit, des Rechts und der Verfassung vielfach verletzt.Der Wortbruch der etablierten Parteien bezüglich des Euros und die immer weitere Zentralisierung der EU bedrohen die Zukunft unseres Landes (sowie der nachfolgenden Generationen).Die Qualität der Entscheidung der Bürger muss gesichert werden durch einen ausgeprägten und autonomen Prozess der Willensbildung, durch Informationspflichten und -rechte und eine breite gesellschaftliche Debatte.Das ist die Lehre aus der Wiedervereinigung, aus der Ablehnung des „EU-Verfassungsvertrags durch das französische und das niederländische Volk und der Entscheidung des Vereinigten Königreichs, aus der Europäischen Union auszutreten.Die Gewaltenteilung gewährleisten  Zahlreiche Gesetze und die politische Praxis haben die  Gewaltenteilung in Deutschland als Kernelement des Rechtsstaats über

In [12]:
summary2 = summarizer(parser2.document, len(text2)//2000)
text2_summary = ""
for sentence in summary2:
    text2_summary+=str(sentence)
print(text2_summary)

Die Hilfe für Menschen in Not muss jedoch von der Aufnahme in Deutschland entkoppelt und in den Krisenregionen selbst geleistet werden, wo dies weit effektiver ist und viel mehr wirklich Hilfsbedürftige erreicht.Die unlösbaren Folgeprobleme  Die desaströsen Folgen der unregulierten Massenzu wanderung seit  sind unübersehbar und verschär fen sich weiter: überproportionale Zuwandererkrimi nalität, Terroranschläge und islamischer Separatismus  belegen dies ebenso wie dreistellige Milliardenkosten, Wohnraummangelund die hohe Arbeitslosigkeitunterr  den Zuwanderern.Solange ein dauerhafter und wirksamer Schutz der EU-Außengrenzen fehlt und die untaugliche überstaatliche Rechtslage fortbesteht, muss Deutschland die Kontrolle der Grenzen und die Unterbindung illegaler Grenzüberschreitungen selbst in die Hand nehmen.» Die Grenzen müssen zur Abwehr unerlaubter Zuwanderung und zur Kriminalitätsbekämpfung mit  Deutschland muss die einermodernen und wirksamen Sicherung ausgeKontrolle der Grenzen st

In [13]:
summary_afd = summarizer(parser_afd.document, 15)
text_afd_summary = ""
for sentence in summary_afd:
    text_afd_summary+=str(sentence)
print(text_afd_summary)

Demokratie und Rechtsstaat  Demokratie und Rechtsstaat  Das Volk ist der Souverän  Die Regierungspolitiker in Bund und Ländern haben mit ihrer Flüchtlings-, Europaund Corona-Politik die Prinzipien der deutschen Staatlichkeit, des Rechts und der Verfassung vielfach verletzt.Die Gewaltenteilung gewährleisten  Zahlreiche Gesetze und die politische Praxis haben die  Gewaltenteilung in Deutschland als Kernelement des Rechtsstaats über die Jahre in Gefahr gebracht und zu einer überbordenden Staatsgewalt geführt: Ehemalige Politiker auf Richterstühlen, Abgeordnete, die zugleich Kanzler, Ministerpräsident oder Minister sind und sich insoweit selbst kontrollieren, sowie parteipolitische Netzwerke, die durch verbotene, verfassungswidrige Ämterpatronage entstehen, können nicht länger toleriert werden.Auf  Landesebene sollten die Biersteuer und auf der Kommunalebene die Vergnügungssteuer, die Schankerlaubnissteuer, die Jagdund Fischereisteuer und die Zweitwohnungssteuer entfallen.Zusätzliche Infla